In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import os
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=100)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.01):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        Re_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+10
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
#         nn.init.sparse_(self.PFC.weight_ih.data,sparse)
#         nn.init.sparse_(self.HPC.weight_ih.data,sparse)
#         nn.init.sparse_(self.PFC.weight_hh.data,sparse)
#         nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.5
        var = 0.5
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*const, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0
        var = 1
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.05
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
class MyLSTM_3lay(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM_3lay, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        sparse = 0.1
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input,hiddens[2])
        PFC_input = hiddens[2][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        vHPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        vHPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    

class MyLSTM_feedforward(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_feedforward, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+10
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.Linear(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input)
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0.02
        v = 0.02
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*v
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def hidden_size_list(self):
        return [self.hidden_size_PFC,self.hidden_size_HPC,self.hidden_size_Re]
    

class MyLSTM_RNN_uniPFC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=1):
        super(MyLSTM_RNN_uniPFC, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)
        # nn.init.normal_(self.PFC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.Re.weight_ih.data,0,0.1/10)
        # nn.init.normal_(self.PFC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.Re.weight_hh.data,0,1/10)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = hiddens[1][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0.02
        v = 0.02
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*v
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def hidden_size_list(self):
        return [self.hidden_size_PFC,self.hidden_size_HPC,self.hidden_size_Re]


def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq


def main(num):
    training_size = 200
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
#     model_path = 'model/ReModeltest_30reset3addlong_estref_L2_interRNNrand_Reinh_AddRe_OUT5_131_s'+str(num)+'_100_4.pth'
    model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s'+str(num)+'_100_1_2_3.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s3_100_3.pth'
#     model_path = 'model/ReModeltest_30reset3addlong_L2_interRNNrand_Reinh_AddRe_OUT5_131_s10_100_1.pth'
#     model_path = 'model/ReModeltest_30addlong_interRNNrand_Reinh_AddRe_OUT5_131_s3_100_2.pth'
#     model_path = 'model/ReModel_interRNNrand_AddRe_OUT5_long_s7_200_2_1_'+str(num)+'.pth'
#     model_path = 'model/ReModel_interRNNrand_AddRe_OUT5_171_s'+str(num)+'_100_1.pth'
#     model_path = 'model/ReModel_LSTMrand_noise_long131test_s'+str(num)+'_100_1.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s'+str(num)+'_100_4.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_Reinh_AddRe_OUT5_11311_s6_100_1.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_Reinh_AddRe_OUT5_H121_s'+str(num)+'_100_2.pth'
#     model_path = 'model/FeedModel_L2_interRNNrand_AddRe_OUT5_161_s'+str(num)+'_100_2.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_131_s10_100_1.pth'
#     model_path = 'model/ReModel_interRNNrand_diffHPC5_long131test_s'+str(num)+'_100_2.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_HPC5_OUT5_161_s'+str(num)+'_100_2.pth'
#     model_path = 'model/R+/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s'+str(num)+'_100_2.pth'
#     model_path = 'model/R+/ReModel_L2_interRNNrand_AddRe_OUT5_151_s3_100_1_epoch50.pth'
#     model_path = 'model/R+/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s3_100_2.pth'
#     model_path = 'model/R+2/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s1_100_2_epoch'+str(num*5+125)+'.pth'
#     model_path = 'model/R+2/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s1_100_2_epoch170.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s10_100_3_epoch'+str(num*5+150)+'.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s10_100_6_epoch'+str(num*5+100)+'.pth'

#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch105.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s1_100_2_epoch'+str(num*5+150)+'.pth'
#     model_path = 'model/H+/ReModel_L2_interRNNrand_AddHPC100_OUT5_151_s7_100_1_epoch'+str(num*5+75)+'.pth'
#     model_path = 'model/ReModel_interRNN_Yl_s5_200_4_2000.pth'
#     model_path = 'model/ReModel_interRNN_long131test_s2_100_1.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s1_100_2_TrainOUT_epoch9.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s6_100_1_TrainOUT_epoch'+str(num)+'.pth'
#     model_path = 'model/R20_uniPFC/ReModel_L2_interRNNrand_OUT1_uniPFC_151_s1_100_2_epoch'+str(num*5+150)+'.pth'
#     model_path = 'model/R20_uniPFC/ReModel_L2_interRNNrand_OUT1_uniPFC_151_s1_100_2_epoch'+str(num*5+150)+'.pth'
#     model_path = 'model/R20_feed/FeedModel_L2_interRNNrand_OUT1_151_s6_100_1_epoch'+str(num*5+150)+'.pth'



    
    filename = "primal_long131test"
    
    if os.path.exists(model_path):
        print(model_path)
    else:
        print("Not exist")
        return
    
#     train_x,train_y = mkOwnDataSet(training_size,filename,data_length)
    train_x,train_y = mkOwnDataSet_auto(training_size,5)
    

#     rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyLSTM_feedforward(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
        
#     for n, p in rnn.named_parameters():
#         if n == "HPC.weight_ih":
#             p.data[:,inputsize:].sub_(p.data[:,inputsize:])

    
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 1
    for k in range(data_limit):
#             print(data[k].shape)
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(torch.rand(10,2),hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*2):
#             output += torch.randn(10,2)*0.01
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    fig = plt.figure()
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:100,0,0],traj[:100,0,1])
    plt.plot(traj[100:,0,0],traj[100:,0,1])
    plt.show()
    
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], 1)
#     MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_w = np.array(p.data)
            if n == "linear.weight":
                Output_w = np.array(p.data)

    fig2 = plt.figure(figsize=(10,5))
    ax1 = fig2.add_subplot(141)
    ax2 = fig2.add_subplot(142)
    axre = fig2.add_subplot(143)
    axout = fig2.add_subplot(144)
    
    ax1.imshow(PFC_w,cmap="coolwarm")
    ax2.imshow(HPC_w,cmap="coolwarm")
    axre.imshow(Re_w,cmap="coolwarm")
    axout.imshow(Output_w,cmap="coolwarm")
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
    axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_w),np.min(Re_w)))
    axout.set_title("max = {:.2f},min = {:.2f}".format(np.max(Output_w),np.min(Output_w)))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_hh":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_w = np.array(p.data)
                
    
    fig2 = plt.figure()
    ax1 = fig2.add_subplot(131)
    ax2 = fig2.add_subplot(132)
    axre = fig2.add_subplot(133)
    
    ax1.imshow(PFC_w,cmap="coolwarm")
    ax2.imshow(HPC_w,cmap="coolwarm")
    axre.imshow(Re_w,cmap="coolwarm",vmax=1,vmin=-1)
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
    axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_w),np.min(Re_w)))      
    
    
    fig3 = plt.figure()
    ax3 = fig3.add_subplot(131)
    ax4 = fig3.add_subplot(132)
    axre2 = fig3.add_subplot(133)
    # ax3.imshow(np.corrcoef(np.array(PFCstate)[:,0].T,np.array(HPCstate)[:,0].T))
    # ax4.imshow(np.corrcoef(np.array(HPCstate)[:,0].T,np.array(Restate).T))
    # axre2.imshow(np.corrcoef(np.array(PFCstate)[:,0].T,np.array(Restate).T))
    ax3.imshow(np.corrcoef(np.array(PFCstate)[:,0]),vmin=-1,vmax=1,cmap=plt.get_cmap("seismic"))
    ax4.imshow(np.corrcoef(np.array(HPCstate)[:,0]),vmin=-1,vmax=1,cmap=plt.get_cmap("seismic"))
    axre2.imshow(np.corrcoef(np.array(Restate)[:]),vmin=-1,vmax=1,cmap=plt.get_cmap("seismic"))
    ax3.set_title("PFC")   
    ax4.set_title("HPC")  
    axre2.set_title("Re")  
    
    
    pca = PCA()
    dfs = np.array(HPCstate)[0:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0],np.array(Restate)),axis=1)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    HPC_feature = np.copy(feature)
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
#     ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()

    
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(HPCstate)[:,0,i])
#         plt.show()

#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))
    
    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    PFC_feature = np.copy(feature)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
#     ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(PFCstate)[:,0,i])
#         plt.show()

#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))

    pca = PCA()
    dfs = np.array(Restate)
#     dfs = np.array(Restate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    Re_feature = np.copy(feature)
#     print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
#     print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
#     ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], alpha=0.8)
    ax3d.plot(moving_average(feature[:, 0]), moving_average(feature[:, 1]), moving_average(feature[:, 2]), alpha=0.8)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(Restate)[:,i])
#         plt.show()
    
#     linelist = search_delay(traj[:,0])
#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
    
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))
#     plt.vlines(linelist,0,1)

#     fig2d = plt.figure()
#     plt.plot(feature[:200, 0], feature[:200, 1], alpha=0.8)
#     plt.plot(feature[0:1, 0], feature[0:1, 1], "o", alpha=1)
#     plt.show()

    cellsize = 20
    lyap_exp_list = []
#     for i in range(cellsize):
#         lyap_exp = nolds.lyap_r(np.array(Restate)[:,i],min_tsep = 30, lag=10)
# #         lyap_exp2 = nolds.lyap_e(np.array(Restate)[:,i])
# #         lyap_exp = nolds.lyap_r(np.array(PFCstate)[:,0,i],min_tsep = 15, lag=1)
# #         lyap_exp = nolds.lyap_r(np.array(PFCstate)[:,0,i],min_tsep = 150, lag=55)
#         print(lyap_exp)
#         lyap_exp_list.append(lyap_exp)
    
#     return lyap_exp_list

    plt.figure()
    plt.plot(traj[:,:,0])
    # plt.plot(traj[:,:,1])
    
    
if __name__ == '__main__':
    features = []
    for i in range(10):
        features.append(main(i+1))

#     fig2d = plt.figure()
#     for feature in features:
#         plt.plot(feature[:200, 0], feature[:200, 1], alpha=0.2)
#         plt.plot(feature[0:1, 0], feature[0:1, 1], "o", alpha=1)
#     plt.show()

#     fig2d = plt.figure()
#     for i,feature in enumerate(features):
#         plt.plot(np.ones(len(feature))*(i+1), feature, "o", alpha=1)
#     plt.show()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        else:
            traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=100)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=100)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.01):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        Re_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.RNNCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        sparse = 10
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*var, torch.rand(self.batch_size, self.hidden_size)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0
        var = 1
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size)*const, torch.ones(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*var, torch.rand(self.batch_size, self.hidden_size)*var]
        Re_hidden = torch.ones(self.batch_size, self.hidden_size)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def hidden_randinsert(self,hidden):
        var = 1
        insert = [torch.rand(self.batch_size, self.hidden_size)*var, torch.rand(self.batch_size, self.hidden_size)*var]
#         insert = torch.rand(self.batch_size, self.hidden_size)*var
        return [hidden[0],insert,hidden[2]]
    
class MyLSTM_3lay(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM_3lay, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        sparse = 0.1
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input,hiddens[2])
        PFC_input = hiddens[2][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        vHPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        vHPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]
    

def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq


    

def main(num):
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
#     model_path = 'model/ReModel_interRNN_long131test_s'+str(num)+'_100_3.pth'
#     model_path = 'model/ReModel_interRNN_long131test_s1_100_3_plus_'+str(num)+'.pth'
    model_path = 'model/ReModel_interRNN_long131test_s1_100_1.pth'
    filename = "primal_long131test"
    
    if os.path.exists(model_path):
        print(model_path)
    else:
        print("Not exist")
        return
    
    train_x,train_y = mkOwnDataSet(training_size,filename,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
        


    
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 20
    for k in range(data_limit):
#             print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(torch.rand(10,2),hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    hidden = rnn.hidden_randinsert(hidden)
    for k in range(data.shape[0]*4):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    fig = plt.figure()
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:100,0,0],traj[:100,0,1])
    plt.plot(traj[100:,0,0],traj[100:,0,1])
    plt.show()
    
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

#     for n, p in rnn.named_parameters():
#             if n == "PFC.weight_ih":
#                 PFC_w = np.array(p.data)
#             if n == "HPC.weight_ih":
#                 HPC_w = np.array(p.data)
#             if n == "Re.weight_ih":
#                 Re_w = np.array(p.data)

#     fig2 = plt.figure()
#     ax1 = fig2.add_subplot(131)
#     ax2 = fig2.add_subplot(132)
#     axre = fig2.add_subplot(133)
    
#     ax1.imshow(PFC_w,cmap="coolwarm")
#     ax2.imshow(HPC_w,cmap="coolwarm")
#     axre.imshow(Re_w,cmap="coolwarm")
#     ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
#     ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
#     axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_w),np.min(Re_w)))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_hh":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_w = np.array(p.data)
    
    fig2 = plt.figure()
    ax1 = fig2.add_subplot(131)
    ax2 = fig2.add_subplot(132)
    axre = fig2.add_subplot(133)
    
    ax1.imshow(PFC_w,cmap="coolwarm")
    ax2.imshow(HPC_w,cmap="coolwarm")
    axre.imshow(Re_w,cmap="coolwarm")
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
    axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_w),np.min(Re_w)))                
    
    
    pca = PCA()
    dfs = np.array(HPCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
    ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    ax3d.plot(feature[20:21, 0], feature[20:21, 1], feature[20:21, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(HPCstate)[:,0,i])
#         plt.show()

#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))
    
    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
    ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    ax3d.plot(feature[20:21, 0], feature[20:21, 1], feature[20:21, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(PFCstate)[:,0,i])
#         plt.show()

#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))

    pca = PCA()
    dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
    ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    ax3d.plot(feature[20:21, 0], feature[20:21, 1], feature[20:21, 2],"o", alpha=1)
    ax3d.plot(moving_average(feature[:, 0]), moving_average(feature[:, 1]), moving_average(feature[:, 2]), alpha=0.8)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(Restate)[:,i])
#         plt.show()
    
#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
    
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))
    
    
if __name__ == '__main__':
    for i in range(10):
        main(i+1)

In [ ]:

def main(num):
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 200
    inputsize = 2
    outputsize = 2
#     model_path = 'model/ReModel_mixRNN_long_s'+str(num)+'_200_1.pth'
#     model_path = 'model/ReModel_interRNN_Yl_s5_200_5_2000.pth'
    filename = "primal_Y_long"
    print(model_path)
    
    train_x,train_y = mkOwnDataSet(training_size,filename,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)

     
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden()
    data_limit = 200
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*1):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))

    model_path = 'model/ReModel_interRNN_long_s5_200_1.pth'
    filename = "primal_long"
    print(model_path)
    
    train_x,train_y = mkOwnDataSet(training_size,filename,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    hidden = rnn.initHidden()
            
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*1):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))


    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
    
    pca = PCA()
    dfs = np.array(HPCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:200, 0], feature[:200, 1], feature[:200, 2], alpha=0.8)
    plt.show()
    ax3d.plot(feature[200:, 0], feature[200:, 1], feature[200:, 2], alpha=0.8)
    plt.show()
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(HPCstate)[:,0,i])
#         plt.show()
    
    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    print(feature[200:])
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:200, 0], feature[:200, 1], feature[:200, 2], alpha=0.8)
    plt.show()
    ax3d.plot(feature[200:, 0], feature[200:, 1], feature[200:, 2], alpha=0.8)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(PFCstate)[:,0,i])
#         plt.show()

    pca = PCA()
    dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:200, 0], feature[:200, 1], feature[:200, 2], alpha=0.8)
    plt.show()
    ax3d.plot(feature[200:, 0], feature[200:, 1], feature[200:, 2], alpha=0.8)
    plt.show()
#     ax3d.plot(moving_average(feature[:, 0]), moving_average(feature[:, 1]), moving_average(feature[:, 2]), alpha=0.8)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(Restate)[:,i])
#         plt.show()
    
#     moving_feature = np.concatenate(([moving_average(feature[:, 0])], [moving_average(feature[:, 1])], [moving_average(feature[:, 2])]),axis=0).T
#     print(moving_feature.shape)
#     plot_distance(feature,moving_feature)
    
#     plt.figure()
#     plt.plot(overlap_coefficient(moving_feature,feature))
#     plt.plot(moving_average(overlap_coefficient(moving_feature,feature)))
    
    
if __name__ == '__main__':
    for i in range(1):
        main(i+1)

In [ ]:
def check_distance(list_a,list_b):
    result = []
    for point in list_b:
        dis_list = []
        for target in list_a:
            dis_list.append(np.linalg.norm(point[:3] - target[:3]))
        result.append(np.min(dis_list))
    return np.array(result)

def overlap_coefficient(list_a,list_b):
    threshold = 0.025
    #Aの各点からBにある任意の点の最小距離を計算
    distance = check_distance(list_a,list_b)
    
    #集合Aの要素数を取得
    num_listA = len(list_a)
    return distance

def plot_distance(data1,data2):
    r = data1
    l = data2
    
    overlap=overlap_coefficient(r,l)

    plt.figure()
#     plt.ylim(0,1)
    plt.plot(overlap)

In [ ]:

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long_s9_200_1.pth'

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    for n, p in rnn.named_parameters():
        # if n == "PFC.weight_ih":
        #     p.data[hidden_size:,hidden_size:].sub_(p.data[hidden_size:,hidden_size:])
        if n == "HPC.weight_ih":
            p.data[:,inputsize:].sub_(p.data[:,inputsize:])
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    xnum = 100
    ynum = 100
    initx = 0.2
    endx = 0.8
    widex = (endx-initx)/xnum
    
    inity = 0.0
    endy = 0.6
    widey = (endy-inity)/ynum
    
    place = np.zeros((hidden_size,xnum+1,ynum+1))
    grid = [[] for i in range(hidden_size)]
    data_limit = 20
    for y in range(ynum+1):
        for x in range(xnum+1):
            hidden = rnn.initHidden()
            point = torch.tensor([[initx+widex*x,widey*y] for i in range(batch_size)])
            for k in range(data_limit):
                    #print(data[k].shape)
                    output,hidden = rnn(point,hidden)
            for n in range(hidden_size):
                place[n][y][x] = hidden[1][0].tolist()[0][n]
                grid[n].append([initx+widex*x,widey*y,hidden[1][0].tolist()[0][n]])
                
    grid = np.array(grid)
    for i in range(hidden_size):
        fig = plt.figure()
        plt.imshow(place[i],cmap='jet',origin='lower',interpolation='bilinear', vmin=0)
        plt.show()
        
#         fig = plt.figure()
#         ax = fig.add_subplot(111, projection='3d')
#         ax.scatter(grid[i,:,0],grid[i,:,1],grid[i,:,2])
#         plt.show()
        
        fig = plt.figure()
        x = np.arange(initx, endx+widex, widex)
        y = np.arange(inity, endy+widey, widey)
        x, y = np.meshgrid(x, y)
        z = place[i]
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(x, y, z, cmap='jet')
        plt.show()
    
    #MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
    
if __name__ == '__main__':
    main()

In [ ]:
#init ari


def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t],cmap='jet',origin='lower',interpolation='bilinear')
        ims.append([img])
    ani = animation.ArtistAnimation(fig, ims, interval=50)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long_s5_200_1.pth'

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
#     for n, p in rnn.named_parameters():
#         # if n == "PFC.weight_ih":
#         #     p.data[hidden_size:,hidden_size:].sub_(p.data[hidden_size:,hidden_size:])
#         if n == "HPC.weight_ih":
#             p.data[hidden_size:,inputsize:].sub_(p.data[hidden_size:,inputsize:])
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    xnum = 20
    ynum = 20
    initx = 0
    endx = 1
    widex = (endx-initx)/xnum
    
    inity = 0.0
    endy = 0.6
    widey = (endy-inity)/ynum
    
    data_limit = 100
    place = np.zeros((data_limit,hidden_size,xnum+1,ynum+1))
    grid = [[] for i in range(hidden_size)]
    
    x_list = np.array(range(xnum+1))
    y_list = np.array(range(ynum+1))
    np.random.shuffle(y_list)
    for y in y_list:
        np.random.shuffle(x_list)
        for x in x_list:
            hidden = rnn.initHidden()
            point = torch.tensor([[initx+widex*x,widey*y] for i in range(batch_size)])
            for k in range(data_limit):
                    #print(data[k].shape)
                    output,hidden = rnn(point,hidden)
                    for n in range(hidden_size):
                        place[k][n][y][x] = hidden[1][0].tolist()[0][n]

    for i in range(hidden_size):
        fig = plt.figure()
        plt.imshow(place[0][i],cmap='jet',origin='lower',interpolation='bilinear')
        plt.show()
        
        MakeAnimation_img(place[:,i],"test!!!"+str(i))
        
    
if __name__ == '__main__':
    main()

In [ ]:
#init nashi

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long_s5_200_1.pth'

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    for n, p in rnn.named_parameters():
        if n == "HPC.weight_ih":
            p.data[:,inputsize:].sub_(p.data[:,inputsize:])
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    xnum = 50
    ynum = 50
    initx = 0.2
    endx = 0.8
    widex = (endx-initx)/xnum
    
    inity = 0.0
    endy = 0.6
    widey = (endy-inity)/ynum
    
    place = np.zeros((hidden_size,xnum+1,ynum+1))
    grid = [[] for i in range(hidden_size)]
    hidden = rnn.initHidden()
    
    y_list = np.array(range(ynum+1))
    np.random.shuffle(y_list)
    for y in y_list:
        x_list = np.array(range(xnum+1))
        np.random.shuffle(x_list)
        for x in x_list:
            #if y % 2 == 0: x = xnum - x
            point = torch.tensor([[initx+widex*x,widey*y] for i in range(batch_size)])
            output,hidden = rnn(point,hidden)
            for n in range(hidden_size):
                place[n][y][x] = hidden[1][0].tolist()[0][n]
                grid[n].append([initx+widex*x,widey*y,hidden[1][0].tolist()[0][n]])
                
    grid = np.array(grid)
    for i in range(hidden_size):
        fig = plt.figure()
        plt.imshow(place[i],cmap='jet',origin='lower',interpolation='bilinear')
        plt.show()
        
#         fig = plt.figure()
#         ax = fig.add_subplot(111, projection='3d')
#         ax.scatter(grid[i,:,0],grid[i,:,1],grid[i,:,2])
#         plt.show()
        
        fig = plt.figure()
        x = np.arange(initx, endx+widex, widex)
        y = np.arange(inity, endy+widey, widey)
        x, y = np.meshgrid(x, y)
        z = place[i]
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(x, y, z, cmap='jet')
        plt.show()
    
    #MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from networkx.algorithms import bipartite

%matplotlib notebook

def mkOwnDataSet(data_size, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt("primal_long_r.csv",delimiter=',')
    y = np.loadtxt("primal_long_l.csv",delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 200
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long131test_s1_100_3.pth'

    train_x,train_y = mkOwnDataSet(training_size,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     

    target = [40,60]
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)[target[0]:target[1],0:20]
                PFC_w /= np.max(PFC_w)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)[target[0]:target[1],inputsize:]
                HPC_w /= np.max(HPC_w)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)[0:20,0:20]
                Re_w /= np.max(Re_w)
                
    fig = plt.figure()
    G = nx.from_numpy_matrix(np.matrix(PFC_w), create_using=nx.DiGraph)
    layout = nx.spring_layout(G)
    nx.draw(G, layout)
    plt.show()
    
    fig2 = plt.figure()
    ax1 = fig2.add_subplot(121)
    ax2 = fig2.add_subplot(122)
    
    ax1.imshow(PFC_w,cmap="coolwarm")
    ax2.imshow(HPC_w,cmap="coolwarm")
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
    
    plt.figure()
    A = nx.Graph()
    PFCnode = [("PFC"+str(i)) for i in range(20)]
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    Renode = [("Re"+str(i)) for i in range(20)]
    A.add_nodes_from(PFCnode, bipartite=0)
    A.add_nodes_from(HPCnode, bipartite=1)
    A.add_nodes_from(Renode, bipartite=2)
    for i in range(20):
        for k in range(20):
            if PFC_w[i,k]>0:
                A.add_edge(PFCnode[i],HPCnode[k],weight=PFC_w[i,k])
            if Re_w[i,k]>0:
                A.add_edge(Renode[i],PFCnode[k],weight=Re_w[i,k])
    pos = {}
    pos.update((node, (2, index)) for index, node in enumerate(PFCnode))
    pos.update((node, (1, index)) for index, node in enumerate(HPCnode))
    pos.update((node, (3, index)) for index, node in enumerate(Renode))
    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.eigenvector_centrality(A,weight=True)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, width=1.0, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
    print(np.count_nonzero(PFC_w>0.3, axis=0),np.count_nonzero(HPC_w>0.3, axis=0),np.count_nonzero(Re_w>0.3, axis=0))
    
    plt.figure()
    A = nx.Graph()
    PFCnode = [("PFC"+str(i)) for i in range(20)]
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    PFC2node = [("Re"+str(i)) for i in range(20)]
    A.add_nodes_from(PFCnode, bipartite=0)
    A.add_nodes_from(HPCnode, bipartite=1)
    A.add_nodes_from(Renode, bipartite=2)
    for i in range(20):
        for k in range(20):
            if HPC_w[i,k]>0.5:
                A.add_edge(HPCnode[i],PFCnode[k],weight=HPC_w[i,k])
            if PFC_w[i,k]>0.5:
                A.add_edge(PFC2node[i],HPCnode[k],weight=PFC_w[i,k])

    pos = {}
    pos.update((node, (1, index)) for index, node in enumerate(PFCnode))
    pos.update((node, (2, index)) for index, node in enumerate(HPCnode))
    pos.update((node, (3, index)) for index, node in enumerate(PFC2node))
    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.eigenvector_centrality(A,weight=True)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, width=1.0, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
    plt.figure()
    B = nx.Graph()
    PFCnode = [("PFC"+str(i)) for i in range(20)]
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    B.add_nodes_from(PFCnode, bipartite=0)
    B.add_nodes_from(HPCnode, bipartite=1)
    for i in range(20):
        for k in range(20):
            if PFC_w[i,k]<0.8:
                B.add_edge(PFCnode[i],HPCnode[k],weight=0)
            else:
                B.add_edge(PFCnode[i],HPCnode[k],weight=PFC_w[i,k])
    bottom_nodes, top_nodes = bipartite.sets(B)
    pos = {}
    pos.update((node, (2, index)) for index, node in enumerate(bottom_nodes))
    pos.update((node, (1, index)) for index, node in enumerate(top_nodes))
    print(pos)
    edges,weights = zip(*nx.get_edge_attributes(B,'weight').items())
    nx.draw(B, pos=pos, node_color='b', edgelist=edges, edge_color=weights, width=1.0, edge_cmap=plt.cm.Reds)
    plt.show()
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from networkx.algorithms import bipartite

%matplotlib notebook

def mkOwnDataSet(data_size, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt("primal0_r.csv",delimiter=',')
    y = np.loadtxt("primal0_l.csv",delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/v4_2Model_0_3.pth'

    train_x,train_y = mkOwnDataSet(training_size,data_length)

    rnn = MyLSTM(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    traj = []
    PFCstate = []
    HPCstate = []
    Gate_states = []
    hidden = rnn.initHidden()
    data_limit = 100
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*3):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            #print(output)
            Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    fig = plt.figure()
    print(pltdata.shape)
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:,0,0],traj[:,0,1])
    plt.show()
    
    
    print(np.array(PFCstate)[:,0].shape)
    #MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)[0:20,:]
                PFC_w /= np.max(PFC_w)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)[0:20,inputsize:]
                HPC_w /= np.max(HPC_w)
    
    plt.figure()
    A = nx.Graph()
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    A.add_nodes_from(HPCnode, bipartite=0)
    pos = {}
    pos.update((node, (1, index)) for index, node in enumerate(HPCnode))
    for t in range(1,10):
        if t % 2 == 1:
            PFCnode = [("PFC"+str((t//2)*20+i)) for i in range(20)]
            A.add_nodes_from(PFCnode, bipartite=t+1)
            pos.update((node, (t+1, index)) for index, node in enumerate(PFCnode))
            PFC = True
            HPC = False
        else:
            HPCnode = [("HPC"+str((t//2)*20+i)) for i in range(20)]
            A.add_nodes_from(HPCnode, bipartite=t+1)
            pos.update((node, (t+1, index)) for index, node in enumerate(HPCnode))
            HPC = True
            PFC = False
        for i in range(20):
            for k in range(20):
                if PFC and PFC_w[i,k]>0.5:
                    A.add_edge(PFCnode[i],HPCnode[k],weight=PFC_w[i,k])
                if HPC and HPC_w[i,k]>0.5:
                    A.add_edge(HPCnode[i],PFCnode[k],weight=HPC_w[i,k])
    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.degree_centrality(A)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from networkx.algorithms import bipartite

%matplotlib notebook

def mkOwnDataSet(data_size, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt("primal0_r.csv",delimiter=',')
    y = np.loadtxt("primal0_l.csv",delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long131test_s1_100_3.pth'

    train_x,train_y = mkOwnDataSet(training_size,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)[0:20,:]
                PFC_w /= np.max(PFC_w)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)[0:20,inputsize:]
                HPC_w /= np.max(HPC_w)
    
    plt.figure()
    A = nx.Graph()
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    pos = {}
    pos.update((node, (1, index)) for index, node in enumerate(HPCnode))
    for t in range(1,10):
        if t % 2 == 1:
            PFCnode = [("PFC"+str((t//2)*20+i)) for i in range(20)]
            PFC = True
            HPC = False
        else:
            HPCnode = [("HPC"+str((t//2)*20+i)) for i in range(20)]
            HPC = True
            PFC = False
        for i in range(20):
            for k in range(20):
                if PFC and PFC_w[i,k]>0.5:
                    A.add_edge(PFCnode[i],HPCnode[k],weight=PFC_w[i,k])
                    pos.update({PFCnode[i]: (t+1, i)})
                    pos.update({HPCnode[k]: (t, k)})
                if HPC and HPC_w[i,k]>0.5:
                    A.add_edge(HPCnode[i],PFCnode[k],weight=HPC_w[i,k])
                    pos.update({HPCnode[i]: (t+1, i)})
                    pos.update({PFCnode[k]: (t, k)})
    
    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.degree_centrality(A)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
    
    plt.figure()
    A = nx.Graph()
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    pos = {}
    pos.update((node, (1, index)) for index, node in enumerate(HPCnode))
    initnode = 12
    nodelinks = [HPCnode[initnode]]
    for t in range(1,10):
        if t % 2 == 1:
            PFCnode = [("PFC"+str((t//2)*20+i)) for i in range(20)]
            PFC = True
            HPC = False
        else:
            HPCnode = [("HPC"+str((t//2)*20+i)) for i in range(20)]
            HPC = True
            PFC = False
        for i in range(20):
            for k in range(20):
                if PFC and PFC_w[i,k]>0.5 and HPCnode[k] in nodelinks:
                    A.add_edge(PFCnode[i],HPCnode[k],weight=PFC_w[i,k])
                    pos.update({PFCnode[i]: (t+1, i)})
                    pos.update({HPCnode[k]: (t, k)})
                    nodelinks.append(PFCnode[i])
                if HPC and HPC_w[i,k]>0.5 and PFCnode[k] in nodelinks:
                    A.add_edge(HPCnode[i],PFCnode[k],weight=HPC_w[i,k])
                    pos.update({HPCnode[i]: (t+1, i)})
                    pos.update({PFCnode[k]: (t, k)})
                    nodelinks.append(HPCnode[i])

    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.degree_centrality(A)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from networkx.algorithms import bipartite

%matplotlib notebook

def mkOwnDataSet(data_size, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt("primal0_r.csv",delimiter=',')
    y = np.loadtxt("primal0_l.csv",delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/v4_2Model_0_3.pth'

    train_x,train_y = mkOwnDataSet(training_size,data_length)

    rnn = MyLSTM(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    traj = []
    PFCstate = []
    HPCstate = []
    Gate_states = []
    hidden = rnn.initHidden()
    data_limit = 100
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*3):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            #print(output)
            Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    fig = plt.figure()
    print(pltdata.shape)
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:,0,0],traj[:,0,1])
    plt.show()
    
    
    print(np.array(PFCstate)[:,0].shape)
    #MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)[0:20,:]
                PFC_w /= np.max(PFC_w)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)[0:20,inputsize:]
                HPC_w /= np.max(HPC_w)
    
    plt.figure()
    A = nx.DiGraph()
    HPCnode = [("HPC"+str(i)) for i in range(20)]
    A.add_nodes_from(HPCnode, bipartite=0)
    pos = {}
    pos.update((node, (1, index)) for index, node in enumerate(HPCnode))
    for t in range(1,10):
        if t % 2 == 1:
            PFCnode = [("PFC"+str((t//2)*20+i)) for i in range(20)]
            A.add_nodes_from(PFCnode, bipartite=t+1)
            pos.update((node, (t+1, index)) for index, node in enumerate(PFCnode))
            PFC = True
            HPC = False
        else:
            HPCnode = [("HPC"+str((t//2)*20+i)) for i in range(20)]
            A.add_nodes_from(HPCnode, bipartite=t+1)
            pos.update((node, (t+1, index)) for index, node in enumerate(HPCnode))
            HPC = True
            PFC = False
        for i in range(20):
            for k in range(20):
                if PFC and PFC_w[i,k]>0.5:
                    A.add_edge(HPCnode[k],PFCnode[i],weight=PFC_w[i,k])
                if HPC and HPC_w[i,k]>0.5:
                    A.add_edge(PFCnode[k],HPCnode[i],weight=HPC_w[i,k])
    sourcenode = 'HPC12'
    targetnode = PFCnode[14]
#     links = list(nx.shortest_path(A,target=targetnode).keys())
    links = list(nx.shortest_path(A,source=sourcenode).keys())
    for node in list(A.nodes(data=False)):
        if node not in links:
            A.remove_node(node)
            pos.pop(node)
    
    
    edges,weights = zip(*nx.get_edge_attributes(A,'weight').items())
    measures = nx.degree_centrality(A)
    nx.draw(A, pos=pos, nodelist=list(measures.keys()), node_color=list(measures.values()),  edgelist=edges, edge_color=weights, edge_cmap=plt.cm.Reds, with_labels=True)
    plt.show()
    
if __name__ == '__main__':
    main()

In [ ]:
import glob

# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s4_100_4.pth')
# model_list = glob.glob('model/R+/ReModel_L2_interRNNrand_AddRe_OUT5_151_s3_100_*.pth')
# model_list = glob.glob('model/R+2/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s1_100_1_epoch*.pth')
model_list = glob.glob('model/R20_H_bigbatch/ReModel_L2_interRNNrand_OUT1_121_s*_100_1_epoch190.pth')
# model_list = glob.glob('model/R20_H_uniPFC_bigbatch/ReModel_L2_interRNNrand_OUT1_121_s*_100_1_epoch190.pth')




model_list = sorted(model_list)
print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [4,5]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

target_list = model_list[:]

weight_list_Re = []
weight_list_PFC = []

for model_path in target_list:
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyLSTM_RNN_uniPFC(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "linear.weight":
                OUT_w = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)

#     fig1.hist(PFC_w[PFC_w.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True)
#     fig1.hist(PFC_inw[PFC_inw.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True)  
#     fig2.hist(HPC_inw[HPC_inw.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True)  
#     fig3.hist(Re_w[Re_w.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True) 
#     fig3.hist(Re_inw[Re_inw.nonzero()],bins=40,range=(-4,4),alpha=0.5,density=True) 
    
    print(PFC_w[PFC_w.nonzero()])
#     fig1.hist(PFC_w[PFC_w.nonzero()],bins=40,alpha=0.5,density=True)
    fig1.hist(PFC_inw[PFC_inw.nonzero()],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.append(PFC_w[PFC_w.nonzero()],PFC_inw[PFC_inw.nonzero()]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w.nonzero()],bins=40,alpha=0.5,density=True)  
    fig2.hist(HPC_inw[HPC_inw.nonzero()],bins=40,alpha=0.5,density=True)  
#     fig2.hist(np.append(HPC_w[HPC_w.nonzero()],HPC_inw[HPC_inw.nonzero()]),bins=40,alpha=0.5,density=True)
#     fig3.hist(Re_w[Re_w.nonzero()],bins=40,alpha=0.5,density=True) 
    fig3.hist(Re_inw[Re_inw.nonzero()],bins=40,alpha=0.5,density=True) 
#     fig3.hist(Re_inw,bins=40,alpha=0.5,density=True) 
#     fig3.hist(OUT_w[OUT_w.nonzero()],bins=40,alpha=0.5,density=True) 

    
#     fig1.hist(PFC_b[PFC_b.nonzero()],bins=20,alpha=0.5,density=False)
#     fig1.hist(PFC_inb[PFC_inb.nonzero()],bins=10,alpha=0.5,density=False)
#     fig2.hist(HPC_b[HPC_b.nonzero()],bins=20,alpha=0.5,density=False)  
#     fig2.hist(HPC_inb[HPC_inb.nonzero()],bins=10,alpha=0.5,density=False)  
#     fig3.hist(Re_b[Re_b.nonzero()],bins=20,alpha=0.5,density=False) 
#     fig3.hist(Re_inb[Re_inb.nonzero()],bins=10,alpha=0.5,density=False) 

    weight_list_Re.append(Re_inw)
    weight_list_PFC.append(PFC_w)

# plt.figure()
# plt.imshow(-weight_list_PFC[0]+weight_list_PFC[1],cmap="coolwarm")
# plt.figure()
# plt.imshow(-weight_list_PFC[1]+weight_list_PFC[2],cmap="coolwarm")
# plt.figure()
# plt.imshow(-weight_list_Re[0]+weight_list_Re[2],cmap="coolwarm")

In [ ]:
import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
# model_list = glob.glob('model/R20/*s4_100_1*.pth')
# model_list = glob.glob('model/R20_H_bigbatch/ReModel_L2_interRNNrand_OUT1_121_s7_100_3_epoch*.pth')
model_list = glob.glob('model/R20_H_uniPFC_bigbatch/ReModel_L2_interRNNrand_OUT1_121_s1_100_1_epoch*.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len)

print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [19,22]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig_eigen = plt.figure()
fig_eigenval = fig_eigen.add_subplot(121)
fig_wcorr_eigen = fig_eigen.add_subplot(122)

target_list = model_list[1:]
PCA_list = np.array([[0,1,2]])

num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
    print(model_path)
#     rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN_uniPFC(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
    
#     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w>0]),bins=80,density=True)
#     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw>0]),bins=40,density=True)
#     fig1.plot((make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
#     fig1.plot((make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
#     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w<0]),bins=80,density=True)
#     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw<0]),bins=40,density=True)
    PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w.nonzero()]),bins=80,density=True)
    PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw.nonzero()]),bins=40,density=True)
#     PFC_hist = np.histogram(np.abs(np.append(PFC_w[PFC_w.nonzero()],PFC_inw[PFC_inw.nonzero()])),bins=40,density=True)
#     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b.nonzero()]),bins=80,density=True)
#     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb.nonzero()]),bins=40,density=True)
    fig1.plot((make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
#     fig1.plot((make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
#     fig1.plot((make_bins(PFC_hist[1])),np.log(PFC_hist[0]))
    a1 = regression(make_bins(PFC_w_hist[1]),np.log(PFC_w_hist[0]+0.01))
    a2 = regression(make_bins(PFC_inw_hist[1]),np.log(PFC_inw_hist[0]+0.01))


#     HPC_w_hist = np.histogram(HPC_w[HPC_w>0],bins=80,density=True)
#     HPC_inw_hist = np.histogram(HPC_inw[HPC_inw>0],bins=40,density=True)
#     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
#     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
#     HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w<0]),bins=80,density=True)
#     HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw<0]),bins=40,density=True)
    HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w.nonzero()]),bins=80,density=True)
    HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw.nonzero()]),bins=40,density=True)
#     HPC_w_hist = np.histogram(np.abs(HPC_w),bins=80,density=True)
#     HPC_inw_hist = np.histogram(np.abs(HPC_inw),bins=40,density=True)
#     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
#     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
#     fig2.plot(np.log(make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]+0.01),color=color)
    fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]+0.01),color=color)
    b1 = regression(make_bins(HPC_w_hist[1]),np.log(HPC_w_hist[0]+0.01))
    b2 = regression(make_bins(HPC_inw_hist[1]),np.log(HPC_inw_hist[0]+0.01))


#     Re_w_hist = np.histogram(Re_w[Re_w>0],bins=100,density=True)
#     Re_inw_hist = np.histogram(Re_inw[Re_inw>0],bins=100,density=True)
#     Re_w_hist = np.histogram(np.abs(Re_w[Re_w<0]),bins=100,density=True)
#     Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw<0]),bins=20,density=True)
    Re_w_hist = np.histogram(np.abs(Re_w[Re_w.nonzero()]),bins=40,density=True)
    Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw.nonzero()]),bins=40,density=True)
#     Re_inw_hist = np.histogram(np.abs(Re_inw),bins=40,density=True)
    Re_initw_hist = np.histogram(np.abs(Re_initw),bins=40,density=True)
#     fig3.plot(np.log(make_bins(Re_w_hist[1])),np.log(Re_w_hist[0]+0.01),color=color)
    fig3.plot((make_bins(Re_inw_hist[1])),np.log(Re_inw_hist[0]+0.01),color=color)
#     fig3.plot(np.log(make_bins(Re_initw_hist[1])),np.log(Re_initw_hist[0]+0.01))
    c1 = regression(make_bins(Re_w_hist[1]),np.log(Re_w_hist[0]+0.01))
    c2 = regression(make_bins(Re_inw_hist[1]),np.log(Re_inw_hist[0]+0.01))
    
    Re_inuw,Re_inuv = LA.eig(Re_inw)
#     fig_eigenval.plot(Re_inuw.real,Re_inuw.imag,"o",color=color)
#     print(Re_inuw)
    i = 0
    PFC_inuw,PFC_inuv = LA.eig(PFC_inw[i:i+20]*PFC_inw[i+40:i+60])
#     print(PFC_inuw,PFC_inuv)
#     fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=color)
#     for k in range(20):
#         fig_eigenval.text(PFC_inuw.real[k],PFC_inuw.imag[k],k)
#     print(LA.matrix_rank(PFC_inw[i:i+20]))
#     print(np.count_nonzero(PFC_inuw.real>0))
    
    i = 0
    HPC_inuw,HPC_inuv = LA.eig(HPC_inw[i:i+20])
#     print(HPC_inuw,HPC_inuv)
#     fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color=color)
    fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=color)
#     for k in range(20):
#         fig_eigenval.text(HPC_inuw.real[k],HPC_inuw.imag[k],k)
#     print(LA.matrix_rank(HPC_inw[i:i+20]))
    print(np.count_nonzero(HPC_inuw.real>0))
    print(np.count_nonzero(HPC_inuw.imag==0))
    print(np.count_nonzero((HPC_inuw.imag==0)*(HPC_inuw.real>0)))
    
    
    PCA_list = np.concatenate((PCA_list,[[a1,a2,c2]]),axis=0)
    print(a1-a2)
    
#     fig1.hist(PFC_w[PFC_w>0],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.abs(PFC_w[PFC_w<0]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig2.hist(np.abs(HPC_w[HPC_w<0]),bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(Re_w[Re_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(np.abs(Re_w[Re_w<0]),bins=40,alpha=0.5,log=True,density=True)

    fig_wcorr_eigen.plot(np.mean(np.abs(PFC_inw)),np.mean(np.abs(PFC_inuw)),"o",color=color)

# # print(PCA_list[1:])
# pca = PCA()
# dfs = PCA_list[1:]
# pca.fit(dfs)
# feature = pca.transform(dfs)

# plt.figure(figsize=(6, 6))
# plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8)
# #plt.scatter(feature[100:200, 0], feature[100:200, 1], alpha=0.8)
# plt.grid()
# plt.xlabel("PC1")
# plt.ylabel("PC2")
# for i,n in enumerate(target_list):
#     plt.annotate(n[-12:],(feature[i, 0], feature[i, 1]))
# plt.show()

# fig3d = plt.figure()
# ax3d = Axes3D(fig3d)
# ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], "o", alpha=0.8)
# plt.show()

In [ ]:
import glob
import pandas as pd
from sklearn import linear_model

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
#     print(coef,score)
    return coef[0][0]

model_list = glob.glob('model/ReModel_interRNN_long_s*_200_?.pth')
print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

# NGlist_name = np.array(["s3_100_4","s1_100_3","s9_100_2","s4_100_1"])
NGlist = np.array([])

# for NG in NGlist_name:
#     for i, n in enumerate(model_list):
#         if n[-12:-4] == NG:
#             NGlist = np.append(NGlist,int(i))
# print(NGlist)
    
# target = [2,4,7,8,9]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

target_list = model_list
PCA_list = np.array([np.ones(4)])

for model_path in target_list:
    print(model_path)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
    
    PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w>0]),bins=40,density=True)
    PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw>0]),bins=20,density=True)
    a1 = regression(make_bins(PFC_w_hist[1]),np.log(PFC_w_hist[0]))
    a2 = regression(make_bins(PFC_inw_hist[1]),np.log(PFC_inw_hist[0]))
    PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w<0]),bins=40,density=True)
    PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw<0]),bins=20,density=True)
    a3 = regression(make_bins(PFC_w_hist[1]),np.log(PFC_w_hist[0]))
    a4 = regression(make_bins(PFC_inw_hist[1]),np.log(PFC_inw_hist[0]))

    HPC_w_hist = np.histogram(HPC_w[HPC_w>0],bins=40,density=True)
    HPC_inw_hist = np.histogram(HPC_inw[HPC_inw>0],bins=20,density=True)
    b1 = regression(make_bins(HPC_w_hist[1]),np.log(HPC_w_hist[0]))
    b2 = regression(make_bins(HPC_inw_hist[1]),np.log(HPC_inw_hist[0]))
    HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w<0]),bins=40,density=True)
    HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw<0]),bins=20,density=True)
    b3 = regression(make_bins(HPC_w_hist[1]),np.log(HPC_w_hist[0]))
    b4 = regression(make_bins(HPC_inw_hist[1]),np.log(HPC_inw_hist[0]))


    Re_w_hist = np.histogram(Re_w[Re_w>0],bins=40,density=True)
    Re_inw_hist = np.histogram(Re_inw[Re_inw>0],bins=20,density=True)
    c1 = regression(make_bins(Re_w_hist[1]),np.log(Re_w_hist[0]))
    c2 = regression(make_bins(Re_inw_hist[1]),np.log(Re_inw_hist[0]))
    Re_w_hist = np.histogram(np.abs(Re_w[Re_w<0]),bins=40,density=True)
    Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw<0]),bins=20,density=True)
    c3 = regression(make_bins(Re_w_hist[1]),np.log(Re_w_hist[0]))
    c4 = regression(make_bins(Re_inw_hist[1]),np.log(Re_inw_hist[0]))
    
#     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b>0]),bins=20,density=True)
#     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb>0]),bins=10,density=True)
#     a1 = regression(make_bins(PFC_b_hist[1]),np.log(PFC_b_hist[0]))
#     a2 = regression(make_bins(PFC_inb_hist[1]),np.log(PFC_inb_hist[0]))
#     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b<0]),bins=80,density=True)
#     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb<0]),bins=40,density=True)
#     a3 = regression(make_bins(PFC_b_hist[1]),np.log(PFC_b_hist[0]))
#     a4 = regression(make_bins(PFC_inb_hist[1]),np.log(PFC_inb_hist[0]))
    
    PCA_list = np.concatenate((PCA_list,[[a2,b2,a4,b4]]),axis=0)
    
#     fig1.hist(PFC_w[PFC_w>0],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.abs(PFC_w[PFC_w<0]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig2.hist(np.abs(HPC_w[HPC_w<0]),bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(Re_w[Re_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(np.abs(Re_w[Re_w<0]),bins=40,alpha=0.5,log=True,density=True)

# print(PCA_list[1:])
pca = PCA()
dfs = PCA_list[1:]
pca.fit(dfs)
feature = pca.transform(dfs)

plt.figure(figsize=(6, 6))
plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
for i,n in enumerate(target_list):
    plt.annotate(n[-12:],(feature[i, 0], feature[i, 1]))
for i in NGlist:
    i = int(i)
    plt.scatter(feature[i, 0], feature[i, 1], color="r", alpha=0.8)
plt.show()

plt.figure(figsize=(6, 6))
plt.scatter(dfs[:, 2], dfs[:, 3], alpha=0.8)
plt.grid()
plt.xlabel("a4")
plt.ylabel("b4")
for i,n in enumerate(target_list):
    plt.annotate(n[-12:],(dfs[i, 2], dfs[i, 3]))
for i in NGlist:
    i = int(i)
    plt.scatter(dfs[i, 2], dfs[i, 3], color="r", alpha=0.8)
plt.show()

fig3d = plt.figure()
ax3d = Axes3D(fig3d)
ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], "o", alpha=0.8)
for i in NGlist:
    i = int(i)
    ax3d.plot(feature[[i], 0], feature[[i], 1], feature[[i], 2], "o", color="r",alpha=0.8)
plt.show()

# print(pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(dfs.shape[1])]).head())
# print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
# print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
 

In [ ]:
import glob
import pandas as pd
import scipy
from sklearn import linear_model

model_list = glob.glob('model/ReModel_interRNN_long131test_s*_100_*.pth')
model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')

print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

NGlist_name = np.array(["s3_100_4","s1_100_3","s9_100_2","s4_100_1"])
# NGlist_name = np.array([])
NGlist = np.array([])

for NG in NGlist_name:
    for i, n in enumerate(model_list):
        if n[-12:-4] == NG:
            NGlist = np.append(NGlist,int(i))
print(NGlist)
    
# target = [2,4,7,8,9]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

target_list = model_list
PCA_list = np.array([np.ones(6)])

exp_plot = plt.figure()
exp_plot1 = exp_plot.add_subplot(111)

for model_path in target_list:
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
    
    PFC_w_exp = scipy.stats.expon.fit(np.abs(PFC_w[PFC_w>0]))
    PFC_inw_exp = scipy.stats.expon.fit(np.abs(PFC_inw[PFC_inw>0]))
    a1 = PFC_w_exp[1]
    a2 = PFC_inw_exp[1]
    PFC_w_exp = scipy.stats.expon.fit(np.abs(PFC_w[PFC_w<0]))
    PFC_inw_exp = scipy.stats.expon.fit(np.abs(PFC_inw[PFC_inw<0]))
    a3 = PFC_w_exp[1]
    a4 = PFC_inw_exp[1]

    HPC_w_exp = scipy.stats.expon.fit(np.abs(HPC_w[HPC_w>0]))
    HPC_inw_exp = scipy.stats.expon.fit(np.abs(HPC_inw[HPC_inw>0]))
    b1 = HPC_w_exp[1]
    b2 = HPC_inw_exp[1]
    HPC_w_exp = scipy.stats.expon.fit(np.abs(HPC_w[HPC_w<0]))
    HPC_inw_exp = scipy.stats.expon.fit(np.abs(HPC_inw[HPC_inw<0]))
    b3 = HPC_w_exp[1]
    b4 = HPC_inw_exp[1]


    Re_w_exp = scipy.stats.expon.fit(np.abs(Re_w[Re_w>0]))
    Re_inw_exp = scipy.stats.expon.fit(np.abs(Re_inw[Re_inw>0]))
    c1 = Re_w_exp[1]
    c2 = Re_inw_exp[1]
    Re_w_exp_n = scipy.stats.expon.fit(np.abs(Re_w[Re_w<0]))
    Re_inw_exp_n = scipy.stats.expon.fit(np.abs(Re_inw[Re_inw<0]))
    c3 = Re_w_exp_n[1]
    c4 = Re_inw_exp_n[1]
    
#     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b>0]),bins=20,density=True)
#     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb>0]),bins=10,density=True)
#     a1 = regression(make_bins(PFC_b_hist[1]),np.log(PFC_b_hist[0]))
#     a2 = regression(make_bins(PFC_inb_hist[1]),np.log(PFC_inb_hist[0]))
#     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b<0]),bins=80,density=True)
#     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb<0]),bins=40,density=True)
#     a3 = regression(make_bins(PFC_b_hist[1]),np.log(PFC_b_hist[0]))
#     a4 = regression(make_bins(PFC_inb_hist[1]),np.log(PFC_inb_hist[0]))
    
    PCA_list = np.concatenate((PCA_list,[[b2,b4,a2,a4,c2,c4]]),axis=0)
    
#     fig1.hist(PFC_w[PFC_w>0],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.abs(PFC_w[PFC_w<0]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig2.hist(np.abs(HPC_w[HPC_w<0]),bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(Re_w[Re_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(np.abs(Re_w[Re_w<0]),bins=40,alpha=0.5,log=True,density=True)

    xvalues = np.linspace(0,2,100)
    pdf1 = scipy.stats.expon.pdf(xvalues,Re_inw_exp[0],Re_inw_exp[1])
    pdf2 = scipy.stats.expon.pdf(xvalues,Re_inw_exp_n[0],Re_inw_exp_n[1])
    if model_path[-12:-4] == "s9_100_2":
        exp_plot1.plot(pdf1,c="r",alpha=1)
        exp_plot1.plot(pdf2,c="r",alpha=1)
    elif model_path[-12:-4] == "s4_100_1":
        exp_plot1.plot(pdf1,c="g",alpha=1)
        exp_plot1.plot(pdf2,c="g",alpha=1)
    else:
        exp_plot1.plot(pdf1,c="b",alpha=0.01)

# print(PCA_list[1:])
pca = PCA()
dfs = PCA_list[1:]
pca.fit(dfs)
feature = pca.transform(dfs)

print(pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(dfs.shape[1])]).head())
print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
'+str(sparse)+'

plt.figure(figsize=(6, 6))
plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
for i,n in enumerate(target_list):
    plt.annotate(n[-15:],(feature[i, 0], feature[i, 1]))
for i in NGlist:
    i = int(i)
    plt.scatter(feature[i, 0], feature[i, 1], color="r", alpha=0.8)
plt.show()

plt.figure(figsize=(6, 6))
plt.scatter(dfs[:, 0], dfs[:, 1], alpha=0.8)
plt.grid()
plt.xlabel("a4")
plt.ylabel("b4")
for i,n in enumerate(target_list):
    plt.annotate(n[-15:],(dfs[i, 0], dfs[i, 1]))
for i in NGlist:
    i = int(i)
    plt.scatter(dfs[i, 0], dfs[i, 1], color="r", alpha=0.8)
plt.show()

fig3d = plt.figure()
ax3d = Axes3D(fig3d)
ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], "o", alpha=0.8)
for i in NGlist:
    i = int(i)
    ax3d.plot(feature[[i], 0], feature[[i], 1], feature[[i], 2], "o", color="r",alpha=0.8)
plt.show()

# print(pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(dfs.shape[1])]).head())
# print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
# print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
 

In [ ]:
def calc_KL(data1,data2):
    x = np.linspace(-2,2,200)
    params1 = scipy.stats.norm.fit(data1)
    params2 = scipy.stats.norm.fit(data2)
    pdf1 = scipy.stats.norm.pdf(x,params1[0],params1[1])
    pdf2 = scipy.stats.norm.pdf(x,params2[0],params2[1])
    return scipy.stats.entropy(pdf1,pdf2)

def calc_KL_exp(data1,data2):
    x = np.linspace(0,2,200)
    params1 = scipy.stats.expon.fit(np.abs(data1[data1>=0]))
    params2 = scipy.stats.expon.fit(np.abs(data2[data2>=0]))
    pdf1 = scipy.stats.expon.pdf(x,params1[0],params1[1])
    pdf2 = scipy.stats.expon.pdf(x,params2[0],params2[1])
    return scipy.stats.entropy(pdf1,pdf2)

def calc_KL_exp2(data1):
    x = np.linspace(0,2,200)
    params1 = scipy.stats.expon.fit(np.abs(data1[data1>=0]))
    params2 = scipy.stats.expon.fit(np.abs(data1[data1<=0]))
    pdf1 = scipy.stats.expon.pdf(x,params1[0],params1[1])
    pdf2 = scipy.stats.expon.pdf(x,params2[0],params2[1])
    return scipy.stats.entropy(pdf1,pdf2)

In [ ]:
import glob
import scipy

model_list = glob.glob('model/ReModel_interRNN_long131test_s*_100_2.pth')
print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

target_list = []

ANOVA_list = []

# target = [2,4]
# for i in target:
#     target_list.append(model_list[i])

target_list = model_list

fig_Re = plt.figure(figsize=(10,5))
for i,model_path in enumerate(target_list):
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
                weight_PFC_w = PFC_w[PFC_w.nonzero()]
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
                weight_PFC_inw = PFC_inw[PFC_inw.nonzero()]
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
                weight_HPC_w = HPC_w[HPC_w.nonzero()]
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
                weight_HPC_inw = HPC_inw[HPC_inw.nonzero()]
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
                weight_Re_w = Re_w[Re_w.nonzero()]
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
                weight_Re_inw = Re_inw[Re_inw.nonzero()]
    
#     plt.plot(np.ones(weight_Re_inw.shape[0])*i,weight_Re_inw,"o") 
    plt.plot(np.ones(weight_Re_w.shape[0])*i,weight_Re_w,"o") 
    ANOVA_list.append(weight_Re_w)
    
    print(model_path)
    
#     print(calc_KL(weight_HPC_inw,weight_HPC_w)+calc_KL(weight_PFC_inw,weight_PFC_w)+calc_KL(weight_Re_inw,weight_Re_w))
#     print(calc_KL(weight_HPC_w,weight_HPC_inw)+calc_KL(weight_PFC_w,weight_PFC_inw)+calc_KL(weight_Re_w,weight_Re_inw))

#     print(calc_KL_exp(weight_HPC_inw,weight_HPC_w))
#     print(calc_KL_exp(weight_HPC_w,weight_HPC_inw))
#     print(calc_KL_exp(weight_PFC_inw,weight_PFC_w))
#     print(calc_KL_exp(weight_PFC_w,weight_PFC_inw))
#     print(calc_KL_exp(weight_Re_inw,weight_Re_w))
#     print(calc_KL_exp(weight_Re_w,weight_Re_inw))
    
#     print(calc_KL_exp2(weight_HPC_inw))
#     print(calc_KL_exp2(weight_PFC_inw))
    print(calc_KL_exp2(weight_Re_inw))
#     print(calc_KL_exp2(weight_HPC_w))
#     print(calc_KL_exp2(weight_PFC_w))
#     print(calc_KL_exp2(weight_Re_w))

print(scipy.stats.f_oneway(ANOVA_list[3], ANOVA_list[5]))


In [ ]:
import glob
import scipy

model_list = glob.glob('model/ReModel_interRNN_long131test_s1_100_3_plus_*.pth')
print(model_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

target_list = []

ANOVA_list = []

target = [4,8]
for i in target:
    target_list.append(model_list[i])

# target_list = model_list

fig_Re = plt.figure(figsize=(10,5))
for i,model_path in enumerate(target_list):
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
                weight_PFC_w = PFC_w
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
                weight_PFC_inw = PFC_inw
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
                weight_HPC_w = HPC_w
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
                weight_HPC_inw = HPC_inw
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
                weight_Re_w = Re_w
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
                weight_Re_inw = Re_inw
    
#     plt.plot(np.ones(weight_Re_inw.shape[0])*i,weight_Re_inw,"o") 
    plt.plot(np.ones(weight_Re_w.shape[0])*i,weight_Re_w,"o") 
    ANOVA_list.append(weight_Re_inw)
    
    print(model_path)
    
#     print(calc_KL(weight_HPC_inw,weight_HPC_w)+calc_KL(weight_PFC_inw,weight_PFC_w)+calc_KL(weight_Re_inw,weight_Re_w))
#     print(calc_KL(weight_HPC_w,weight_HPC_inw)+calc_KL(weight_PFC_w,weight_PFC_inw)+calc_KL(weight_Re_w,weight_Re_inw))

#     print(calc_KL_exp(weight_HPC_inw,weight_HPC_w))
#     print(calc_KL_exp(weight_HPC_w,weight_HPC_inw))
#     print(calc_KL_exp(weight_PFC_inw,weight_PFC_w))
#     print(calc_KL_exp(weight_PFC_w,weight_PFC_inw))
#     print(calc_KL_exp(weight_Re_inw,weight_Re_w))
#     print(calc_KL_exp(weight_Re_w,weight_Re_inw))
    
#     print(calc_KL_exp2(weight_HPC_inw))
#     print(calc_KL_exp2(weight_PFC_inw))
    print(calc_KL_exp2(weight_Re_inw))
#     print(calc_KL_exp2(weight_HPC_w))
#     print(calc_KL_exp2(weight_PFC_w))
#     print(calc_KL_exp2(weight_Re_w))

print(scipy.stats.f_oneway(ANOVA_list[0], ANOVA_list[1]))
plt.figure()
plt.imshow(ANOVA_list[0]-ANOVA_list[1])


In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        else:
            traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=100)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=100)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)

def plot_distance_bet2traj(traj1,traj2):
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    return result


class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.RNNCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        sparse = 0.5
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    model_path = 'model/ReModel_interRNN_long131test_s6_100_2.pth'
    filename = "primal_long131test"
    
    train_x,train_y = mkOwnDataSet(training_size,filename,data_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 10
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*2):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
            
    hidden = rnn.initHidden_rand()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*0.9
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*2):
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()

    
    
    print(np.array(PFCstate)[:,0].shape)
    #MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
    
    pca = PCA()
    dfs = np.array(HPCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    dividenum = int(np.array(feature).shape[0]/2)
    
    fig = plt.figure()
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:dividenum,0,0],traj[:dividenum,0,1])
    plt.plot(traj[dividenum:,0,0],traj[dividenum:,0,1])
    plt.show()
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
    ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(HPCstate)[:,0,i])
#         plt.show()
    
    plot_distance(feature[:dividenum],feature[dividenum:])
    HPC_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])
    
    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
    ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(PFCstate)[:,0,i])
#         plt.show()

    plot_distance(feature[:dividenum],feature[dividenum:])
    PFC_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])

    pca = PCA()
    dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
    ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()

    plot_distance(feature[:dividenum],feature[dividenum:])
    Re_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])
    
#     MakeAnimation(feature[:dividenum, 0], feature[:dividenum, 1], feature[dividenum:, 0], feature[dividenum:, 1], data_limit)

    plot_distance_bet2traj(feature[:dividenum],feature[dividenum:])
    
    plt.figure()
    plt.plot(HPC_dis)
    plt.plot(PFC_dis)
    plt.plot(Re_dis)
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from tslearn.clustering import TimeSeriesKMeans

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#             traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
#     order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(traj[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(traj[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result

def vec_var(datas):
    datas = np.array(datas)
    average = np.average(datas,axis=0)
    result = 0
    for data in datas:
        result += np.linalg.norm(data-average)
    result /= datas.shape[0]
    return result
    
def lyapunov_exp(data):
    result = np.mean(np.log(np.abs(np.diff(data))))
    return result

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5
#     model_path = 'model/ReModel_interRNN_long131test_s7_100_1.pth'
#     model_path = 'model/ReModel_interRNNrand_AddRe_OUT5_161_s5_100_1.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s9_100_1_2_2.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s5_100_4.pth'
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s5_100_1_epoch125.pth'
    
    filename = "primal_long131test"
    
    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 140
    est_length = 5
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
            
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
    
    pca = PCA()
    dfs = np.array(HPCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    dividenum = int(np.array(feature).shape[0]/2)
    
#     MakeAnimation(traj[:dividenum,0,0],traj[:dividenum,0,1], traj[dividenum:,0,0], traj[dividenum:,0,1], data_limit)
#     MakeAnimation(traj[:dividenum,0,0],traj[:dividenum,0,1], traj_noise[:,0,0], traj_noise[:,0,1], data_limit)
#     MakeAnimation_img(np.array(PFCstate)[:dividenum,0],"PFCr")
#     MakeAnimation_img(np.array(PFCstate)[dividenum:,0],"PFCl")
#     MakeAnimation_img(np.array(PFCstate_noise)[:,0],"PFCnoise")
    
    fig = plt.figure()
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:dividenum,0,0],traj[:dividenum,0,1])
    plt.plot(traj[dividenum:,0,0],traj[dividenum:,0,1])
    plt.show()
    
    linelist = search_delay(traj[:dividenum,0])
    linelist2 = search_delay(traj[dividenum:,0])
    linelist_all = np.append(linelist,linelist2)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
    ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(HPCstate)[:,0,i])
#         plt.show()
    
    plot_distance(feature[:dividenum],feature[dividenum:])
    HPC_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])
    
    HPC_samedis = plot_distance_bet2traj(np.array(HPCstate)[:dividenum,0],np.array(HPCstate)[dividenum:,0],linelist_all)
    
    print(linelist,linelist2)
    
    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
    ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    ax3d.plot(feature[20:21, 0], feature[20:21, 1], feature[20:21, 2],"o", alpha=1)
    ax3d.plot(feature[60:61, 0], feature[60:61, 1], feature[60:61, 2],"o", alpha=1)
    plt.show()
#     for i in range(20):
#         fig_place = plt.figure()
#         place3d = Axes3D(fig_place)
#         place3d.plot(traj[:,0,0], traj[:,0,1], np.array(PFCstate)[:,0,i])
#         plt.show()

#   ### for 3d divided plot ###
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    end = 0
    for i in range(3):
        start = end
        end = int(linelist[i*2+1])
        ax3d.plot(feature[start:start+1, 0], feature[start:start+1, 1], feature[start:start+1, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start+20:start+21, 0], feature[start+20:start+21, 1], feature[start+20:start+21, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start+40:start+41, 0], feature[start+40:start+41, 1], feature[start+40:start+41, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start:end, 0], feature[start:end, 1], feature[start:end, 2], color=colors[i], alpha=0.8)
    plt.show()
    
    
    plot_distance(feature[:dividenum],feature[dividenum:])
    PFC_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])
    
    PFC_samedis = plot_distance_bet2traj(np.array(PFCstate)[:dividenum,0],np.array(PFCstate)[dividenum:,0],linelist)

    pca = PCA()
    dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
#     ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
#     ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2],"o", alpha=1)
    plt.show()
    
#   ### for 3d divided plot ###
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    end = 0
    for i in range(3):
        start = end
        end = int(linelist[i*2+1])
        ax3d.plot(feature[start:start+1, 0], feature[start:start+1, 1], feature[start:start+1, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start+20:start+21, 0], feature[start+20:start+21, 1], feature[start+20:start+21, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start+40:start+41, 0], feature[start+40:start+41, 1], feature[start+40:start+41, 2], "o", color=colors[i], alpha=0.8)
#         ax3d.plot(feature[start+20:start+21, 0], feature[start+20:start+21, 1], feature[start+20:start+21, 2], "o", color=colors[i], alpha=0.8)
        ax3d.plot(feature[start:end, 0], feature[start:end, 1], feature[start:end, 2], color=colors[i], alpha=0.8)
    plt.show()

    plot_distance(feature[:dividenum],feature[dividenum:])
    Re_dis = overlap_coefficient(feature[:dividenum],feature[dividenum:])
    
#     MakeAnimation(feature[:dividenum, 0], feature[:dividenum, 1], feature[dividenum:, 0], feature[dividenum:, 1], data_limit)

    Re_samedis = plot_distance_bet2traj(Restate[:dividenum],Restate[dividenum:],linelist)
    
    
    
    plt.figure()
    plt.plot(HPC_dis)
    plt.plot(PFC_dis)
    plt.plot(Re_dis)
    plt.vlines(linelist,0,1)
    plt.show()
    
    plt.figure()
    plt.plot(HPC_samedis)
    plt.plot(PFC_samedis)
    plt.plot(Re_samedis)
    plt.vlines(linelist,0,3)
    plt.show()
    
    Re_noisedisr = plot_distance_bet2traj(Restate[:dividenum],Restate_noise[:],linelist)
    Re_noisedisl = plot_distance_bet2traj(Restate[dividenum:],Restate_noise[:],linelist)
    PFC_noisedisr = plot_distance_bet2traj(np.array(PFCstate)[:dividenum,0],np.array(PFCstate_noise)[:,0],linelist)
    PFC_noisedisl = plot_distance_bet2traj(np.array(PFCstate)[dividenum:,0],np.array(PFCstate_noise)[:,0],linelist)
    plt.figure()
    plt.plot(Re_noisedisr)
    plt.plot(Re_noisedisl)
    plt.vlines(linelist,0,3)
    plt.show()
    
    plt.figure()
    plt.plot(PFC_noisedisr)
    plt.plot(PFC_noisedisl)
    plt.vlines(linelist,0,3)
    plt.show()
    
    plot_activity_bet2traj(Restate[dividenum:],Restate[:dividenum],linelist)
#     plot_activity_bet2traj(np.array(PFCstate)[:dividenum,0],np.array(PFCstate)[dividenum:,0],linelist)

    points = np.array(np.where(traj_noise[:,0,1]<0.05)[0]).astype("int64")
    Replot = plt.figure()
    axre2 = Replot.add_subplot(111)
    axre2.imshow(np.corrcoef(np.array(np.array(Restate_noise)[points])))
    axre2.set_title("Re")  
    
    pca = PCA()
    dfs = np.array(np.array(Restate_noise)[points])
#     dfs = np.array(np.array(Restate_noise))
#     dfs = np.array(HPCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)    
    
#     delays = pick_delay(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_delay(traj_noise[:,0], np.array(Restate_noise))
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
    delays = pick_traj(traj_noise[:,0], np.array(Restate_noise))
    linelist_delay = [0]
    for i in delays:
        print(np.array(i).shape)
        linelist_delay.append(linelist_delay[-1]+len(i))
    
    fig = plt.figure()
    plt.plot(feature[:dividenum, 0], alpha=0.8)
    plt.vlines(linelist_delay,0,3)
    plt.show()
    
    fig = plt.figure()
    plt.plot(feature[:dividenum, 1], alpha=0.8)
    plt.show()
    
    fig = plt.figure()
    plt.plot(feature[:dividenum, 2], alpha=0.8)
    plt.show()
    
    fig = plt.figure()
    plt.plot(feature[:dividenum, 0]+feature[:dividenum, 1]+feature[:dividenum, 2], alpha=0.8)
#     plt.vlines(linelist,0,3)
    plt.show()

#     print(np.array(delays[:-1]).shape)
#     tskm = TimeSeriesKMeans(n_clusters=4,metric='euclidean',max_iter=200)
    
#     data_kmeans = match_length(delays[:-1])
#     tskm_pred = tskm.fit_predict(np.array(data_kmeans))
#     print(tskm_pred)
#     tskm_feature = tskm.transform(np.array(data_kmeans))
#     print(tskm_feature)

#     fig = plt.figure()
# #     plt.plot(distance_bet2traj(delays[0][:40],delays[1][:40]))
# #     plt.plot(distance_bet2traj(delays[0][:40],delays[2][:40]))
# #     plt.plot(distance_bet2traj(delays[0][:40],delays[3][:40]))
    
#     data = match_length(delays[:-1])
#     for k in range(6):
#         fig = plt.figure()
#         target = k
#         for i in range(6):
#             plt.plot(distance_bet2traj(data[target][:],data[i][:]))
#         plt.show()
    
#     plt.show()
    
# #     delays = pick_delay(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
# #     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
# #     delays = pick_delay(traj_noise[:,0], np.array(Restate_noise))
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
# #     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise))
#     linelist_delay = [0]

#     print(np.array(delays[:-1]).shape)
#     tskm = TimeSeriesKMeans(n_clusters=4,metric='euclidean',max_iter=200)
    
#     data_kmeans = match_length(delays[:-1])
#     tskm_pred = tskm.fit_predict(np.array(data_kmeans))
#     print(tskm_pred)
#     tskm_feature = tskm.transform(np.array(data_kmeans))
#     print(tskm_feature)

#     fig = plt.figure()
    
#     data = match_length(delays[:-1])
#     for k in range(6):
#         fig = plt.figure()
#         target = k
#         for i in range(6):
#             plt.plot(distance_bet2traj(data[target][:],data[i][:]))
#         plt.show()
    
#     plt.show()
    
#     stacked_data = np.array(data)[:,:,0]
#     cellsize = 20
    
#     for i in range(cellsize-1):
#         num = int(i+1)
#         np.array(data)[:,:,num]
#         stacked_data = np.concatenate([stacked_data,np.array(data)[:,:,num]],axis=0)
                                      
#     pca = PCA()
# #     dfs = np.array(data)[:,:,0]
#     dfs = stacked_data
# #     dfs = np.array(np.array(Restate_noise))
# #     dfs = np.array(HPCstate)[0:,0]
#     pca.fit(dfs)
#     feature = pca.transform(dfs)  
#     print(dfs.shape)
    
#     fig3d = plt.figure()
#     ax3d = Axes3D(fig3d)
# #     ax3d.plot(feature[:dividenum, 0], feature[:dividenum, 1], feature[:dividenum, 2], alpha=0.8)
# #     ax3d.plot(feature[dividenum:, 0], feature[dividenum:, 1], feature[dividenum:, 2], alpha=0.8)
# #     ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
# #     ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2],"o", alpha=1)
#     for i in range(cellsize):
#         start = i*8
#         end = (i+1)*8
#         if vec_var(dfs[start:end]) > 0.5:
#             print(i)
#             ax3d.plot(feature[start:end, 0], feature[start:end, 1], feature[start:end, 2],"o", alpha=1)
#         else:
#             ax3d.plot(feature[start:end, 0], feature[start:end, 1], feature[start:end, 2],"o", alpha=0.1)
#     start = 0
#     end = 8
# #     ax3d.plot(feature[start:end, 0], feature[start:end, 1], feature[start:end, 2],"o", alpha=1)

# #     for k in range(8):
# #         points = np.arange(k,cellsize*8+k,step=8)
# #         ax3d.plot(feature[points, 0], feature[points, 1], feature[points, 2], "o", alpha=0.1)
# #     points = np.arange(2,cellsize*8+2,step=8)
# #     ax3d.plot(feature[points, 0], feature[points, 1], feature[points, 2], "o", alpha=1)
    
#     plt.show()

    
#     for i in range(cellsize):
#         start = i*8
#         end = (i+1)*8
#         if vec_var(dfs[start:end]) > 0:
#             plt.figure()
#             data = dfs[start:end]
#             for k in range(4):
#                 plt.plot(data[k])
#                 plt.ylim(-1.1,1.1)
# #                 print(lyapunov_exp(data[i]))
# #                 print(nolds.lyap_r(data[k],min_tsep = 15, lag=1))
#             plt.show()
    
#     print("Lyapnov")
#     for i in range(cellsize):
#         print(nolds.lyap_r(np.array(Restate_noise)[:,i],min_tsep = 150, lag=55))
            
#     for i in range(1):
#         plt.figure()
#         plt.plot(np.array(PFCstate)[:100,0,i])
#         plt.title("neuron#"+str(i+1))
#         print(Re_w.shape)
        
#     for i in range(20):
#         plt.figure()
#         for k in range(5):
#             plt.plot(np.array(PFCstate)[:400,k,i])
#         plt.title("neuron#"+str(i+1))
    
if __name__ == '__main__':
    main()

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from tslearn.clustering import TimeSeriesKMeans
from scipy.special import expit

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#             traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
    # order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2,0,0,2])
    # order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(traj[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(traj[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result

def vec_var(datas):
    datas = np.array(datas)
    average = np.average(datas,axis=0)
    result = 0
    for data in datas:
        result += np.linalg.norm(data-average)
    result /= datas.shape[0]
    return result
    
def lyapunov_exp(data):
    result = np.mean(np.log(np.abs(np.diff(data))))
    return result

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens


def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq    


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 2
#     model_path = 'model/ReModel_interRNN_long131test_s7_100_1.pth'
#     model_path = 'model/ReModel_interRNNrand_AddRe_OUT5_161_s5_100_1.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s3_100_1_2_2.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s5_100_4.pth'
#     model_path = 'model/R+/ReModel_L2_interRNNrand_AddRe_OUT5_151_s3_100_1_epoch80.pth'
#     model_path = 'model/R+2/ReModel_L2_interRNNrand_AddRe100_OUT5_151_s1_100_1_epoch70.pth'
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch125.pth'
    


    filename = "primal_long131test"
    
    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
        
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w_b = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "linear.weight":
                OUT_w_b = np.array(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 120
    est_length = 5
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
            
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch105.pth'
    rnn.load_state_dict(torch.load(model_path))
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "linear.weight":
                OUT_w = np.array(p.data)

    length = 360
# #     plt.figure()
# #     plt.plot(np.array(Restate_noise)[:length,0])
# #     plt.show()
#     for i in range(20):
#         plt.figure()
#         neurons = np.concatenate((np.array(PFCstate_noise)[:,0],np.array(HPCstate_noise)[:,0]),axis=1)
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k += 20
#             data = neurons[:length,k]*Re_w[i,k]
#             if np.sum(np.abs(data)) > 5 and np.sum(np.abs(data)) > 5:
#                 print(k,Re_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)
#     for i in range(20):
#         plt.figure()
#         neurons = np.array(Restate_noise)[:length]
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k+=0
#             data = neurons[:length,k]*Re_inw[i,k]
#             if np.sum(np.abs(data)) > 5 and np.sum(np.abs(data)) > 5:
#                 print(k,Re_inw[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)

#     for i in range(20):
#         plt.figure()
#         neurons = np.concatenate((np.array(traj_noise)[:,0],np.array(Restate_noise)[:]),axis=1)
#         neurons_b = np.concatenate((np.array(traj)[:,0],np.array(Restate)[:]),axis=1)
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(HPCstate_noise)[:length,0,9],alpha=0.3)
# #         plt.plot(np.array(HPCstate)[:length,0,9],alpha=0.3)
#         plt.ylim(-1,1)
#         a = i
#         i = 11
#         for k in range(1):
#             k = 2 + a 
#             data = moving_average(neurons[:length,k]*HPC_w[i,k])[:length]
#             data_b = moving_average(neurons_b[:length,k]*HPC_w_b[i,k])[:length]
#             if np.sum(np.abs(data)) > 0 and np.sum(np.abs(data)) > 0:
#                 print(k,HPC_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)
#                 plt.plot(data_b)
#                 plt.plot(moving_average(neurons[:length,k]*HPC_w[i,k])[:length])
#         ###### test zone for the difference between left and right  ########
#         for k in range(1):
#             k = 9
#             data = neurons[:length,k]*HPC_w[i,k]
#             if np.sum(np.abs(data)) > 0 and np.sum(np.abs(data)) > 0:
#                 print(k,HPC_w[i,k],np.sum(np.abs(data)))
# #                 plt.plot(data[:120])
# #                 plt.plot(data[120:240])
# #                 plt.plot(data[240:360])
#                 plt.plot(data[120:240]-data[240:360])
#         ##### test zone end ###########
#     for i in range(20):
#         plt.figure()
#         neurons = np.array(Restate_noise)[:length]
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k+=0
#             data = neurons[:length,k]*Re_inw[i,k]
#             if np.sum(np.abs(data)) > 5 and np.sum(np.abs(data)) > 5:
#                 print(k,Re_inw[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)

#     for i in range(2):
#         plt.figure()
#         neurons = np.array(HPCstate_noise)[:,0]
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(traj_noise)[:length,0,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             data = neurons[:length,k]*OUT_w[i,k]
#             if np.sum(np.abs(data)) > 3 and np.sum(np.abs(data)) > 3:
#                 print(k,HPC_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)

                
#     for i in range(20):
#         plt.figure()
#         neurons = np.array(Restate_noise)[:length]
#         for k in range(1):
#             k+=0
#             data = Re_inw[i]
#             plt.plot(data)
                
#     ### random initials / batch
#     for i in range(20):
#         plt.figure()
#         for k in range(5):
#             plt.plot(np.array(HPCstate)[:140,k,i])
#         plt.title("neuron#"+str(i+1))
    
if __name__ == '__main__':
    main()

In [ ]:
import nolds

In [ ]:
#v4 without Re
#v4_1 revised forward function

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from tslearn.clustering import TimeSeriesKMeans

%matplotlib notebook

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.99
        var = 0.99
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
#         HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
#         PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens

    

def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5
#     model_path = 'model/ReModel_interRNN_long131test_s7_100_1.pth'
#     model_path = 'model/ReModel_interRNNrand_AddRe_OUT5_161_s5_100_1.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s3_100_1_2_2.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s5_100_4.pth'
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s6_100_1.pth'
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s6_100_2_epoch90.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s5_100_2.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s4_100_2_epoch120.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s10_100_10.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s9_100_3_epoch325.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_TrainOUT_epoch0.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT1_151_s6_100_2_TrainOUT_cont_epoch1.pth'
    model_path = 'model/R20_uniPFC/ReModel_L2_interRNNrand_OUT1_uniPFC_151_s5_100_1_epoch190.pth'
    filename = "primal_long131test"
    
    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

#     rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN_uniPFC(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
     
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 10
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))

    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)

    length = 120
#     plt.figure()
#     plt.plot(np.array(Restate_noise)[:length,0])
#     plt.show()
#     for i in range(20):
#         plt.figure()
#         neurons = np.concatenate((np.array(PFCstate_noise)[:,0],np.array(HPCstate_noise)[:,0]),axis=1)
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k += 0
#             data = neurons[:length,k]*Re_w[i,k]
#             if np.sum(np.abs(data)) > 10 and np.sum(np.abs(data)) > 10:
#                 print(k,Re_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)
#     for i in range(20):
#         plt.figure()
#         neurons = np.array(Restate_noise)[:length]
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             data = neurons[:length,k]*Re_inw[i,k]
#             if np.sum(np.abs(data)) > 10 and np.sum(np.abs(data)) > 10:
#                 print(k,Re_inw[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)


    fig = plt.figure()
    for i in range(10):
#         plt.plot(traj[:,i,0],traj[:,i,1])
#         plt.plot(traj_noise[:,i,0],traj_noise[:,i,1])
        plt.plot(traj_noise[:,i,0])
    plt.show()

    ### random initials / batch
    for i in range(2):
        plt.figure()
        for k in range(5):
            plt.plot(np.array(Restate_noise)[:200,k,i])
        plt.title("neuron#"+str(i+1))
    
if __name__ == '__main__':
    main()

In [ ]:
%matplotlib notebook

import collections

def Culc_gate(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = torch.cat([hiddens[1][0][0],hiddens[2][0]])
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
#             traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
            traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
#     order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2,0,0,2])
#     order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
#     order = [1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2]
#     #Test zone
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [1])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [1])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2])
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(states[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.1
        var = 0.1
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.01
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.05
        index = np.array([11])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.ones(self.batch_size, index.size)*100
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.ones(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens

def moving_average(data):
    y = np.ones(9)/9
    mean_seq = np.convolve(data, y)
    return mean_seq


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 2

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyLSTM_feedforward(inputsize, hidden_size, outputsize, batch_size)
    
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s4_100_2_epoch120.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s1_100_4_epoch220.pth'
    model_path = 'model/R20_1212121/ReModel_L2_interRNNrand_OUT1_1212121_s3_100_3_epoch130.pth'
#     model_path = 'model/R20_transfer3_1212121/ReModel_L2_interRNNrand_OUT1_transfers64_1212121_s9_100_3_epoch40.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s1_100_1_2_2.pth'
#     model_path = 'model/R20_feed/FeedModel_L2_interRNNrand_OUT1_151_s5_100_2_epoch150.pth'

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size).float()
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 10
    est_length = 2
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
    print(np.array(Gate_states).shape)
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)
                
            
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s2_100_1_epoch100.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s3_100_5_epoch215.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_TrainOUT_epoch0.pth'
#     model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s10_100_1_2_3.pth'
#     model_path = 'model/R20_feed/FeedModel_L2_interRNNrand_OUT1_151_s5_100_2_epoch145.pth'
    rnn.load_state_dict(torch.load(model_path))
    
#     for n, p in rnn.named_parameters():
#         if n == "HPC.weight_ih":
#             p.data[inputsize:,hidden_size:].sub_(p.data[inputsize:,hidden_size:])
 
    
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_x, batch_size).float()
#     data_limit = 20
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            Gate_states_noise.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
#             hidden = rnn.noiseHidden_select(hidden)
#             if traj_noise[-1][0][1] < 0.1:
#                 for cell in range(1):
# #                     hidden[2][0][cell+0] = np.average(np.array(Restate_noise)[-6:,cell+0])
#                     hidden[2][0][cell+10] = np.average(np.array(Restate)[k-5+data_limit:k+5+data_limit,cell+10])
# #                     hidden[2][0][cell+0] = moving_average(np.array(Restate)[:,cell+0])[4+k+data_limit]
# #                     hidden[2][0][cell+0] = np.array(Restate)[k+data_limit,cell+0] - moving_average(np.array(Restate)[:,cell+0])[4+k+data_limit]
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            Gate_states_noise.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_b = np.array(p.data)
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
#     MakeAnimation(traj[:,0,0],traj[:,0,1], traj_noise[:,0,0], traj_noise[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    fig = plt.figure()
    plt.plot(traj[:,0,0],traj[:,0,1])
#     plt.plot(traj_noise[:,0,0],traj_noise[:,0,1])
    plt.show()

    for i in range(20):
        plt.figure()
#         plt.ylim(-1,1)
        plt.plot(traj[:,0,0])
        plt.plot(traj_noise[:,0,0])
#         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
        for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
            plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
            plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
#             plt.plot(moving_average(np.array(Restate)[:,i]),alpha=0.5)
#             plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#             plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#             plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
#             plt.plot(np.array(Gate_states)[:,0,20+i],alpha=0.5)
#             plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
#             plt.plot(np.abs(np.array(Gate_states)[:,1,0+i]-np.array(Gate_states_noise)[:,1,0+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states)[:,0,0+i]/np.array(Gate_states)[:,0,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states_noise)[:,0,0+i]/np.array(Gate_states_noise)[:,0,20+i]),alpha=0.5)
        plt.title("neuron#"+str(i+1))
    
#     for i in range(20):
#         plt.figure()
#         plt.ylim(-1,1)
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         plt.plot(traj[:,0,0])
#         plt.plot(traj_noise[:,0,0])
#         for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:300,i],alpha=0.5)
# #             plt.plot(np.array(Restate_noise)[:300,i],alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate)[:300,i]),alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate_noise)[:300,i]),alpha=0.5)
#         plt.title("HPCneuron#"+str(i+1))
    
#     plt.figure()
#     a = np.abs(np.corrcoef(np.array(PFCstate)[:,0].T) - np.corrcoef(np.array(PFCstate_noise)[:,0].T))
# #     a = np.abs(np.corrcoef(np.array(Restate)[:].T) - np.corrcoef(np.array(Restate_noise)[:].T))
#     plt.imshow(a,cmap=plt.get_cmap("Reds"))

    pca = PCA()
    # dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0],np.array(Restate)),axis=1)
    # dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0]),axis=1)
    dfs = np.array(PFCstate)[:,0]
#     dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], alpha=0.8)
    plt.show()
    
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(PFCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
    delays = pick_traj(traj[:,0], np.array(Restate)[:])
#     delays = pick_traj(traj_noise[:,0], np.array(traj_noise)[:])[3:-1]
#     delays = pick_traj(traj[:,0], np.array(Gate_states)[:,1])
#     delays = pick_delay(traj[:,0], np.array(Gate_states)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
#     delays = pick_delay(traj_noise[:,0], np.array(Gate_states_noise)[:,0])
#     delays = pick_delay(traj[:,0], feature[:])
#     delays = pick_traj(traj[:,0], feature[:])
    bifur = np.array([])
    for i in range(20):
        plt.figure()
#         plt.ylim(-1.1,1.1)
#         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
        for data in delays[1:-1]:
#             plt.plot(np.array(data)[:,0+i,0],alpha=0.5)
            plt.plot(np.array(data)[:,0+i],alpha=0.5)
            print(len(data))
#             check_bifur = np.argmax(np.abs(np.array(data)[:,0+i,0] - 0.5))
#             bifur = np.append(bifur,data[check_bifur,i,0])
#             print(check_bifur,data[check_bifur,i,0])
        plt.title("PCA#"+str(i+1))
#     print(np.var(bifur),np.median(bifur)-np.mean(bifur))

#     for i in range(20):
#         plt.figure()
#         for k in range(1):
#             k+=0
#             data = PFC_w_a[i]
#             plt.plot(data)
#             data = PFC_w_b[i]
#             plt.plot(data)

#     length = 140
# #     plt.figure()
# #     plt.plot(np.array(Restate_noise)[:length,0])
# #     plt.show()
#     neuron_list = []
#     neuron_list_Re = []
#     for i in range(20):
#         plt.figure()
#         neurons_a = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0]),axis=1)
#         neurons_b = np.concatenate((np.array(PFCstate_noise)[:,0],np.array(HPCstate_noise)[:,0]),axis=1)
#         plt.plot(np.array(Restate)[:length,i],alpha=0.3)
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(40):
#             k += 0
#             data_a = neurons_a[:length,k]*Re_w_a[i,k]
#             data_b = neurons_b[:length,k]*Re_w_b[i,k]
#             diff = data_a - data_b
#             if np.sum(np.abs(data_a)) > 5 and np.sum(np.abs(data_a)) > 5:
#                 print("a",k,Re_w_a[i,k],np.sum(np.abs(data_a)))
#                 plt.plot(data_a)
#             if np.sum(np.abs(data_b)) > 5 and np.sum(np.abs(data_b)) > 5:
#                 print("b",k,Re_w_b[i,k],np.sum(np.abs(data_b)))
#                 plt.plot(data_b)
#             if np.sum(np.abs(diff)) > 5 and np.sum(np.abs(diff)) > 5:
#                 print("diff",k,Re_w_a[i,k],Re_w_b[i,k],np.sum(np.abs(diff)))
#                 neuron_list.append(k+1)
# #                 plt.plot(diff)
            
#     for i in range(20):
#         plt.figure()
#         i+=0
#         neurons_a = np.array(Restate)[:]
#         neurons_b = np.array(Restate_noise)[:]
#         plt.plot(np.array(Restate)[:length,i],alpha=0.3)
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k += 0
#             data_a = neurons_a[:length,k]*Re_inw_a[i,k]
#             data_b = neurons_b[:length,k]*Re_inw_b[i,k]
#             diff = data_a - data_b
#             if np.sum(np.abs(data_a)) > 5 and np.sum(np.abs(data_a)) > 5:
#                 print("a",k,Re_inw_a[i,k],np.sum(np.abs(data_a)))
#                 plt.plot(data_a)
#             if np.sum(np.abs(data_b)) > 5 and np.sum(np.abs(data_b)) > 5:
#                 print("b",k,Re_inw_b[i,k],np.sum(np.abs(data_b)))
#                 plt.plot(data_b)
#             if np.sum(np.abs(diff)) > 5 and np.sum(np.abs(diff)) > 5:
#                 print("diff",k,Re_inw_a[i,k],Re_inw_b[i,k],np.sum(np.abs(diff)))
#                 neuron_list_Re.append(k+1)
# #                 plt.plot(diff)

#     print(collections.Counter(neuron_list))
#     print(collections.Counter(neuron_list_Re))
    plt.figure()
    plt.plot(traj[:,0,0])
    plt.plot(traj[:,0,1])
    
    
if __name__ == '__main__':
    main()

In [ ]:
%matplotlib notebook

### fourier!!!

import collections
from scipy import signal
from scipy.fftpack import fft 
import pandas as pd

def Culc_gate(input, params, hiddens):
    Re = hiddens[0][0]
    HPC_input = torch.cat([input,Re],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    return [PFC_gates.tolist(), HPC_gates.tolist()]

def highpass(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "high")           #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y    
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
#             traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
            traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(states[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.05
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*v
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.05
        index = np.array([3])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens

def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch160.pth'
    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 2
    est_length = 3
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)
                
            
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch170.pth'
    rnn.load_state_dict(torch.load(model_path))
    
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_x, batch_size)
    data_limit = 2
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
#             hidden = rnn.noiseHidden_select(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()
    
    
    linelist = search_delay(traj_noise[:,0])
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_b = np.array(p.data)
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
#     MakeAnimation(traj[:,0,0],traj[:,0,1], traj_noise[:,0,0], traj_noise[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    fig = plt.figure()
    plt.plot(traj[:,0,0],traj[:,0,1])
    plt.plot(traj_noise[:,0,0],traj_noise[:,0,1])
    plt.show()
    

#     #########  FFT with numpy  ###########
#     for i in range(20):
#         plt.figure()
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
#             data = np.array(PFCstate_noise)[:,k,i]
#             plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
#             data = np.array(PFCstate_noise)[:,k,i]
#             fourier = np.fft.fft(data)
#             freq = np.fft.fftfreq(len(fourier))*15
#             plt.plot(freq,fourier)
#         plt.title("neuron#"+str(i+1))
    
#     for i in range(20):
#         plt.figure()
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
#             data = np.array(HPCstate_noise)[:,k,i]
# #             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#             fourier = np.fft.fft(data)
#             freq = np.fft.fftfreq(len(fourier))*15
#             plt.plot(freq,fourier)
#         plt.title("HPCneuron#"+str(i+1))


#     #########  FFT with scipy  ###########
    
#     fp = 2      #通過域端周波数[Hz]
#     fs = 1      #阻止域端周波数[Hz]
#     gpass = 3       #通過域端最大損失[dB]
#     gstop = 40
#     samplerate = 1500    

#     for i in range(20):
#         plt.figure()
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
#             data = np.array(PFCstate_noise)[:,k,i]
#             plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
#             data = np.array(PFCstate)[:,k,i]
#             data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#             fourier = fft(data_filt)
#             plt.plot(np.angle(fourier))
#             data = np.array(PFCstate_noise)[:,k,i]
#             data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#             fourier = fft(data_filt)
#             plt.plot(np.angle(fourier))
#         plt.title("neuron#"+str(i+1))
    
#     for i in range(20):
#         plt.figure()
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
#             data = np.array(HPCstate_noise)[:,k,i]
# #             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#             fourier = np.fft.fft(data)
#             freq = np.fft.fftfreq(len(fourier))*15
#             plt.plot(freq,fourier)
#         plt.title("HPCneuron#"+str(i+1))
        
#     for i in range(20):
#         plt.figure()
#         for k in range(1):
#             data = np.array(Restate_noise)[:,i]
#             plt.plot(np.array(Restate)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
# #             data = np.array(Restate)[:,i]
# # #             data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
# #             data_filt = data
# #             fourier = fft(data_filt)
# #             freq = np.fft.fftfreq(len(fourier))*15
# #             plt.plot(freq,np.abs(fourier))
# # #             plt.plot(np.degrees(np.angle(fourier)))
            
#             data = np.array(Restate_noise)[:,i]
# #             data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#             data_filt = data
#             fourier = fft(data_filt)
#             freq = np.fft.fftfreq(len(fourier))*15
#             plt.plot(np.log(np.abs(freq)),np.log(np.abs(fourier) ** 2),"o")
# #             plt.plot(np.degrees(np.angle(fourier)))
#         plt.title("neuron#"+str(i+1))

        
#     shift = 2
#     seglen = 140
        
#     for i in range(20):
#         plt.figure()
#         for k in range(1):
#             plt.plot(np.array(Restate)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
#             data = np.array(Restate_noise)[:,i]
# #             fourier = np.fft.fft(data)
# #             freq = np.fft.fftfreq(len(fourier))*15
# #             plt.plot(freq,fourier)
#             freqs,times,sx1 = signal.spectrogram(data,fs=15,nperseg=seglen,noverlap=seglen-shift,detrend=False,scaling='spectrum')
#             plt.pcolormesh(times,freqs, np.log10(sx1))
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
#             data = np.array(Restate)[:,i]
# #             data = np.array(HPCstate_noise)[:,k,i]
# #             fourier = np.fft.fft(data)
# #             freq = np.fft.fftfreq(len(fourier))*15
# #             plt.plot(freq,fourier)
#             freqs,times,sx2 = signal.spectrogram(data,fs=15,nperseg=seglen,noverlap=seglen-shift,detrend=False,scaling='spectrum')
# #             plt.pcolormesh(times,freqs, np.log10(sx2))
#             plt.pcolormesh(times,freqs, np.abs(np.log10(sx1)-np.log10(sx2)))
#         plt.title("neuron#"+str(i+1))
        
#     for i in range(20):
#         plt.figure()
#         for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
# #             data = np.array(HPCstate_noise)[:,k,i]
#             data = np.array(Restate)[:,i]
# #             fourier = np.fft.fft(data)
# #             freq = np.fft.fftfreq(len(fourier))*15
# #             plt.plot(freq,fourier)
#             freqs,times,sx = signal.spectrogram(data,fs=15,nperseg=seglen,noverlap=seglen-shift,detrend=False,scaling='spectrum')
#             plt.pcolormesh(times,freqs, np.log10(sx))
#         plt.title("neuron#"+str(i+1))
#         plt.figure()
#         for k in range(1):
#             data = np.array(HPCstate)[:,k,i]
# #             fourier = np.fft.fft(data)
# #             freq = np.fft.fftfreq(len(fourier))*15
# #             plt.plot(freq,fourier)
#             freqs,times,sx = signal.spectrogram(data,fs=15,nperseg=seglen,noverlap=seglen-shift,detrend=False,scaling='spectrum')
#             plt.pcolormesh(times,freqs, np.log10(sx))
#         plt.title("neuron#"+str(i+1))
    
#     plt.figure()
#     a = np.abs(np.corrcoef(np.array(PFCstate)[:,0].T) - np.corrcoef(np.array(PFCstate_noise)[:,0].T))
# #     a = np.abs(np.corrcoef(np.array(Restate)[:].T) - np.corrcoef(np.array(Restate_noise)[:].T))
#     plt.imshow(a,cmap=plt.get_cmap("Reds"))
    
# #     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
#     for i in range(20):
#         plt.figure()
#         plt.ylim(-1,1)
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for data in delays[:]:
#             plt.plot(np.array(data)[:300,i],alpha=0.5)
#         plt.title("neuron#"+str(i+1))    


    fp = 1     #通過域端周波数[Hz]
    fs = 2      #阻止域端周波数[Hz]
    gpass = 3       #通過域端最大損失[dB]
    gstop = 40
    samplerate = 15
    result_a = []
    result_b = []
    for i in range(20):
        data = np.array(PFCstate)[:,0,i]
#         data = np.array(Restate)[:,i]

        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.figure()
#         plt.vlines(linelist,0,0.5)
# #         plt.plot(data)
#         plt.plot(data_filt)
        result_a.append(data_filt)
#         result_a.append(data)

#     for i in range(20):
        data = np.array(PFCstate_noise)[:,0,i]

        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.plot(data_filt)
        result_b.append(data_filt)
#         result_b.append(data)

    result_PFC = np.append(np.array(result_a).T,np.array(result_b).T,axis=0)
    
    pca = PCA()
    dfs = np.array(result_PFC)[:]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    plt.figure()
    len = np.array(PFCstate).shape[0]
#     plt.plot(feature[:len,0],alpha=0.5)
#     plt.plot(feature[:len,1],alpha=0.5)
#     plt.plot(feature[len:,0],alpha=0.5)
#     plt.plot(feature[len:,1],alpha=0.5)
#     plt.plot(feature[:len,0]-feature[len:,0],alpha=0.5)
#     plt.plot(feature[:len,1]-feature[len:,1],alpha=0.5)
#     plt.plot(feature[:len,2]-feature[len:,2],alpha=0.5)
#     plt.plot(feature[:len,3]-feature[len:,3],alpha=0.5)


#     fig3d = plt.figure()
#     ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[len:, 0], feature[len:, 1], feature[len:, 2], "o",alpha=0.8)
#     ax3d.plot(feature[:len, 0], feature[:len, 1], feature[:len, 2], "o",alpha=0.8)
#     ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
#     plt.show()
    
    
    result_a = []
    result_b = []
    for i in range(20):
        data = np.array(HPCstate)[:,0,i]
        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.figure()
#         plt.vlines(linelist,0,0.5)
# #         plt.plot(data)
#         plt.plot(data_filt)
        data = np.array(HPCstate)[:,0,i]
        result_a.append(data_filt)
#         result_a.append(data)
#     for i in range(20):
        data = np.array(HPCstate_noise)[:,0,i]
        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.plot(data_filt)
        result_b.append(data_filt)
#         result_b.append(data)

    result_HPC = np.append(np.array(result_a).T,np.array(result_b).T,axis=0)
    
    pca = PCA()
    dfs = np.array(result_HPC)[:]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    len = np.array(HPCstate).shape[0]
#     plt.plot(feature[:len,0],alpha=0.5)
#     plt.plot(feature[:len,1],alpha=0.5)
#     plt.plot(feature[:len,2],alpha=0.5)
#     plt.plot(feature[len:,0],alpha=0.5)
#     plt.plot(feature[len:,1],alpha=0.5)
#     plt.plot(feature[len:,2],alpha=0.5)
#     plt.vlines(linelist,0,0.5)
    
    
    result_a = []
    result_b = []
    for i in range(20):
#         data = np.array(PFCstate)[:,0,i]
        data = np.array(Restate)[:,i]
        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.figure()
#         plt.plot(traj[:,0,0])
#         plt.plot(traj_noise[:,0,0])
# #         plt.vlines(linelist,0,0.5)
# #         plt.plot(data)
#         plt.plot(data_filt)
        data = np.array(Restate)[:,i]
        result_a.append(data_filt)
#         result_a.append(data)
#     for i in range(20):
#         data = np.array(PFCstate_noise)[:,0,i]
        data = np.array(Restate_noise)[:,i]
        data_filt = highpass(data, samplerate, fp, fs, gpass, gstop)
#         plt.plot(data_filt)
        result_b.append(data_filt)
#         result_b.append(data)

    result_Re = np.append(np.array(result_a).T,np.array(result_b).T,axis=0)
    
    pca = PCA()
    dfs = np.array(result_Re)[:]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    len = np.array(Restate).shape[0]
    plt.plot(feature[:len,0],alpha=0.5)
    plt.plot(feature[:len,0],alpha=0.5)
#     plt.plot(feature[:len,1],alpha=0.5)
#     plt.plot(feature[:len,2],alpha=0.5)
    plt.plot(feature[len:,0],alpha=0.5)
#     plt.plot(feature[len:,1],alpha=0.5)
#     plt.plot(feature[len:,3],alpha=0.5)
#     plt.plot(feature[:len,0]-feature[len:,0],alpha=0.5)
#     plt.plot(feature[:len,1]-feature[len:,1],alpha=0.5)
#     plt.plot(feature[:len,2]-feature[len:,2],alpha=0.5)
#     plt.plot(feature[:len,3]-feature[len:,3],alpha=0.5)
#     plt.vlines(linelist,0,0.5)
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[len:, 0], feature[len:, 1], feature[len:, 2], "o", alpha=0.8)
    ax3d.plot(feature[:len, 0], feature[:len, 1], feature[:len, 2], "o", alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()

    plt.figure()
    plt.plot(feature[len:, 0],feature[len:, 1])
#     plt.plot(feature[len:, 1])
#     plt.plot(feature[len:, 2])
#     plt.plot(feature[len:, 3])
#     plt.plot(feature[len:, 4])

    
if __name__ == '__main__':
    main()

In [ ]:
%matplotlib notebook

import collections


def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
#     order = [1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(states[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.9
        var = 0.9
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
#         HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
#         PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.01
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.05
        index = np.array([11])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.ones(self.batch_size, index.size)*100
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.ones(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens

def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 7
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
            
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            #Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w_a = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw_a = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)
            if n == "linear.weight":
                Output_w_a = np.array(p.data)

                
            
    model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_TrainOUT_epoch0.pth'
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch105.pth'
    rnn.load_state_dict(torch.load(model_path))
    
#     for n, p in rnn.named_parameters():
#         if n == "HPC.weight_ih":
#             p.data[inputsize:,hidden_size:].sub_(p.data[inputsize:,hidden_size:])

    
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
#     data_limit = 7
    init_point = init_point*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
#             hidden = rnn.noiseHidden_select(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w_b = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw_b = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_b = np.array(p.data)
            if n == "linear.weight":
                Output_w_b = np.array(p.data)


    fig = plt.figure()
    plt.plot(traj[:dividenum,0,0],traj[:dividenum,0,1])
#     plt.plot(traj[dividenum:,0,0],traj[dividenum:,0,1])
    plt.plot(traj_noise[:,0,0],traj_noise[:,0,1])
#     plt.plot(data[1,0,0],data[1,0,1],"o")
    plt.show()


    
#     for i in range(20):
#         plt.figure()
#         plt.ylim(-1,1)
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         plt.plot(traj[:dividenum,0,0])
#         plt.plot(traj[dividenum:,0,0])
#         plt.plot(traj_noise[:,0,0])
#         for k in range(1):
# #             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#             plt.plot(np.array(HPCstate)[:dividenum,k,i]-np.array(HPCstate_noise)[:dividenum,k,i],alpha=0.5)
#         plt.title("HPCneuron#"+str(i+1))
        
    pca = PCA()
    dfs = np.array(HPCstate)[:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    plt.figure()
    plt.plot(traj[:dividenum,0,0])
    plt.plot(traj_noise[:,0,0])
#     plt.plot(traj[:dividenum,0,1])
#     plt.plot(traj[dividenum:,0,1])
#     plt.plot(feature[:dividenum,0]-feature[dividenum:,0],alpha=0.5)
#     plt.plot(feature[:dividenum,1]-feature[dividenum:,1],alpha=0.5)
#     plt.plot(feature[:dividenum,2]-feature[dividenum:,2],alpha=0.5)
#     plt.plot(np.abs(feature[:dividenum,:5]-feature[dividenum:,:5]),alpha=0.5)
    plt.show()
    
    limit = 480
    a = np.abs(traj[:limit,0,0]-0.5)
    b = np.abs(traj[dividenum:dividenum+limit,0,0]-0.5)
    c = distance_bet2traj(np.array(HPCstate)[:limit,0],np.array(HPCstate)[dividenum:dividenum+limit,0])
    plt.figure()
    plt.plot(a)
    plt.plot(b)
    plt.plot(c)
    plt.plot(np.abs(a-b))

#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
    delays = pick_traj(traj[:dividenum,0], np.array(Restate)[:dividenum])
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    print(len(delays[0]))
    print(len(delays[-2]))
#     for i in range(20):
#         for data in delays[:-1]:
#             print(len(data))
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
    delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
#     delays = pick_traj(traj[dividenum:,0], np.array(Restate)[dividenum:])
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    print(len(delays[0]))
    print(len(delays[-2]))
#     for i in range(20):
#         plt.figure()
# #         plt.ylim(-1,1)
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for data in delays[:-1]:
#             plt.plot(np.array(data)[:,0+i],alpha=0.5)
#             print(len(data))
#         plt.title("neuron#"+str(i+1))      

    fig2 = plt.figure(figsize=(10,5))
    ax1 = fig2.add_subplot(141)
    ax2 = fig2.add_subplot(142)
    axre = fig2.add_subplot(143)
    axout = fig2.add_subplot(144)
    
    ax1.imshow(PFC_w_a - PFC_w_b,cmap="coolwarm")
    ax2.imshow(HPC_w_a - HPC_w_b,cmap="coolwarm")
    axre.imshow(Re_w_a - Re_w_b,cmap="coolwarm")
    axout.imshow(Output_w_a - Output_w_b,cmap="coolwarm")
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w_a - PFC_w_b),np.min(PFC_w_a - PFC_w_b)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w_a - HPC_w_b),np.min(HPC_w_a - HPC_w_b)))
    axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_w_a - Re_w_b),np.min(Re_w_a - Re_w_b)))
    axout.set_title("max = {:.2f},min = {:.2f}".format(np.max(Output_w_a - Output_w_b),np.min(Output_w_a - Output_w_b)))

    fig2 = plt.figure()
    ax1 = fig2.add_subplot(131)
    ax2 = fig2.add_subplot(132)
    axre = fig2.add_subplot(133)
    
    ax1.imshow(PFC_inw_a - PFC_inw_b,cmap="coolwarm")
    ax2.imshow(HPC_inw_a - HPC_inw_b,cmap="coolwarm")
    axre.imshow(Re_inw_a - Re_inw_b,cmap="coolwarm")
    ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_inw_a - PFC_inw_b),np.min(PFC_inw_a - PFC_inw_b)))
    ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_inw_a - HPC_inw_b),np.min(HPC_inw_a - HPC_inw_b)))
    axre.set_title("max = {:.2f},min = {:.2f}".format(np.max(Re_inw_a - Re_inw_b),np.min(Re_inw_a - Re_inw_b)))  
    
    uvr,uw,uvl = np.linalg.svd(Re_inw_a - Re_inw_b,full_matrices=False)
    print(uw)
    

if __name__ == '__main__':
    main()

In [ ]:
import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA
import scipy.stats as st

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]


# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_?.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
# model_list = glob.glob('model/R20/*s*_100_*.pth')
model_list = glob.glob('model/R20_cont_add/*OUT1**s*_100_*_epoch1??.pth')
# model_list = glob.glob('model_test/4_2/*.pth')

model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)
with open("attract2_list.txt") as f:
    lines = f.read().splitlines()
    
# with open("attract_contfail_test_list2.txt") as f:
with open("attract_contfail_list_add_test.txt") as f:
    test_list_A = f.read().splitlines()
# with open("attract_contgood_test_list.txt") as f:
with open("attract_contgood_list_add_test.txt") as f:
    test_list_B = f.read().splitlines()
    
test_list_A_selected = []
test_list_B_selected = []
test_list_A_limited = []
test_list_B_limited = []

for model_path in model_list:
    if model_path in test_list_A:
        test_list_A_selected.append(model_path)
    if model_path in test_list_B:
        test_list_B_selected.append(model_path)            

# if len(test_list_A_selected) > len(test_list_B_selected):
#     random_num = np.random.randint(0,len(test_list_A_selected),len(test_list_B_selected))
#     for i in random_num:
#         test_list_A_limited.append(test_list_A_selected[i])
#     test_list_B_limited = test_list_B_selected
# else:
#     random_num = np.random.randint(0,len(test_list_B_selected),len(test_list_A_selected))
#     for i in random_num:
#         test_list_B_limited.append(test_list_B_selected[i])
#     test_list_A_limited = test_list_A_selected

limited_number = 0
if limited_number > 0:
    random_num = np.random.randint(0,len(test_list_A_selected),limited_number)
    for i in random_num:
        test_list_A_limited.append(test_list_A_selected[i])
    random_num = np.random.randint(0,len(test_list_B_selected),limited_number)
    for i in random_num:
        test_list_B_limited.append(test_list_B_selected[i])
else:
    test_list_A_limited = test_list_A_selected
    test_list_B_limited = test_list_B_selected

    
print(len(test_list_A_limited),len(test_list_B_limited))
    
# test_list = glob.glob('model/R20/*s1*_100_2*.pth')
# test_list = glob.glob('model/R20_cont/*OUT1*s*_100_*.pth')
# test_list = glob.glob('model_test/4_2/*.pth')
test_list = model_list
# test_list = lines
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# target = [1,3,4,11,12,13,16,-2,-1]
# target = [0,2,5,6,7,8,9,10,14,15,17]
# target = [11,12,-2]
# target = [21,22,23]
target = [-7, -2]
test_target = []

for i in target:
    test_target.append(test_list[i])
test_list = test_target

# print(test_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])


target_list = model_list[:]

fig_eigen = plt.figure()
fig_eigenval = fig_eigen.add_subplot(111)
test_colors = []

fig3d = plt.figure()
ax3d = Axes3D(fig3d)

PCA_list_A = np.array([[0,1,2,3,4]])
PCA_list_B = np.array([[0,1,2,3,4]])

stat_list_A = []
stat_list_B = []

stat_list_xy_A = [np.array([0]),np.array([0])]
stat_list_xy_B = [np.array([0]),np.array([0])]

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
            if n == "linear.weight":
                Out_w = np.array(p.data)
    
# #     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w>0]),bins=80,density=True)
# #     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw>0]),bins=40,density=True)
# #     fig1.plot((make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
# #     fig1.plot((make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
# #     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w<0]),bins=80,density=True)
# #     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw<0]),bins=40,density=True)
#     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w.nonzero()]),bins=80,density=True)
#     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw.nonzero()]),bins=40,density=True)
# #     PFC_hist = np.histogram(np.abs(np.append(PFC_w[PFC_w.nonzero()],PFC_inw[PFC_inw.nonzero()])),bins=40,density=True)
# #     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b.nonzero()]),bins=80,density=True)
# #     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb.nonzero()]),bins=40,density=True)
# #     fig1.plot(np.log(make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
#     fig1.plot(np.log(make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
# #     fig1.plot((make_bins(PFC_hist[1])),np.log(PFC_hist[0]))
#     a1 = regression(make_bins(PFC_w_hist[1]),np.log(PFC_w_hist[0]+0.01))
#     a2 = regression(make_bins(PFC_inw_hist[1]),np.log(PFC_inw_hist[0]+0.01))


# #     HPC_w_hist = np.histogram(HPC_w[HPC_w>0],bins=80,density=True)
# #     HPC_inw_hist = np.histogram(HPC_inw[HPC_inw>0],bins=40,density=True)
# #     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
# #     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
# #     HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w<0]),bins=80,density=True)
# #     HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw<0]),bins=40,density=True)
#     HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w.nonzero()]),bins=80,density=True)
#     HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw.nonzero()]),bins=40,density=True)
# #     HPC_w_hist = np.histogram(np.abs(HPC_w),bins=80,density=True)
# #     HPC_inw_hist = np.histogram(np.abs(HPC_inw),bins=40,density=True)
# #     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
# #     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
# #     fig2.plot(np.log(make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]+0.01),color=color)
#     fig2.plot(np.log(make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]+0.01),color=color)
#     b1 = regression(make_bins(HPC_w_hist[1]),np.log(HPC_w_hist[0]+0.01))
#     b2 = regression(make_bins(HPC_inw_hist[1]),np.log(HPC_inw_hist[0]+0.01))


# #     Re_w_hist = np.histogram(Re_w[Re_w>0],bins=100,density=True)
# #     Re_inw_hist = np.histogram(Re_inw[Re_inw>0],bins=100,density=True)
# #     Re_w_hist = np.histogram(np.abs(Re_w[Re_w<0]),bins=100,density=True)
# #     Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw<0]),bins=20,density=True)
#     Re_w_hist = np.histogram(np.abs(Re_w[Re_w.nonzero()]),bins=40,density=True)
#     Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw.nonzero()]),bins=40,density=True)
# #     Re_inw_hist = np.histogram(np.abs(Re_inw),bins=40,density=True)
#     Re_initw_hist = np.histogram(np.abs(Re_initw),bins=40,density=True)
#     fig3.plot(np.log(make_bins(Re_w_hist[1])),np.log(Re_w_hist[0]+0.01),color=color)
# #     fig3.plot(np.log(make_bins(Re_inw_hist[1])),np.log(Re_inw_hist[0]+0.01),color=color)
# #     fig3.plot(np.log(make_bins(Re_initw_hist[1])),np.log(Re_initw_hist[0]+0.01))
#     c1 = regression(make_bins(Re_w_hist[1]),np.log(Re_w_hist[0]+0.01))
#     c2 = regression(make_bins(Re_inw_hist[1]),np.log(Re_inw_hist[0]+0.01))
    
#     Re_uw,Re_uv = LA.eig(Re_w[:,20:])
#     fig_eigenval.plot(Re_inuw.real,Re_inuw.imag,"o",color=color)
#     print(Re_inuw)
#     print(np.count_nonzero(Re_uw.real>0))
#     print(np.count_nonzero(Re_uw.imag==0))
#     print(np.count_nonzero((Re_uw.imag==0)*(Re_uw.real>0)))


    i = 0
    PFC_inuw,PFC_inuv = LA.eig(PFC_inw[i:i+20])
#     print(PFC_inuw,PFC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=testcolor,zorder=2)
#     else :
#         fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=color,alpha=0.01,zorder=1)
#     for k in range(20):
#         fig_eigenval.text(PFC_inuw.real[k],PFC_inuw.imag[k],k)
#     print(LA.matrix_rank(PFC_inw[i:i+20]))
#     print(np.count_nonzero(PFC_inuw.real>0))
#     print(np.count_nonzero(PFC_inuw.imag==0))
#     print(np.count_nonzero((PFC_inuw.imag==0)*(HPC_inuw.real>0)))
    
    i = 0
    HPC_inuw,HPC_inuv = LA.eig(HPC_inw[i:i+20])
#     print(HPC_inuw,HPC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color=testcolor,alpha=1,zorder=2)
# #     elif num < 40:
# #         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color="b",alpha=1)
#     else:
#         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color=color,alpha=0.1,zorder=1)
#     for k in range(20):
#         fig_eigenval.text(HPC_inuw.real[k],HPC_inuw.imag[k],k)
#     print(LA.matrix_rank(HPC_inw[i:i+20]))
#     print(np.count_nonzero(HPC_inuw.real>0))
#     print(np.count_nonzero(HPC_inuw.imag==0))
#     print(np.count_nonzero((HPC_inuw.imag==0)*(HPC_inuw.real>0)))
    
#     HPC_uw,HPC_uv = LA.eig(np.pad(HPC_w[i:i+20,2:],[(0,0),(0,0)]))
#     HPC_uw,HPC_uv = LA.eig(np.pad(HPC_w[i:i+20,:2],[(0,0),(0,18)]))
#     print(HPC_uw,HPC_uv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         test_colors.append(testnum/len(test_list))
#         fig_eigenval.plot(HPC_uw.real,HPC_uw.imag,"o",color=testcolor,alpha=1,zorder=2)
#     else:
#         fig_eigenval.plot(HPC_uw.real,HPC_uw.imag,"o",color=color,alpha=0.1,zorder=1)
#     print(np.count_nonzero(HPC_uw.real>0))
#     print(np.count_nonzero(HPC_uw.imag==0))
#     print(np.count_nonzero((HPC_uw.imag==0)*(HPC_uw.real>0)))

# #     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+2*HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]
#     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]+HPC_inw[0:20]*HPC_inw[40:60]
# #     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]
# #     HPC_allw = HPC_inw[0:20]
# #     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+2*PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]
# #     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
#     PFC_allw = (PFC_w[0:20,:20]+PFC_inw[0:20])*(PFC_w[40:60,:20]+PFC_inw[40:60])
# #     PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]+PFC_inw[0:20]*PFC_w[40:60,20:]+PFC_w[0:20,20:]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
#     PFC_allw2 = (PFC_w[0:20,20:]+PFC_inw[0:20])*(PFC_w[40:60,20:]+PFC_inw[40:60])
#     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]
#     PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]
#     Re_allw  = Re_w[:,0:20] + Re_w[:,20:] + Re_inw
#     Re_allw  = Re_w[:,0:20] + Re_inw
#     Re_allw2  = Re_w[:,20:] + Re_inw
#     Re_allw  = Re_w[:,0:20]
#     Re_allw2  = Re_w[:,20:]
#     all_uw,all_uv = LA.eig(HPC_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw)
#     all_uw,all_uv = LA.eig(PFC_allw2)
    all_uw,all_uv = LA.eig(PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(Re_allw)
#     all_uw,all_uv = LA.eig(Re_allw2)
#     all_uw,all_uv = LA.eig(Re_inw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2)
#     all_uw,all_uv = LA.eig(PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * Re_allw2 + PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw * Re_allw2 + HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw * Re_allw * HPC_allw + PFC_allw2 * Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw @ Re_allw @ HPC_allw + PFC_allw2 @ Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(np.pad(Out_w[:,:],[(0,18),(0,0)]))
#     all_uw,all_uv = LA.eig(Out_w[:,:]@(HPC_w[0:20,:2]*HPC_w[40:60,:2]))
#     all_uw,all_uv = LA.eig(np.pad(HPC_w[20:40,:2],[(0,0),(0,18)]))
#     all_uw,all_uv = LA.eig(np.pad(HPC_w[60:,:2],[(0,0),(0,18)]))
#     all_uw,all_uv = LA.eig(HPC_inw[20:40])
# #     print(HPC_inuw,HPC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
# #         testcolor = "r"
#         test_colors.append(testnum/len(test_list))
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color=testcolor,alpha=1,zorder=2)
#         print(model_path,testcolor,np.average(abs(all_uw)))
# #         print(all_uw)
# #         print(all_uv.T)
# #         print(np.count_nonzero(all_uw.real>0))
# #         print(np.count_nonzero(all_uw.imag==0))
# #         print(np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)))
# #         print(np.average(abs(all_uw)))
# #         for k in range(20):
# #             fig_eigenval.text(all_uw.real[k],all_uw.imag[k],str(model_path.split("epoch")[-1].split(".")[0]))
#     else:
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color=color,alpha=0.1,zorder=1)
        
    if model_path in test_list_A_limited:
#     if not model_path in test_list_B_limited:
#         ax3d.plot([np.count_nonzero(all_uw.real>0)], [np.count_nonzero(all_uw.imag==0)], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "r", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [0],"o", color = "r", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "r", alpha=0.1)
        fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color="r",alpha=0.3,zorder=1)
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.zeros(20),all_uw.imag,"o",alpha=0.3,zorder=2)
#         fig_eigenval.plot(1,np.average(abs(all_uw)),"o",color="r",alpha=0.3,zorder=1)
#         fig_eigenval.plot(all_uw.real[np.argmax(abs(all_uw))],all_uw.imag[np.argmax(abs(all_uw))],"o",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.max(abs(all_uw)),"o",color="r",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.average(abs(all_uw)),np.max(abs(all_uw)),"o",color="r",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.max(np.abs(all_uw.real)),np.average(abs(all_uw)),"o",color="r",alpha=0.3,zorder=2)
#         print(model_path,"bad")
#         print([np.average(abs(all_uw)),np.max(all_uw.real),np.min(all_uw.real)])
#         print([np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
        PCA_list_A = np.concatenate((PCA_list_A,[[np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))]]),axis=0)
        stat_list_A.append(np.sort((abs(all_uw))))
#         stat_list_xy_A[0] = np.append(stat_list_xy_A[0],np.array(all_uw.real))
#         stat_list_xy_A[0] = np.append(stat_list_xy_A[0],np.abs(all_uw.imag))
#         stat_list_xy_A[1] = np.append(stat_list_xy_A[1],abs(all_uw))
#         stat_list_xy_A[0] = np.append(stat_list_xy_A[0],all_uw.imag[np.where(all_uw.imag>0)])
#         stat_list_xy_A[1] = np.append(stat_list_xy_A[1],abs(all_uw)[np.where(all_uw.imag>0)])
        stat_list_xy_A[0] = np.append(stat_list_xy_A[0],np.average(abs(all_uw)))
        stat_list_xy_A[1] = np.append(stat_list_xy_A[1],np.max(abs(all_uw)))
#         stat_list_xy_A[1] = np.append(stat_list_xy_A[1],np.average(abs(all_uw))/np.max(abs(all_uw)))
#         fig_eigenval.text(all_uw.real[np.argmax(abs(all_uw))],all_uw.imag[np.argmax(abs(all_uw))],str(model_path.split("s")[-1]))
#         fig_eigenval.text(np.average(abs(all_uw)),np.max(abs(all_uw)),str(model_path.split("s")[-1]))
#         fig_eigenval.text(0,np.average(abs(all_uw)),str(model_path.split("s")[-1]))
    if model_path in test_list_B_limited:
#         ax3d.plot([np.count_nonzero(all_uw.real>0)], [np.count_nonzero(all_uw.imag==0)], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "b", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [1],"o", color = "b", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))],[np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "b", alpha=0.1)
        fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.ones(20),all_uw.imag,"o",color="b",alpha=0.3,zorder=1)
#         fig_eigenval.plot(0,np.average(abs(all_uw)),"o",color="b",alpha=0.3,zorder=1)
#         fig_eigenval.plot(all_uw.real[np.argmax(abs(all_uw))],all_uw.imag[np.argmax(abs(all_uw))],"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.max(abs(all_uw)),"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.average(abs(all_uw)),np.max(abs(all_uw)),"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.plot(np.max(np.abs(all_uw.real)),np.average(abs(all_uw)),"o",color="b",alpha=0.3,zorder=2)
#         print(model_path,"good")
#         print([np.average(abs(all_uw)),np.max(all_uw.real),np.min(all_uw.real)])
#         print([np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
        PCA_list_B = np.concatenate((PCA_list_B,[[np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))]]),axis=0)
        stat_list_B.append(np.sort((abs(all_uw))))
#         stat_list_xy_B[0] = np.append(stat_list_xy_B[0],all_uw.real)
#         stat_list_xy_B[0] = np.append(stat_list_xy_B[0],np.abs(all_uw.imag))
#         stat_list_xy_B[1] = np.append(stat_list_xy_B[1],abs(all_uw))
#         stat_list_xy_B[0] = np.append(stat_list_xy_B[0],abs(all_uw.imag[np.where(all_uw.imag>0)]))
#         stat_list_xy_B[1] = np.append(stat_list_xy_B[1],abs(all_uw)[np.where(all_uw.imag>0)])
        stat_list_xy_B[0] = np.append(stat_list_xy_B[0],np.average(abs(all_uw)))
        stat_list_xy_B[1] = np.append(stat_list_xy_B[1],np.max(abs(all_uw)))
#         stat_list_xy_B[1] = np.append(stat_list_xy_B[1],np.average(abs(all_uw))/np.max(abs(all_uw)))
#         fig_eigenval.text(np.average(abs(all_uw)),np.max(abs(all_uw)),str(model_path.split("s")[-1]))
#         fig_eigenval.text(1,np.average(abs(all_uw)),str(model_path.split("s")[-1]))
        
        
# #     print(LA.matrix_rank(HPC_inw[i:i+20]))
#     print(np.count_nonzero(all_uw.real>0))
#     print(np.count_nonzero(all_uw.imag==0))
#     print(np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)))
    
    
#     PCA_list = np.concatenate((PCA_list,[[a1,a2,c2]]),axis=0)
#     print(a1-a2)
    
#     fig1.hist(PFC_w[PFC_w>0],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.abs(PFC_w[PFC_w<0]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig2.hist(np.abs(HPC_w[HPC_w<0]),bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(Re_w[Re_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(np.abs(Re_w[Re_w<0]),bins=40,alpha=0.5,log=True,density=True)

# # # print(PCA_list[1:])
# PCA_list = np.concatenate((PCA_list_A[1:],PCA_list_B[1:]),axis=0)
# length = PCA_list_A[1:].shape[0]
# pca = PCA()
# dfs = PCA_list
# pca.fit(dfs)
# feature = pca.transform(dfs)
# # print(length,feature)
# plt.figure(figsize=(6, 6))
# plt.scatter(feature[:length, 0], feature[:length, 1], color="r",alpha=0.2)
# plt.scatter(feature[length:, 0], feature[length:, 1], color="b",alpha=0.2)
# #plt.scatter(feature[100:200, 0], feature[100:200, 1], alpha=0.8)
# plt.grid()
# plt.xlabel("PC1")
# plt.ylabel("PC2")
# # for i,n in enumerate(target_list):
# #     plt.annotate(n[-12:],(feature[i, 0], feature[i, 1]))
# plt.show()

# print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
# print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    

# fig3d = plt.figure()
# ax3d = Axes3D(fig3d)
# ax3d.plot(feature[:length, 0], feature[:length, 1], feature[:length, 2], "o", alpha=0.1)
# ax3d.plot(feature[length:, 0], feature[length:, 1], feature[length:, 2], "o", alpha=0.1)
# plt.show()

# gradient = np.array(test_colors)
# gradient = np.vstack((gradient, gradient))

# fig, ax = plt.subplots()
# ax.imshow(gradient, cmap=plt.cm.jet)
# ax.set_axis_off()


# print(np.array(stat_list_xy_A).T,np.array(stat_list_xy_B).T)
# st.f_oneway(np.array(stat_list_xy_A).T,np.array(stat_list_xy_B).T)
st.f_oneway(np.array(stat_list_A).reshape(-1,1),np.array(stat_list_B).reshape(-1,1))
# st.ttest_ind(np.array(stat_list_xy_A).T,np.array(stat_list_xy_B).T,equal_var=False)

In [ ]:
import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA



class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
# model_list = glob.glob('model/R20/*s*_100_*.pth')
# model_list = glob.glob('model/R20_cont_add/*OUT1**s*_100_*.pth')
model_list = glob.glob('model/R20_uniPFC/*s*_100_*.pth')

model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)

test_list = glob.glob('model/R20/*s*_100_1_*.pth')
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# # target = [1,3,4,11,12,13,16,-2,-1]
# # target = [0,2,5,6,7,8,9,10,14,15,17]
# # target = [11,12,-2]
# # target = [19,20,21,22]
# test_target = []

# for i in target:
#     test_target.append(test_list[i])
# test_list = test_target

# print(test_list)

f = open("attract_uniPFCgood_list.txt",mode="w")

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig = plt.figure()
test_colors = []

target_list = model_list[:]
PCA_list = np.array([[0,1,2]])

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

#     rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN_uniPFC(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 7
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)

#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
    delays = pick_traj(traj[:,0], np.array(traj)[:])[3:-1]
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    bifur = np.array([])
    for data in delays:
        check_bifur = np.argmax(np.abs(np.array(data)[:,0,0] - 0.5))
        bifur = np.append(bifur,data[check_bifur,:,0])
#     print(bifur,check_bifur)
#     print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
    if bifur.shape[0] == 0:
        continue
    if np.var(bifur) < np.abs(np.median(bifur)-np.mean(bifur)) or np.var(bifur) < 0.0001:
#     if np.abs(np.median(bifur)-np.mean(bifur)) < 0.0001 and np.var(bifur) < 0.0001:
#         print(model_path)
#         f.write(model_path+"\n")
        pass
    else:
        print(model_path)
        plt.plot(np.log(np.var(bifur)),np.log(np.median(bifur)-np.mean(bifur)),"o")
        print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
        f.write(model_path+"\n")
#         pass

f.close()

In [ ]:
import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA
import scipy.stats as st


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

# model_list = glob.glob('model/R20/*s*_100_*.pth')
model_list = glob.glob('model/R20_cont/*OUT1*s*_100_*.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)
with open("attract1_list.txt") as f:
    lines = f.read().splitlines()
    
# with open("attract1_3_list.txt") as f:
#     test_list_A = f.read().splitlines()
# with open("attract2_list.txt") as f:
#     test_list_B = f.read().splitlines()

with open("attract_contfail_test_list.txt") as f:
    test_list_A = f.read().splitlines()
with open("attract_contgood_test_list.txt") as f:
    test_list_B = f.read().splitlines()

test_list = glob.glob('model/R20_cont/*s6*_100_2_*.pth')
# test_list = lines
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig_eigen = plt.figure()
fig_eigenval = fig_eigen.add_subplot(111)
test_colors = []

fig3d = plt.figure()
ax3d = Axes3D(fig3d)

target_list = model_list[:]
PCA_list_A = np.array([np.ones(6)])
PCA_list_B = np.array([np.ones(6)])

path_namelist_A = []
path_namelist_B = []

stat_list_A = []
stat_list_B = []

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
            if n == "linear.weight":
                Out_w = np.array(p.data)

    PCA_val = np.array([])
    HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]+HPC_inw[0:20]*HPC_inw[40:60]
    PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
    PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]+PFC_inw[0:20]*PFC_w[40:60,20:]+PFC_w[0:20,20:]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
    Re_allw  = Re_w[:,0:20] + Re_inw
    Re_allw2  = Re_w[:,20:] + Re_inw
#     all_uw,all_inuv = LA.eig(HPC_allw)
#     all_uw,all_inuv = LA.eig(PFC_allw)
#     all_uw,all_inuv = LA.eig(Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_allw * Re_allw + PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_allw * Re_allw + HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw * Re_allw + HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_inuv = LA.eig(PFC_allw2 * Re_allw)
#     all_uw,all_inuv = LA.eig(PFC_inw[0:20]*PFC_inw[40:60])

#     all_uw,all_inuv = LA.eig(HPC_allw * Re_allw)
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
#     all_uw,all_inuv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
#     all_uw,all_inuv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60])
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])

    all_uw,all_inuv = LA.eig(HPC_allw)
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
    PCA_val = np.append(PCA_val, np.max(abs(all_uw)))
    eigen_Re = np.average(abs(all_uw))
    
#     all_uw,all_inuv = LA.eig(PFC_allw)
# #     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
#     PCA_val = np.append(PCA_val, np.max(abs(all_uw)))
# #     eigen_Re = np.average(abs(all_uw))

    all_uw,all_inuv = LA.eig(HPC_allw @ Re_allw2 + PFC_allw2 @ Re_allw)
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
    PCA_val = np.append(PCA_val, np.max(abs(all_uw)))
#     eigen_Re = np.max(abs(all_uw))
    
    all_uw,all_inuv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
#     PCA_val = np.append(PCA_val, [np.average(abs(all_uw)),np.max(abs(all_uw))])
    PCA_val = np.append(PCA_val, np.average(abs(all_uw)))
    eigen_HPC = np.max(abs(all_uw))
    
    all_uw,all_inuv = LA.eig(PFC_allw @ Re_allw @ HPC_allw + PFC_allw2 @ Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     PCA_val = np.append(PCA_val, [np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
    PCA_val = np.append(PCA_val, [np.max(abs(all_uw))])
    eigen_PFC = np.average(abs(all_uw))
    
    all_uw,all_uv = LA.eig(np.pad(Out_w[:,:],[(0,18),(0,0)]))
    PCA_val = np.append(PCA_val, [np.max(all_uw.real),np.min(all_uw.real)])
#     PCA_val = np.append(PCA_val, [np.max(all_uw.real),np.min(all_uw.real), np.count_nonzero(all_uw.real>0), np.count_nonzero(all_uw.imag>0)])

    if model_path in test_list_A:
        PCA_list_A = np.concatenate((PCA_list_A,[PCA_val]),axis=0)
        fig_eigenval.plot(eigen_HPC,eigen_PFC,"o",color="r",alpha=0.3,zorder=2)
        fig_eigenval.text(eigen_HPC,eigen_PFC,str(model_path.split("s")[-1]))
        ax3d.plot([eigen_HPC],[eigen_Re],[eigen_PFC], "o", color = "r",alpha=0.1)
        path_namelist_A.append(str(model_path.split("s")[-1].split("epoch")[0]))
#         path_namelist_A.append(str(model_path.split("s")[-1]))
        stat_list_A.append([eigen_HPC,eigen_PFC])
    if model_path in test_list_B:
        PCA_list_B = np.concatenate((PCA_list_B,[PCA_val]),axis=0)
        fig_eigenval.plot(eigen_HPC,eigen_PFC,"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.text(eigen_HPC,eigen_PFC,str(model_path.split("s")[-1]))
        ax3d.plot([eigen_HPC],[eigen_Re],[eigen_PFC], "o", color = "b",alpha=0.1)
        path_namelist_B.append(str(model_path.split("s")[-1].split("epoch")[0]))
#         path_namelist_B.append(str(model_path.split("s")[-1]))
        stat_list_B.append([eigen_HPC,eigen_PFC])

# # print(PCA_list[1:])
PCA_list = np.concatenate((PCA_list_A[1:],PCA_list_B[1:]),axis=0)
length = PCA_list_A[1:].shape[0]
pca = PCA()
dfs = PCA_list
pca.fit(dfs)
feature = pca.transform(dfs)
# print(length,feature)
plt.figure(figsize=(6, 6))
plt.scatter(feature[:length, 0], feature[:length, 1], color="r",alpha=0.2)
plt.scatter(feature[length:, 0], feature[length:, 1], color="b",alpha=0.2)
#plt.scatter(feature[100:200, 0], feature[100:200, 1], alpha=0.8)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
# for i,n in enumerate(path_namelist_A):
#     plt.annotate(n,(feature[i, 0], feature[i, 1]))
# for i,n in enumerate(path_namelist_B):
#     plt.annotate(n,(feature[length+i, 0], feature[length+i, 1]))
plt.show()

print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    

fig3d = plt.figure()
ax3d = Axes3D(fig3d)
ax3d.plot(feature[:length, 0], feature[:length, 1], feature[:length, 2], "o", color = "r",alpha=0.1)
ax3d.plot(feature[length:, 0], feature[length:, 1], feature[length:, 2], "o", color = "b",alpha=0.1)
plt.show()

st.ttest_ind(stat_list_A,stat_list_B)


In [ ]:
##########    Test for mixture weight!!!!!!!!!!!!!   #########################
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from tslearn.clustering import TimeSeriesKMeans
from scipy.special import expit

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = torch.cat([hiddens[1][0][0],hiddens[2][0]])
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#             traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
#     order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(traj[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(traj[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result

def vec_var(datas):
    datas = np.array(datas)
    average = np.average(datas,axis=0)
    result = 0
    for data in datas:
        result += np.linalg.norm(data-average)
    result /= datas.shape[0]
    return result
    
def lyapunov_exp(data):
    result = np.mean(np.log(np.abs(np.diff(data))))
    return result

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens


def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq    


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    
    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
        
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
    rnn.load_state_dict(torch.load(model_path))
        
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_pre = torch.clone(p.data)
            if n == "HPC.weight_ih":
                HPC_w_pre = torch.clone(p.data)
            if n == "Re.weight_ih":
                Re_w_pre = torch.clone(p.data)
            if n == "Re.weight_hh":
                Re_inw = torch.clone(p.data)
            if n == "linear.weight":
                OUT_w_pre = torch.clone(p.data)
                   
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 20
    est_length = 5
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
                
        
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch105.pth'
    model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_TrainOUT_epoch0.pth'
    rnn.load_state_dict(torch.load(model_path))
        
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_pre = torch.clone(p.data)
            if n == "HPC.weight_ih":
                HPC_w_pre = torch.clone(p.data)
            if n == "Re.weight_ih":
                Re_w_pre = torch.clone(p.data)
            if n == "Re.weight_hh":
                Re_inw_pre = torch.clone(p.data)
            if n == "linear.weight":
                OUT_w_pre = torch.clone(p.data)        
        
        
    model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
    rnn.load_state_dict(torch.load(model_path))


#     rnn.PFC.weight_ih.data = PFC_w_pre
#     rnn.HPC.weight_ih.data = HPC_w_pre
#     rnn.Re.weight_ih.data[:20,:20] = 0.1*Re_w_pre[:20,:20] + 0.9*rnn.Re.weight_ih.data[:20,:20]
#     rnn.Re.weight_ih.data[:20,:20] = 1*Re_w_pre[:20,:20] + 0*rnn.Re.weight_ih.data[:20,:20]
#     rnn.Re.weight_ih.data = 1*Re_w_pre + 0*rnn.Re.weight_ih.data
#     rnn.Re.weight_hh.data = 1*Re_inw_pre + 0*rnn.Re.weight_hh.data
    rnn.Re.weight_hh.data[5:15] = 1*Re_inw_pre[5:15]
#     rnn.linear.weight.data = 1*OUT_w_pre + 0*rnn.linear.weight.data

    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                print(p.data)
#             if n == "HPC.weight_ih":
#                 HPC_w_b = np.array(p.data)
#             if n == "Re.weight_ih":
#                 Re_w_b = np.array(p.data)
#             if n == "Re.weight_hh":
#                 Re_inw = np.array(p.data)
#             if n == "linear.weight":
#                 OUT_w_b = np.array(p.data)
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_x, batch_size)
    init_point = torch.rand(10,2)*1
#     data_limit = 125
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            Gate_states_noise.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
#             hidden = rnn.noiseHidden_rand(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            Gate_states_noise.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")


    plt.figure()
    plt.plot(traj_noise[:,0,0])
#     length = 360
# #     plt.figure()
# #     plt.plot(np.array(Restate_noise)[:length,0])
# #     plt.show()
#     for i in range(20):
#         plt.figure()
#         neurons = np.concatenate((np.array(PFCstate_noise)[:,0],np.array(HPCstate_noise)[:,0]),axis=1)
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k += 20
#             data = neurons[:length,k]*Re_w[i,k]
#             if np.sum(np.abs(data)) > 5 and np.sum(np.abs(data)) > 5:
#                 print(k,Re_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)
#     for i in range(20):
#         plt.figure()
#         neurons = np.array(Restate_noise)[:length]
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k+=0
#             data = neurons[:length,k]*Re_inw[i,k]
#             if np.sum(np.abs(data)) > 5 and np.sum(np.abs(data)) > 5:
#                 print(k,Re_inw[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)

#     for i in range(20):
#         plt.figure()
#         neurons = np.concatenate((np.array(traj_noise)[:,0],np.array(Restate_noise)[:]),axis=1)
# #         print(neurons,Re_w[i])
# #         print((neurons*Re_w[i]))
#         plt.plot(np.array(HPCstate_noise)[:length,0,9],alpha=0.3)
# #         plt.plot(np.array(HPCstate)[:length,0,9],alpha=0.3)
#         plt.ylim(-1,1)
#         a = i
#         i = 11
#         for k in range(1):
#             k = 2 + a 
#             data = moving_average(neurons[:length,k]*HPC_w[i,k])[:length]
#             if np.sum(np.abs(data)) > 0 and np.sum(np.abs(data)) > 0:
#                 print(k,HPC_w[i,k],np.sum(np.abs(data)))
#                 plt.plot(data)
# #                 plt.plot(data_b)
# #                 plt.plot(moving_average(neurons[:length,k]*HPC_w[i,k])[:length])

    for i in range(20):
        plt.figure()
#         plt.ylim(-1,1)
        plt.plot(traj[:,0,0])
        plt.plot(traj_noise[:,0,0])
    #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
        for k in range(1):
#                 plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
#                 plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),alpha=0.5)
#                 plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#                 plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]),alpha=0.5)
    #             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
    #             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
    #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
    #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
    #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
    #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
#                 plt.plot(moving_average(np.array(Restate)[:,i]),alpha=0.5)
#                 plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#                 plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#                 plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
#                 plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
#                 plt.plot(np.array(Gate_states_noise)[:,1,20+i],alpha=0.5)
                plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
#                 plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
        plt.title("neuron#"+str(i+1))

if __name__ == '__main__':
    main()

In [ ]:
import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

def moving_var_test(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    var_seq = (data-mean_seq[2:-2])**2
    return var_seq

def moving_var_roll(data):
    var_seq = []
    for i in range(len(data)-4):
        var_seq.append(np.var(data[i:i+4]))
    return np.array(var_seq)


# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
model_list = glob.glob('model/R20/*s*_100_*.pth')
# model_list = glob.glob('model/R20_cont/*OUT1**s*_100_*.pth')

model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)

test_list = glob.glob('model/R20/*s1*_100_1_*.pth')
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# # target = [1,3,4,11,12,13,16,-2,-1]
# # target = [0,2,5,6,7,8,9,10,14,15,17]
# # target = [11,12,-2]
# # target = [19,20,21,22]
# test_target = []

# for i in target:
#     test_target.append(test_list[i])
# test_list = test_target

# print(test_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

with open("attract1_3_list.txt") as f:
    test_list_A = f.read().splitlines()
with open("attract2_list.txt") as f:
    test_list_B = f.read().splitlines()

fig = plt.figure()
test_colors = []

fig_eigen = plt.figure()
fig_eigenval = fig_eigen.add_subplot(111)

target_list = model_list[:]
PCA_list = np.array([[0,1,2]])

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 7
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)

    var_mean = []
    for i in range(20):
        var_mean.append(np.average(moving_var_roll(np.array(Restate)[:,i])))
    print(model_path,np.average(var_mean))
    
    delays = pick_traj(traj[:,0], np.array(traj)[:])
    bifur = np.array([])
    for data in delays[1:-1]:
        print(len(data))
        bifur = np.append(bifur,len(data))
    
    fig_eigenval.set_xlim(0,3)
    if model_path in test_list_A:
#         fig_eigenval.plot(1,np.average(var_mean),"o",color="r",alpha=0.3,zorder=2)
        fig_eigenval.plot(1,np.average(bifur),"o",color="r",alpha=0.3,zorder=2)
    if model_path in test_list_B:
#         fig_eigenval.plot(2,np.average(var_mean),"o",color="b",alpha=0.3,zorder=2)
        fig_eigenval.plot(2,np.average(bifur),"o",color="b",alpha=0.3,zorder=2)
        



In [ ]:
############## test for weight structure ###############

import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

model_list = glob.glob('model/R20_cont/*OUT1*s5_100_1*.pth')
# model_list = glob.glob('model/R20_cont/*s*_100_*.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)
with open("attract1_list.txt") as f:
    lines = f.read().splitlines()
    
with open("attract1_3_list.txt") as f:
    test_list_A = f.read().splitlines()
with open("attract2_list.txt") as f:
    test_list_B = f.read().splitlines()

test_list = glob.glob('model/R20_cont/*s8*_100_1_*.pth')
# test_list = lines
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig3d = plt.figure()
ax3d = Axes3D(fig3d)

target_list = model_list[:]
PCA_list_A = np.array([np.ones(6)])
PCA_list_B = np.array([np.ones(6)])

path_namelist_A = []
path_namelist_B = []

testnum = 0
num=0
fig = plt.figure()
i_f = fig.add_subplot(131)
i_c = fig.add_subplot(132)
i_o = fig.add_subplot(133)
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
            if n == "linear.weight":
                Out_w = np.array(p.data)


    plt.xlim(-5,5)
    plt.ylim(-5,5)
    for t in range(20):
        for i in range(2):
            for k in range(1):
#                 plt.scatter(HPC_w[0+t,i],Out_w[k,t],color=color,alpha=0.1)
#                 print(HPC_w[i,t],Out_w[k,i])
                i_f.scatter(HPC_w[0+t,i],HPC_w[20+t,i],color=color,alpha=0.1)
                i_c.scatter(HPC_w[0+t,i],HPC_w[40+t,i],color=color,alpha=0.1)
                i_o.scatter(HPC_w[0+t,i],HPC_w[60+t,i],color=color,alpha=0.1)
            
    
#     for t in range(20):
#         for i in range(2):
#             for k in range(1):
#                 if model_path in test_list_A:
#                     plt.scatter(HPC_w[0+t,i],HPC_w[40+t,i],color="r",alpha=0.1)
# #                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="r",alpha=0.1)
#                 elif model_path in test_list_B:
#                     plt.scatter(HPC_w[0+t,i],HPC_w[40+t,i],color="b",alpha=0.1)
# #                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="b",alpha=0.1)
#                 else:
#                     plt.scatter(HPC_w[0+t,i],HPC_w[40+t,i],color="g",alpha=0.1)
# #                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="g",alpha=0.1)


In [ ]:
############## test for weight structure ###############

import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

# model_list = glob.glob('model/R20_cont/*OUT1*s*_100_?.pth')
model_list = glob.glob('model/R20_cont_add/*OUT1*s*_100_*.pth')
# model_list = glob.glob('model/R20/*s4_100_1_*.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)
with open("attract1_list.txt") as f:
    lines = f.read().splitlines()
    
# with open("attract_contfail_test_list.txt") as f:
with open("attract_contfail_list_add_test.txt") as f:
    test_list_A = f.read().splitlines()
# with open("attract_contgood_test_list.txt") as f:
with open("attract_contgood_list_add_test.txt") as f:
    test_list_B = f.read().splitlines()
    
    
test_list_A_selected = []
test_list_B_selected = []
test_list_A_limited = []
test_list_B_limited = []

for model_path in model_list:
    if model_path in test_list_A:
        test_list_A_selected.append(model_path)
    if model_path in test_list_B:
        test_list_B_selected.append(model_path)            

if len(test_list_A_selected) > len(test_list_B_selected):
    random_num = np.random.randint(0,len(test_list_A_selected),len(test_list_B_selected))
    for i in random_num:
        test_list_A_limited.append(test_list_A_selected[i])
    test_list_B_limited = test_list_B_selected
else:
    random_num = np.random.randint(0,len(test_list_B_selected),len(test_list_A_selected))
    for i in random_num:
        test_list_B_limited.append(test_list_B_selected[i])
    test_list_A_limited = test_list_A_selected

# limited_number = 0
# if limited_number > 0:
#     random_num = np.random.randint(0,len(test_list_A_selected),limited_number)
#     for i in random_num:
#         test_list_A_limited.append(test_list_A_selected[i])
#     random_num = np.random.randint(0,len(test_list_B_selected),limited_number)
#     for i in random_num:
#         test_list_B_limited.append(test_list_B_selected[i])
# else:
#     test_list_A_limited = test_list_A_selected
#     test_list_B_limited = test_list_B_selected
    

# test_list = glob.glob('model/R20_cont/*s8*_100_1_*.pth')
# # test_list = lines
# test_list = sorted(test_list)
# test_list = sorted(test_list,key=len,reverse=False)[:]

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig3d = plt.figure()
ax3d = Axes3D(fig3d)

target_list = model_list[:]
PCA_list_A = np.array([np.ones(6)])
PCA_list_B = np.array([np.ones(6)])

path_namelist_A = []
path_namelist_B = []

testnum = 0
num=0
fig = plt.figure()
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
            if n == "linear.weight":
                Out_w = np.array(p.data)
                
#     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+2*HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]
#     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]+HPC_inw[0:20]*HPC_inw[40:60]
    HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]
#     HPC_allw = HPC_inw[0:20]
#     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+2*PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]
    PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
    PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]+PFC_inw[0:20]*PFC_w[40:60,20:]+PFC_w[0:20,20:]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
#     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]
#     PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]
#     Re_allw  = Re_w[:,0:20] + Re_w[:,20:] + Re_inw
#     Re_allw  = Re_w[:,0:20] + Re_inw
#     Re_allw2  = Re_w[:,20:] + Re_inw
    Re_allw  = Re_w[:,0:20]
    Re_allw2  = Re_w[:,20:]
#     all_uw,all_uv = LA.eig(HPC_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw)
#     all_uw,all_uv = LA.eig(Re_allw2)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2)
#     all_uw,all_uv = LA.eig(PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * Re_allw2 + PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw * Re_allw2 + HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw * Re_allw * HPC_allw + PFC_allw2 * Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw @ Re_allw @ HPC_allw + PFC_allw2 @ Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(np.pad(Out_w[:,:],[(0,18),(0,0)]))
#     all_uw,all_uv = LA.eig(Out_w[:,:]@(HPC_w[0:20,:2]*HPC_w[40:60,:2]))

#     uvr,uw,uvl = np.linalg.svd(Re_inw,full_matrices=False)
#     uvr,uw,uvl = np.linalg.svd(PFC_inw,full_matrices=False)
#     print(uw)
    ranknum = 0
    plt.xlim(-0.4,0.4)
    plt.ylim(-0.6,0.6)
    for t in range(1):
        for i in range(20):
            for k in range(1):
#                 plt.scatter(uvr[i,ranknum],uvl.T[i,ranknum],color=color,alpha=0.3)
#                 plt.scatter(HPC_w[i,1],uvl.T[i,ranknum],color=color,alpha=0.3)
#                 plt.scatter(HPC_w[i,0],uvr.T[i,ranknum],color=color,alpha=0.3)
                if model_path in test_list_A_limited:
                    uvr,uw,uvl = np.linalg.svd(PFC_inw[:60],full_matrices=False)
                    plt.scatter(uvr[i,ranknum],uvl.T[i,ranknum],color="r",alpha=0.1)
#                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="r",alpha=0.1)
#                     plt.scatter(0,HPC_w[20+t,i],color="r",alpha=0.1)
                    pass
                elif model_path in test_list_B_limited:
                    uvr,uw,uvl = np.linalg.svd(PFC_inw[:60],full_matrices=False)
                    plt.scatter(uvr[i,ranknum],uvl.T[i,ranknum],color="b",alpha=0.1)
#                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="b",alpha=0.1)
#                     plt.scatter(1,HPC_w[20+t,i],color="b",alpha=0.1)
                    pass

#                 else:
#                     plt.scatter(uvr[i,ranknum],uvl.T[i,ranknum],color="g",alpha=0.1)
# #                     plt.scatter(HPC_w[0+t,i],Out_w[k,t],color="g",alpha=0.1)

    


In [ ]:
######　面倒なんで別枠にしたelseテスト用

import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]


# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
# model_list = glob.glob('model/R20/*s*_100_*.pth')
# model_list = glob.glob('model/R20_cont/*OUT1**s*_100_?.pth')
# model_list = glob.glob('model/R20_131/*OUT1**s*_100_*epoch*.pth')
# model_list = glob.glob('model/R20_cont_add/*OUT1**s*_100_5_*epoch???.pth')
model_list = glob.glob('model/R20_uniPFC/*s*_100_?_*epoch*.pth')
model_list = glob.glob('model/R20_uniPFCHPC_151/*s*_100_?_*epoch*.pth')
# model_list = glob.glob('model/R20_uniPFC_131/*s*_100_?_*epoch*.pth')
# model_list = glob.glob('model_test/average_abs/*.pth')

model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)
with open("attract2_list.txt") as f:
    lines = f.read().splitlines()
    
with open("attract_contfail_list.txt") as f:
    test_list_A = f.read().splitlines()
# with open("attract_contgood_list_add.txt") as f:
# with open("model/R20_131/good_list.txt") as f:
# with open("model/R20_uniPFC_131/good_list.txt") as f:
with open("model/R20_uniPFCHPC_151/good_list.txt") as f:
    test_list_B = f.read().splitlines()

# test_list = glob.glob('model/R20/*s1*_100_2*.pth')
test_list = glob.glob('model/R20_cont/*OUT1*s*_100_*.pth')
# test_list = glob.glob('model_test/4_2/*.pth')
# test_list = lines
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# target = [1,3,4,11,12,13,16,-2,-1]
# target = [0,2,5,6,7,8,9,10,14,15,17]
# target = [11,12,-2]
# target = [21,22,23]
# test_target = []

# for i in target:
#     test_target.append(test_list[i])
# test_list = test_target

# print(test_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig_eigen = plt.figure()
fig_eigenval = fig_eigen.add_subplot(111)
test_colors = []

fig3d = plt.figure()
ax3d = Axes3D(fig3d)

target_list = model_list[:]
PCA_list_A = np.array([[0,1,2,3,4]])
PCA_list_B = np.array([[0,1,2,3,4]])

average_list = []

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
#     rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN_uniPFCHPC(inputsize, hidden_size, outputsize, batch_size)
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_initw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_initw = np.array(p.data)
            if n == "Re.weight_ih" or n == "Re.weight":
                Re_initw = np.array(p.data)
            if n == "linear.weight":
                Output_initw = np.array(p.data)

    rnn.load_state_dict(torch.load(model_path))

    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "HPC.weight_hh":
                HPC_inw = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw = np.array(p.data)
            if n == "PFC.bias_ih":
                PFC_b = np.array(p.data)
            if n == "PFC.bias_hh":
                PFC_inb = np.array(p.data)
            if n == "HPC.bias_ih":
                HPC_b = np.array(p.data)
            if n == "HPC.bias_hh":
                HPC_inb = np.array(p.data)
            if n == "Re.bias_ih":
                Re_b = np.array(p.data)
            if n == "Re.bias_hh":
                Re_inb = np.array(p.data)
            if n == "linear.weight":
                Out_w = np.array(p.data)
    
# #     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w>0]),bins=80,density=True)
# #     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw>0]),bins=40,density=True)
# #     fig1.plot((make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
# #     fig1.plot((make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
# #     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w<0]),bins=80,density=True)
# #     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw<0]),bins=40,density=True)
#     PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w.nonzero()]),bins=80,density=True)
#     PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw.nonzero()]),bins=40,density=True)
# #     PFC_hist = np.histogram(np.abs(np.append(PFC_w[PFC_w.nonzero()],PFC_inw[PFC_inw.nonzero()])),bins=40,density=True)
# #     PFC_b_hist = np.histogram(np.abs(PFC_b[PFC_b.nonzero()]),bins=80,density=True)
# #     PFC_inb_hist = np.histogram(np.abs(PFC_inb[PFC_inb.nonzero()]),bins=40,density=True)
# #     fig1.plot(np.log(make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01),color=color)
#     fig1.plot(np.log(make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01),color=color)
# #     fig1.plot((make_bins(PFC_hist[1])),np.log(PFC_hist[0]))
#     a1 = regression(make_bins(PFC_w_hist[1]),np.log(PFC_w_hist[0]+0.01))
#     a2 = regression(make_bins(PFC_inw_hist[1]),np.log(PFC_inw_hist[0]+0.01))


# #     HPC_w_hist = np.histogram(HPC_w[HPC_w>0],bins=80,density=True)
# #     HPC_inw_hist = np.histogram(HPC_inw[HPC_inw>0],bins=40,density=True)
# #     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
# #     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
# #     HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w<0]),bins=80,density=True)
# #     HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw<0]),bins=40,density=True)
#     HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w.nonzero()]),bins=80,density=True)
#     HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw.nonzero()]),bins=40,density=True)
# #     HPC_w_hist = np.histogram(np.abs(HPC_w),bins=80,density=True)
# #     HPC_inw_hist = np.histogram(np.abs(HPC_inw),bins=40,density=True)
# #     fig2.plot((make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]))
# #     fig2.plot((make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]))
# #     fig2.plot(np.log(make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]+0.01),color=color)
#     fig2.plot(np.log(make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]+0.01),color=color)
#     b1 = regression(make_bins(HPC_w_hist[1]),np.log(HPC_w_hist[0]+0.01))
#     b2 = regression(make_bins(HPC_inw_hist[1]),np.log(HPC_inw_hist[0]+0.01))


# #     Re_w_hist = np.histogram(Re_w[Re_w>0],bins=100,density=True)
# #     Re_inw_hist = np.histogram(Re_inw[Re_inw>0],bins=100,density=True)
# #     Re_w_hist = np.histogram(np.abs(Re_w[Re_w<0]),bins=100,density=True)
# #     Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw<0]),bins=20,density=True)
#     Re_w_hist = np.histogram(np.abs(Re_w[Re_w.nonzero()]),bins=40,density=True)
#     Re_inw_hist = np.histogram(np.abs(Re_inw[Re_inw.nonzero()]),bins=40,density=True)
# #     Re_inw_hist = np.histogram(np.abs(Re_inw),bins=40,density=True)
#     Re_initw_hist = np.histogram(np.abs(Re_initw),bins=40,density=True)
#     fig3.plot(np.log(make_bins(Re_w_hist[1])),np.log(Re_w_hist[0]+0.01),color=color)
# #     fig3.plot(np.log(make_bins(Re_inw_hist[1])),np.log(Re_inw_hist[0]+0.01),color=color)
# #     fig3.plot(np.log(make_bins(Re_initw_hist[1])),np.log(Re_initw_hist[0]+0.01))
#     c1 = regression(make_bins(Re_w_hist[1]),np.log(Re_w_hist[0]+0.01))
#     c2 = regression(make_bins(Re_inw_hist[1]),np.log(Re_inw_hist[0]+0.01))
    
#     Re_uw,Re_uv = LA.eig(Re_w[:,20:])
#     fig_eigenval.plot(Re_inuw.real,Re_inuw.imag,"o",color=color)
#     print(Re_inuw)
#     print(np.count_nonzero(Re_uw.real>0))
#     print(np.count_nonzero(Re_uw.imag==0))
#     print(np.count_nonzero((Re_uw.imag==0)*(Re_uw.real>0)))


    i = 0
    PFC_inuw,PFC_inuv = LA.eig(PFC_inw[i:i+20])
#     print(PFC_inuw,PFC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=testcolor,zorder=2)
#     else :
#         fig_eigenval.plot(PFC_inuw.real,PFC_inuw.imag,"o",color=color,alpha=0.01,zorder=1)
#     for k in range(20):
#         fig_eigenval.text(PFC_inuw.real[k],PFC_inuw.imag[k],k)
#     print(LA.matrix_rank(PFC_inw[i:i+20]))
#     print(np.count_nonzero(PFC_inuw.real>0))
#     print(np.count_nonzero(PFC_inuw.imag==0))
#     print(np.count_nonzero((PFC_inuw.imag==0)*(HPC_inuw.real>0)))
    
    i = 0
    HPC_inuw,HPC_inuv = LA.eig(HPC_inw[i:i+20])
#     print(HPC_inuw,HPC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color=testcolor,alpha=1,zorder=2)
# #     elif num < 40:
# #         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color="b",alpha=1)
#     else:
#         fig_eigenval.plot(HPC_inuw.real,HPC_inuw.imag,"o",color=color,alpha=0.1,zorder=1)
#     for k in range(20):
#         fig_eigenval.text(HPC_inuw.real[k],HPC_inuw.imag[k],k)
#     print(LA.matrix_rank(HPC_inw[i:i+20]))
#     print(np.count_nonzero(HPC_inuw.real>0))
#     print(np.count_nonzero(HPC_inuw.imag==0))
#     print(np.count_nonzero((HPC_inuw.imag==0)*(HPC_inuw.real>0)))
    
    HPC_uw,HPC_uv = LA.eig(np.pad(HPC_w[i:i+20,2:],[(0,0),(0,0)]))
#     HPC_uw,HPC_uv = LA.eig(np.pad(HPC_w[i:i+20,:2],[(0,0),(0,18)]))
#     print(HPC_uw,HPC_uv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
#         test_colors.append(testnum/len(test_list))
#         fig_eigenval.plot(HPC_uw.real,HPC_uw.imag,"o",color=testcolor,alpha=1,zorder=2)
#     else:
#         fig_eigenval.plot(HPC_uw.real,HPC_uw.imag,"o",color=color,alpha=0.1,zorder=1)
#     print(np.count_nonzero(HPC_uw.real>0))
#     print(np.count_nonzero(HPC_uw.imag==0))
#     print(np.count_nonzero((HPC_uw.imag==0)*(HPC_uw.real>0)))

#     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+2*HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]
    HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]+HPC_inw[0:20]*HPC_w[40:60,2:]+HPC_w[0:20,2:]*HPC_inw[40:60]+HPC_inw[0:20]*HPC_inw[40:60]
#     HPC_allw = HPC_w[0:20,2:]*HPC_w[40:60,2:]
#     HPC_allw = HPC_inw[0:20]
#     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+2*PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]
    PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]+PFC_inw[0:20]*PFC_w[40:60,:20]+PFC_w[0:20,:20]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
#     PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]+PFC_inw[0:20]*PFC_w[40:60,20:]+PFC_w[0:20,20:]*PFC_inw[40:60]+PFC_inw[0:20]*PFC_inw[40:60]
#     PFC_allw = PFC_w[0:20,:20]*PFC_w[40:60,:20]
#     PFC_allw2 = PFC_w[0:20,20:]*PFC_w[40:60,20:]
#     Re_allw  = Re_w[:,0:20] + Re_w[:,20:] + Re_inw
    Re_allw  = Re_w[:,0:20] + Re_inw
#     Re_allw2  = Re_w[:,20:] + Re_inw
#     Re_allw  = Re_w[:,0:20]
#     Re_allw2  = Re_w[:,20:]
#     all_uw,all_uv = LA.eig(HPC_allw)
    all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw)
#     all_uw,all_uv = LA.eig(PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(Re_allw2)
#     all_uw,all_uv = LA.eig(Re_inw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2)
#     all_uw,all_uv = LA.eig(PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * Re_allw2 + PFC_allw2 * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw2 @ Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw * Re_allw2 + HPC_allw * PFC_allw * Re_allw)
#     all_uw,all_uv = LA.eig(HPC_inw[0:20]*HPC_inw[40:60] + HPC_allw @ Re_allw2 + HPC_allw @ PFC_allw @ Re_allw)
#     all_uw,all_uv = LA.eig(PFC_allw * Re_allw * HPC_allw + PFC_allw2 * Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(PFC_allw @ Re_allw @ HPC_allw + PFC_allw2 @ Re_allw + PFC_inw[0:20]*PFC_inw[40:60])
#     all_uw,all_uv = LA.eig(np.pad(Out_w[:,:],[(0,18),(0,0)]))
#     all_uw,all_uv = LA.eig(Out_w[:,:]@(HPC_w[0:20,:2]*HPC_w[40:60,:2]))
#     all_uw,all_uv = LA.eig(np.pad(HPC_w[20:40,:2],[(0,0),(0,18)]))
#     all_uw,all_uv = LA.eig(HPC_inw[20:40])
# #     print(HPC_inuw,HPC_inuv)
#     if model_path in test_list:
#         testnum+=1
#         testcolor=cm.jet(testnum/len(test_list))
# #         testcolor = "r"
#         test_colors.append(testnum/len(test_list))
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color=testcolor,alpha=1,zorder=2)
#         print(model_path)
# #         print(all_uw)
# #         print(all_uv.T)
# #         print(np.count_nonzero(all_uw.real>0))
# #         print(np.count_nonzero(all_uw.imag==0))
# #         print(np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)))
# #         print(np.average(abs(all_uw)))
# #         for k in range(20):
# #             fig_eigenval.text(all_uw.real[k],all_uw.imag[k],str(model_path.split("epoch")[-1].split(".")[0]))
#     else:
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color=color,alpha=0.1,zorder=1)
        
    if model_path in test_list_B:
#         ax3d.plot([np.count_nonzero(all_uw.real>0)], [np.count_nonzero(all_uw.imag==0)], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "b", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [1],"o", color = "b", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))],[np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "b", alpha=0.1)
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color="b",alpha=0.3,zorder=2)
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",alpha=0.3,zorder=2)
        fig_eigenval.plot(np.average(abs(all_uw)),np.max(abs(all_uw)),"o",color="b",alpha=0.3,zorder=2)
#         print(model_path,"good")
#         print([np.average(abs(all_uw)),np.max(all_uw.real),np.min(all_uw.real)])
#         print([np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
        PCA_list_B = np.concatenate((PCA_list_B,[[np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))]]),axis=0)
#         fig_eigenval.text(all_uw.real[np.argmax(abs(all_uw))],all_uw.imag[np.argmax(abs(all_uw))],str(model_path.split("s")[-1]))
#         fig_eigenval.text(np.average(abs(all_uw)),np.max(abs(all_uw)),str(model_path.split("s")[-1]))
    else:
#         ax3d.plot([np.count_nonzero(all_uw.real>0)], [np.count_nonzero(all_uw.imag==0)], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "r", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [0],"o", color = "r", alpha=0.1)
#         ax3d.plot([np.count_nonzero((all_uw.imag==0)*(all_uw.real>0))], [np.count_nonzero((all_uw.imag==0)*(all_uw.real<0))], [np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0))],"o", color = "r", alpha=0.1)
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",color="r",alpha=0.3,zorder=1)
#         fig_eigenval.plot(all_uw.real,all_uw.imag,"o",alpha=0.3,zorder=1)
        fig_eigenval.plot(np.average(abs(all_uw)),np.max(abs(all_uw)),"o",color="r",alpha=0.3,zorder=2)
#         print(model_path,"bad")
#         print([np.average(abs(all_uw)),np.max(all_uw.real),np.min(all_uw.real)])
#         print([np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))])
        PCA_list_A = np.concatenate((PCA_list_A,[[np.count_nonzero((all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)), np.count_nonzero((all_uw.imag==0)*(all_uw.real<0)), np.count_nonzero((all_uw.imag!=0)*(all_uw.real>0)),np.count_nonzero((all_uw.imag!=0)*(all_uw.real<0))]]),axis=0)
#         fig_eigenval.text(all_uw.real[np.argmax(abs(all_uw))],all_uw.imag[np.argmax(abs(all_uw))],str(model_path.split("s")[-1]))    
#         fig_eigenval.text(np.average(abs(all_uw)),np.max(abs(all_uw)),str(model_path.split("epoch")[-1]))      
    
# #     print(LA.matrix_rank(HPC_inw[i:i+20]))
#     print(np.count_nonzero(all_uw.real>0))
#     print(np.count_nonzero(all_uw.imag==0))
#     print(np.count_nonzero((all_uw.imag==0)*(all_uw.real>0)))
    
    average_list.append(np.average(abs(all_uw)))
    
#     PCA_list = np.concatenate((PCA_list,[[a1,a2,c2]]),axis=0)
#     print(a1-a2)
    
#     fig1.hist(PFC_w[PFC_w>0],bins=40,alpha=0.5,density=True)
#     fig1.hist(np.abs(PFC_w[PFC_w<0]),bins=40,alpha=0.5,density=True)
#     fig2.hist(HPC_w[HPC_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig2.hist(np.abs(HPC_w[HPC_w<0]),bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(Re_w[Re_w>0],bins=40,alpha=0.5,log=True,density=True)
#     fig3.hist(np.abs(Re_w[Re_w<0]),bins=40,alpha=0.5,log=True,density=True)

# # # print(PCA_list[1:])
# PCA_list = np.concatenate((PCA_list_A[1:],PCA_list_B[1:]),axis=0)
# length = PCA_list_A[1:].shape[0]
# pca = PCA()
# dfs = PCA_list
# pca.fit(dfs)
# feature = pca.transform(dfs)
# # print(length,feature)
# plt.figure(figsize=(6, 6))
# plt.scatter(feature[:length, 0], feature[:length, 1], color="r",alpha=0.2)
# plt.scatter(feature[length:, 0], feature[length:, 1], color="b",alpha=0.2)
# #plt.scatter(feature[100:200, 0], feature[100:200, 1], alpha=0.8)
# plt.grid()
# plt.xlabel("PC1")
# plt.ylabel("PC2")
# # for i,n in enumerate(target_list):
# #     plt.annotate(n[-12:],(feature[i, 0], feature[i, 1]))
# plt.show()

# print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
# print(pd.DataFrame(pca.components_, columns=["Hidden{}".format(x + 1) for x in range(dfs.shape[1])], index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))
    

# fig3d = plt.figure()
# ax3d = Axes3D(fig3d)
# ax3d.plot(feature[:length, 0], feature[:length, 1], feature[:length, 2], "o", alpha=0.1)
# ax3d.plot(feature[length:, 0], feature[length:, 1], feature[length:, 2], "o", alpha=0.1)
# plt.show()

# gradient = np.array(test_colors)
# gradient = np.vstack((gradient, gradient))

# fig, ax = plt.subplots()
# ax.imshow(gradient, cmap=plt.cm.jet)
# ax.set_axis_off()
print(np.count_nonzero(np.array(average_list)<0.1))

In [ ]:
#############  compare list 作成用  ######################

import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA



class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    
class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.LSTM1 = nn.LSTMCell(input_size, hidden_size)
        self.LSTM2 = nn.LSTMCell(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    def forward(self, input, hiddens):
        hidden1 = self.LSTM1(input, hiddens[0])
        hidden2 = self.LSTM2(hidden1[0], hiddens[1])
        output = self.linear(hidden2[0])
        return output, [hidden1,hidden2]

    def initHidden(self):
        hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [hidden,hidden]

    def initHidden_rand(self):
        hidden = [torch.rand(self.batch_size, self.hidden_size)*0.2, torch.rand(self.batch_size, self.hidden_size)*0.2]
        return [hidden,hidden]    

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

def check_bifur_posi(data):
    data = data - 0.5
    positive = np.count_nonzero(data>=0)
    negative = np.count_nonzero(data<0)
    
    return np.abs(positive - negative)

# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
model_list = glob.glob('model/R20/*s*_100_*.pth')
# model_list = glob.glob('model/R20_cont_add/*OUT1**s*_100_*.pth')
# model_list = glob.glob('model/compare_cont/*s*_100_??.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)

test_list = glob.glob('model/R20/*s1*_100_1_*.pth')
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# # target = [1,3,4,11,12,13,16,-2,-1]
# # target = [0,2,5,6,7,8,9,10,14,15,17]
# # target = [11,12,-2]
# # target = [19,20,21,22]
# test_target = []

# for i in target:
#     test_target.append(test_list[i])
# test_list = test_target

# print(test_list)

# f = open("compare_contgood_list_add.txt",mode="w")

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig = plt.figure()
test_colors = []

target_list = model_list[:]
PCA_list = np.array([[0,1,2]])

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 7
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)

#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
    delays = pick_traj(traj[:,0], np.array(traj)[:])[3:-1]
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    bifur = np.array([])
    for data in delays:
        check_bifur = np.argmax(np.abs(np.array(data)[:,0,0] - 0.5))
        bifur = np.append(bifur,data[check_bifur,:,0])
#     print(bifur,check_bifur)
#     print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
    if bifur.shape[0] == 0:
        continue
#     if np.var(bifur) < np.abs(np.median(bifur)-np.mean(bifur)) or np.var(bifur) < 0.0001:
# #     if np.abs(np.median(bifur)-np.mean(bifur)) < 0.0001 and np.var(bifur) < 0.0001:
# #         print(model_path)
# #         f.write(model_path+"\n")
#         pass
#     else:
#         print(model_path)
#         plt.plot(np.log(np.var(bifur)),np.log(np.median(bifur)-np.mean(bifur)),"o")
#         print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
#         f.write(model_path+"\n")
# #         pass

    if np.var(bifur) > 0.0001:
#     if np.abs(np.median(bifur)-np.mean(bifur)) < 0.0001 and np.var(bifur) < 0.0001:
        print(model_path)
        print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)),check_bifur_posi(bifur))
#         print(np.median(bifur),np.mean(bifur),bifur)
#         pass

f.close()

In [ ]:
########### imihu test ####################

import glob
from sklearn import linear_model
import matplotlib.cm as cm
import numpy.linalg as LA



class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.normal_(self.PFC.weight_ih.data,0,1)
        nn.init.normal_(self.HPC.weight_ih.data,0,1)
        nn.init.normal_(self.Re.weight_ih.data,0,0.1)
        nn.init.normal_(self.PFC.weight_hh.data,0,1)
        nn.init.normal_(self.HPC.weight_hh.data,0,1)
        nn.init.normal_(self.Re.weight_hh.data,0,0.1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
#         HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
#         PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]

def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def regression(datax,datay):
    clf = linear_model.LinearRegression()
    x = datax[datay!=-float("inf")].reshape(-1,1)
    y = datay[datay!=-float("inf")].reshape(-1,1)
    clf.fit(x,y)
    coef = clf.coef_
    score = clf.score(x,y)
    print(coef,score)
    return coef[0][0]

def check_bifur_posi(data):
    data = data - 0.5
    positive = np.count_nonzero(data>=0)
    negative = np.count_nonzero(data<0)
    
    return np.abs(positive - negative)



# model_list = glob.glob('model/ReModel_interRNN_long_s5_200_1.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_OUT5_1212121_s*_100_1_2_2.pth')
# model_list = glob.glob('model/ReModel_L2_interRNNrand_AddRe_PnoiseRe_OUT5_161_s*_100_4.pth')
# model_list = glob.glob('model/R+2/*s2*_100_1*.pth')
# model_list = glob.glob('model/P+/*s3_100_1*add.pth')
# model_list = glob.glob('model/R20/*s*_100_*.pth')
model_list = glob.glob('model/R20_cont_add/*OUT1**s*_100_*epoch??.pth')


model_list = sorted(model_list)
model_list = sorted(model_list,key=len,reverse=False)

# print(model_list)

test_list = glob.glob('model/R20/*s1*_100_1_*.pth')
test_list = sorted(test_list)
test_list = sorted(test_list,key=len,reverse=False)[:]

# # target = [1,3,4,11,12,13,16,-2,-1]
# # target = [0,2,5,6,7,8,9,10,14,15,17]
# # target = [11,12,-2]
# # target = [19,20,21,22]
# test_target = []

# for i in target:
#     test_target.append(test_list[i])
# test_list = test_target

# print(test_list)

training_size = 100
test_size = 1000
epochs_num = 10
hidden_size = 20
batch_size = 10
data_length = 100
inputsize = 2
outputsize = 2

fig_weight = plt.figure(figsize=(10,5))
fig1 = fig_weight.add_subplot(131)
fig2 = fig_weight.add_subplot(132)
fig3 = fig_weight.add_subplot(133)

# target = [22,23,-2,-1]
# target_list = []

# for i in target:
#     target_list.append(model_list[i])

fig = plt.figure()
test_colors = []

target_list = model_list[:]
PCA_list = np.array([[0,1,2]])

testnum = 0
num=0
for model_path in target_list:
    num+=1
    color=cm.jet(num/len(target_list))
#     print(model_path)
    
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 7
    est_length = 8
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)

#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
    delays = pick_traj(traj[:,0], np.array(traj)[:])[3:-1]
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    bifur = np.array([])
    for data in delays:
        check_bifur = np.argmax(np.abs(np.array(data)[:,0,0] - 0.5))
        bifur = np.append(bifur,data[check_bifur,:,0])
#     print(bifur,check_bifur)

    if bifur.shape[0] == 0:
        continue
    if np.var(bifur) > 0.0001:
#     if np.abs(np.median(bifur)-np.mean(bifur)) < 0.0001 and np.var(bifur) < 0.0001:
        print(model_path)
        print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)),check_bifur_posi(bifur))
#         print(np.median(bifur),np.mean(bifur),bifur)
#         pass
    else:
#         print(model_path)
#         plt.plot(np.log(np.var(bifur)),np.log(np.median(bifur)-np.mean(bifur)),"o")
#         print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
#         f.write(model_path+"\n")
        pass

f.close()

In [ ]:
############## uni PFC test ################



import collections

def Culc_gate(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
#             traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
            traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
#     order = [1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(states[start:])
    return states_list

def pick_delay_num(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(int(start))
        start = int(k)
    states_list.append(int(start))
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result


class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.1
        var = 0.1
#         HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
#         PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.01
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.05
        index = np.array([11])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.ones(self.batch_size, index.size)*100
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.ones(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
class MyLSTM_RNN_uniPFC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=1):
        super(MyLSTM_RNN_uniPFC, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)
        # nn.init.normal_(self.PFC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.Re.weight_ih.data,0,0.1/10)
        # nn.init.normal_(self.PFC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.Re.weight_hh.data,0,1/10)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = hiddens[1][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    
    def noiseHidden_rand(self, hiddens):
        c = 0.02
        v = 0.02
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*v
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def hidden_size_list(self):
        return [self.hidden_size_PFC,self.hidden_size_HPC,self.hidden_size_Re]
    
    
class MyLSTM_RNN_uniHPC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=1):
        super(MyLSTM_RNN_uniHPC, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)
        # nn.init.normal_(self.PFC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.Re.weight_ih.data,0,0.1/10)
        # nn.init.normal_(self.PFC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.Re.weight_hh.data,0,1/10)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = hiddens[0][0]
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0.02
        v = 0.02
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*v
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def initHidden_test(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    
    def hidden_size_list(self):
        return [self.hidden_size_PFC,self.hidden_size_HPC,self.hidden_size_Re]


class MyLSTM_RNN_uniPFCHPC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=1):
        super(MyLSTM_RNN_uniPFCHPC, self).__init__()

        self.hidden_size_PFC = hidden_size+0
        self.hidden_size_HPC = hidden_size+0
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.Re.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)
        nn.init.sparse_(self.Re.weight_hh.data,sparse)
        # nn.init.normal_(self.PFC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_ih.data,0,1/10)
        # nn.init.normal_(self.Re.weight_ih.data,0,0.1/10)
        # nn.init.normal_(self.PFC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.HPC.weight_hh.data,0,1/10)
        # nn.init.normal_(self.Re.weight_hh.data,0,1/10)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = hiddens[0][0]
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = hiddens[1][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0.02
        v = 0.02
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*v
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*c
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def hidden_size_list(self):
        return [self.hidden_size_PFC,self.hidden_size_HPC,self.hidden_size_Re]

    



def moving_average(data):
    y = np.ones(9)/9
    mean_seq = np.convolve(data, y)
    return mean_seq


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 5

    colors = ['C0','C1','C2','C3','C4','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_epoch100.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s4_100_2_epoch115.pth'
#     model_path = 'model/R20_uniPFC/ReModel_L2_interRNNrand_OUT1_uniPFC_151_s1_100_3_epoch195.pth'
#     model_path = 'model/R20_uniHPC_151/ReModel_L2_interRNNrand_OUT1_uniHPC_151_s1_100_1_epoch185.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s1_100_2_epoch195.pth'
#     model_path = 'model/R20_uniPFCHPC_151/ReModel_L2_interRNNrand_OUT1_uniPFCHPC_151_s3_100_2_epoch135.pth'
    model_path = 'model/R20_traintest/ReModel_L2_interRNNrand_OUT1_131_s1_100_1_epoch50.pth'
#     model_path = 'model/R20_131/ReModel_L2_interRNNrand_OUT1_131_s5_100_1_epoch85.pth'
#     model_path = 'model/R20_slow_bigbatch/ReModel_L2_interRNNrand_OUT1_131_s6_100_3_epoch115.pth'


    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 87
    est_length = 4
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            output,hidden = rnn(output,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
    print(np.array(Gate_states).shape)
            
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_a = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_a = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_a = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_a = np.array(p.data)
                
    delays = pick_delay_num(traj[:,0], np.array(Restate)[:])
    
#     model_path = 'model/R20/ReModel_L2_interRNNrand_OUT5_151_s2_100_1_epoch100.pth'
#     model_path = 'model/R20_cont/ReModel_L2_interRNNrand_OUT1_151_s4_100_2_epoch125.pth'
#     model_path = 'model/R20_Retrain/ReModel_L2_interRNNrand_OUT5_151_s4_100_1_TrainOUT_epoch0.pth'
#     model_path = 'model/R20_uniPFC/ReModel_L2_interRNNrand_OUT1_uniPFC_151_s1_100_1_epoch195.pth'
#     model_path = 'model/R20_cont_add/ReModel_L2_interRNNrand_OUT1_151_s1_100_2_epoch195.pth'
    rnn.load_state_dict(torch.load(model_path))
    
#     for n, p in rnn.named_parameters():
#         if n == "HPC.weight_ih":
#             p.data[inputsize:,hidden_size:].sub_(p.data[inputsize:,hidden_size:])
 
    
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_x, batch_size)
#     data_limit = 20
    init_point = init_point*1
    Delayflag = False
    Delaynum = 0
    Delaytime = 0
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
    for k in range(data.shape[0]*est_length):
            if traj_noise[-1][0][1] < 0.1:
                Delayflag = True
#                 startpoint = delays[Delaynum]
                for cell in range(20):
#                     print(hidden[2][0][cell+0],np.array(Restate)[int(delays[Delaynum])+Delaytime,cell+0]-moving_average(np.array(Restate)[:,cell+0])[4+int(delays[Delaynum])+Delaytime])
#                     hidden[2][0][cell+8] = np.average(np.array(Restate_noise)[-6:,cell+8])
#                     hidden[2][0][cell+8] = np.average(np.array(Restate)[k-5+data_limit:k+5+data_limit,cell+8])
#                     hidden[2][0][cell+0] = moving_average(np.array(Restate)[:,cell+0])[4+k+data_limit]
#                     hidden[2][0][cell+0] = moving_average(np.array(Restate)[:,cell+0])[4+int(delays[Delaynum])+Delaytime]
#                     hidden[2][0][cell+0] = np.array(Restate)[startpoint+Delaytime,cell+0]-moving_average(np.array(Restate)[:,cell+0])[4+startpoint+Delaytime]
                    hidden[2][0][cell+0] = np.array(Restate)[k+data_limit,cell+0] - moving_average(np.array(Restate)[:,cell+0])[4+k+data_limit]
                Delaytime += 1
            hidden = rnn.noiseHidden_rand(hidden)
            hidden = rnn.noiseHidden_select(hidden)
            output,hidden = rnn(output,hidden) 
#             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
#             hidden = rnn.noiseHidden_rand(hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
            if Delayflag == True and traj_noise[-1][0][1] > 0.1:
                Delayflag = False
                Delaynum += 1
                Delaytime = 0
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()
    
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_b = np.array(p.data)
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
#     MakeAnimation(traj[:,0,0],traj[:,0,1], traj_noise[:,0,0], traj_noise[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

    fig = plt.figure()
    plt.plot(traj[:,0,0],traj[:,0,1])
    plt.plot(traj_noise[:,0,0],traj_noise[:,0,1])
    plt.show()

    for i in range(20):
        plt.figure()
#         plt.ylim(-1,1)
        plt.plot(traj[:,0,0],alpha=0.5)
        plt.plot(traj_noise[:,0,0],alpha=0.5)
#         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
        for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
            plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
            plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
#             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
#             plt.plot(moving_average(np.array(Restate)[:,i])[2:],alpha=1)
#             plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#             plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
#             plt.plot(np.array(Restate)[:,i]-moving_average(np.array(Restate)[:,i])[4:-4],alpha=0.5)
#             plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
#             plt.plot(np.array(Gate_states)[:,1,0+i],alpha=0.5)
#             plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
#             plt.plot(np.abs(np.array(Gate_states)[:,1,0+i]-np.array(Gate_states_noise)[:,1,0+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states)[:,0,0+i]/np.array(Gate_states)[:,0,20+i]),alpha=0.5)
#             plt.plot(np.log(np.array(Gate_states_noise)[:,0,0+i]/np.array(Gate_states_noise)[:,0,20+i]),alpha=0.5)
        plt.title("neuron#"+str(i+1))
    
#     for i in range(20):
#         plt.figure()
#         plt.ylim(-1,1)
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         plt.plot(traj[:,0,0])
#         plt.plot(traj_noise[:,0,0])
#         for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:300,i],alpha=0.5)
# #             plt.plot(np.array(Restate_noise)[:300,i],alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate)[:300,i]),alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate_noise)[:300,i]),alpha=0.5)
#         plt.title("HPCneuron#"+str(i+1))
    
#     plt.figure()
#     a = np.abs(np.corrcoef(np.array(PFCstate)[:,0].T) - np.corrcoef(np.array(PFCstate_noise)[:,0].T))
# #     a = np.abs(np.corrcoef(np.array(Restate)[:].T) - np.corrcoef(np.array(Restate_noise)[:].T))
#     plt.imshow(a,cmap=plt.get_cmap("Reds"))
    
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
    delays = pick_traj(traj[:,0], np.array(PFCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj[:,0], np.array(Restate)[:])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
#     delays = pick_traj(traj_noise[:,0], np.array(traj_noise)[:])[3:-1]
#     delays = pick_traj(traj[:,0], np.array(Gate_states)[:,1])
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
    bifur = np.array([])
    for i in range(20):
        plt.figure()
        plt.ylim(-1,1)
#         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
        for data in delays[1:-1]:
#             plt.plot(np.array(data)[:,0+i,0],alpha=0.5)
            plt.plot(np.array(data)[:,0+i],alpha=0.5)
            print(len(data))
#             check_bifur = np.argmax(np.abs(np.array(data)[:,0+i,0] - 0.5))
#             bifur = np.append(bifur,data[check_bifur,i,0])
#             print(check_bifur,data[check_bifur,i,0])
        plt.title("neuron#"+str(i+1))
#     print(np.var(bifur),np.median(bifur)-np.mean(bifur))

#     for i in range(20):
#         plt.figure()
#         for k in range(1):
#             k+=0
#             data = PFC_w_a[i]
#             plt.plot(data)
#             data = PFC_w_b[i]
#             plt.plot(data)

#     length = 140
# #     plt.figure()
# #     plt.plot(np.array(Restate_noise)[:length,0])
# #     plt.show()
#     neuron_list = []
#     neuron_list_Re = []
#     for i in range(20):
#         plt.figure()
#         neurons_a = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0]),axis=1)
#         neurons_b = np.concatenate((np.array(PFCstate_noise)[:,0],np.array(HPCstate_noise)[:,0]),axis=1)
#         plt.plot(np.array(Restate)[:length,i],alpha=0.3)
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(40):
#             k += 0
#             data_a = neurons_a[:length,k]*Re_w_a[i,k]
#             data_b = neurons_b[:length,k]*Re_w_b[i,k]
#             diff = data_a - data_b
#             if np.sum(np.abs(data_a)) > 5 and np.sum(np.abs(data_a)) > 5:
#                 print("a",k,Re_w_a[i,k],np.sum(np.abs(data_a)))
#                 plt.plot(data_a)
#             if np.sum(np.abs(data_b)) > 5 and np.sum(np.abs(data_b)) > 5:
#                 print("b",k,Re_w_b[i,k],np.sum(np.abs(data_b)))
#                 plt.plot(data_b)
#             if np.sum(np.abs(diff)) > 5 and np.sum(np.abs(diff)) > 5:
#                 print("diff",k,Re_w_a[i,k],Re_w_b[i,k],np.sum(np.abs(diff)))
#                 neuron_list.append(k+1)
# #                 plt.plot(diff)
            
#     for i in range(20):
#         plt.figure()
#         i+=0
#         neurons_a = np.array(Restate)[:]
#         neurons_b = np.array(Restate_noise)[:]
#         plt.plot(np.array(Restate)[:length,i],alpha=0.3)
#         plt.plot(np.array(Restate_noise)[:length,i],alpha=0.3)
#         plt.ylim(-1,1)
#         for k in range(20):
#             k += 0
#             data_a = neurons_a[:length,k]*Re_inw_a[i,k]
#             data_b = neurons_b[:length,k]*Re_inw_b[i,k]
#             diff = data_a - data_b
#             if np.sum(np.abs(data_a)) > 5 and np.sum(np.abs(data_a)) > 5:
#                 print("a",k,Re_inw_a[i,k],np.sum(np.abs(data_a)))
#                 plt.plot(data_a)
#             if np.sum(np.abs(data_b)) > 5 and np.sum(np.abs(data_b)) > 5:
#                 print("b",k,Re_inw_b[i,k],np.sum(np.abs(data_b)))
#                 plt.plot(data_b)
#             if np.sum(np.abs(diff)) > 5 and np.sum(np.abs(diff)) > 5:
#                 print("diff",k,Re_inw_a[i,k],Re_inw_b[i,k],np.sum(np.abs(diff)))
#                 neuron_list_Re.append(k+1)
# #                 plt.plot(diff)

    pca = PCA()
    dfs = np.array(PFCstate)[0:,0]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    PFC_feature = np.copy(feature)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)

    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
#     ax3d.plot(feature[:100, 0], feature[:100, 1], feature[:100, 2], alpha=0.8)
#     ax3d.plot(feature[100:, 0], feature[100:, 1], feature[100:, 2], alpha=0.8)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2], alpha=0.8)
    ax3d.plot(feature[0:1, 0], feature[0:1, 1], feature[0:1, 2],"o", alpha=1)
    plt.show()

#     print(collections.Counter(neuron_list))
#     print(collections.Counter(neuron_list_Re))

    plt.figure()
    plt.plot(traj[:,0,0],alpha=0.5)
    plt.plot(traj_noise[:,0,0],alpha=0.5)
    
if __name__ == '__main__':
    main()

In [ ]:
### Accuray check ###
%matplotlib notebook
import glob
import matplotlib.pyplot as plt
import numpy as np
import random
plt.rcParams["font.size"] = 16

def epoch_count(path):
    epoch_list = np.arange(0,400,5)
    epoch_num_list = []
    epoch_rate_list = []
    with open(path+"good_list.txt", mode="r") as f:
        good_list = f.read().splitlines()
    for i in epoch_list:
        model_list = glob.glob(path+'*epoch'+str(i)+'.pth')
        n = 0
        for model_path in model_list:
            if model_path in good_list:
                n+=1
        epoch_num_list.append(n)
        epoch_rate_list.append(n/len(model_list))
    return epoch_num_list,epoch_rate_list

def epoch_count_b(path):
#     model_list = glob.glob(path+'*OUT1*.pth')
#     sparse_max = np.max(np.array(int(model_list.split("_s")[-1].split("_100")[0])))
    sparse_max = 10
#     index_max = np.max(np.array(int(model_list.split("_100_")[-1].split("_epoch")[0])))
    index_max = 10
    sparse_list = []
    for i in range(sparse_max):
        for k in range(index_max):
            sparse_list.append("s"+str(i+1)+"_100_"+str(k+1))
    epoch_num_list = []
    epoch_rate_list = []
    with open(path+"good_list.txt", mode="r") as f:
        good_list = f.read().splitlines()
    for i in sparse_list:
        n = -1
        model_list = glob.glob(path+'*'+str(i)+'_epoch*.pth')
        model_list = sorted(model_list)
        model_list = sorted(model_list,key=len,reverse=False)
        for model_path in model_list:
            if model_path in good_list:
                n = model_path.split("_epoch")[-1].split(".pth")[0]
#                 print(n)
                break
        epoch_num_list.append(int(n))
        
        
# ##### simple version ##########        
#     epoch_list = np.arange(0,200,5)
#     cumulate = [0]
#     cumulate_rate = []
# #     modelnum = len(glob.glob(path+'*epoch5.pth'))
#     modelnum = len(glob.glob(path+'*100_1_*epoch5.pth'))*index_max
#     for k in epoch_list:
# #         model_list = glob.glob(path+'*epoch'+str(k)+'.pth')
#         cumulate.append(cumulate[-1]+np.count_nonzero(epoch_num_list == k))
#         cumulate_rate.append(cumulate[-1]/modelnum)
# #         print(modelnum)
#     return epoch_num_list, cumulate, cumulate_rate

##### divided version ##########
    epoch_list = np.arange(0,265,5)
    divided_list = []
    result_list = []
    result_rate_list = []
#     modelnum = len(glob.glob(path+'*epoch5.pth'))
    divide_num = 5
    modelnum = int(len(epoch_num_list)/divide_num)
    cumulate_result = np.zeros(epoch_list.size)
    cumulate_rate_result = []
    
#     random.shuffle(epoch_num_list)  # shuffle control
    
    for i in range(divide_num):
        divided_list.append(epoch_num_list[modelnum*i:modelnum*(i+1)])
    for epoch_num_divided in divided_list:
        cumulate = [0]
        cumulate_rate = []
        for k in epoch_list:
    #         model_list = glob.glob(path+'*epoch'+str(k)+'.pth')
            cumulate.append(cumulate[-1]+np.count_nonzero(epoch_num_divided == k))
            cumulate_rate.append(cumulate[-1]/modelnum)
    #         print(modelnum)
        cumulate_result += cumulate[1:]
        cumulate_rate_result.append(cumulate_rate)
    return divided_list, cumulate, np.mean(cumulate_rate_result,axis=0), np.std(cumulate_rate_result,axis=0)

# colorlist = ["r","#ff7f0e","y"]
colornum = 0
plt.figure()
plt.ylim(-0.1,1.1)
# plotlist = ["model/R20_cont_add/","model/compare30_151/","model/PFCHPC_30_151/","model/single60_151/"]
# plotlist = ["model/PFCHPC_30_151/","model/single60_151/"]
# plotlist = ["model/R20_cont_add/","model/compare_cont/"]
delay_length = 3
# plotlist = ["model/R20_1"+str(delay_length)+"1/","model/compare30_1"+str(delay_length)+"1/","model/PFCHPC_30_1"+str(delay_length)+"1/"]
# plotlist = ["model/R20_traintest/","model/compare30_traintest/","model/PFCHPC_30_1"+str(delay_length)+"1/"]
plotlist = ["model/R20_traintest/","model/compare30_traintest/","model/PFCHPC30_traintest/"]
# plotlist = ["model/R20_traintest_bigbatch/","model/compare30_traintest_bigbatch/","model/PFCHPC30_1"+str(delay_length)+"1_bigbatch/"]
# plotlist = ["model/R20_traintest/","model/R20_traintest_bigbatch/"]
# plotlist = ["model/compare_1"+str(delay_length)+"1/","model/compare25_1"+str(delay_length)+"1/","model/compare30_1"+str(delay_length)+"1/"]
# plotlist = ["model/R20_H/","model/R20_H_transferB/","model/R20_uniPFC_H/","model/compare30_H/","model/PFCHPC30_H/"]
# plotlist = ["model/R20_H/","model/R20_H_transferB/","model/compare30_H/","model/PFCHPC_H/","model/compare_transferB/","model/R20_H_transfer2/","model/R20_H_transfer3/"]
# plotlist = ["model/R20_H/","model/R20_H_transferB/","model/compare30_H/","model/PFCHPC_H/","model/compare_transferB/","model/R20_H_transfer2/","model/R20_H_transfer3/"]
# plotlist = ["model/R20_H_transfer3/","model/R20_H_transfer3B/","model/compare30_transfer2/","model/compare30_transfer2B/"]
# plotlist = ["model/R20_H_transfer3/","model/compare30_transfer3/"]
# plotlist = ["model/R20_cont_add/","model/R20_uniPFC/","model/R20_uniPFCHPC_151/","model/R20_uniHPC_151/"]
# plotlist = ["model/R20_131/","model/R20_uniPFC_131/"]
# plotlist = ["model/PFCHPC_30_1"+str(delay_length)+"1/","model/PFCHPC_1"+str(delay_length)+"1/"]
# plotlist = ["model/R20_cont_add/","model/R20_uniPFC/","model/R20_uniPFCHPC_151/","model/R20_uniHPC_151/","model/PFCHPC_30_151/"]
# plotlist = ["model/compare30_H/","model/compare30_transfer3/","model/compare30_transfer3B/"]
# plotlist = ["model/compare30_transfer/","model/compare30_transfer2/","model/compare30_transfer3/"]
# plotlist = ["model/R20_H/","model/compare30_H/","model/PFCHPC30_H/","model/R20_uniPFC_H/"]
# plotlist = ["model/R20_feedReinhReLU_H_bigbatch/","model/compare25_H_bigbatch/","model/PFCHPC25_H_bigbatch/"]
# plotlist = ["model/R20_traintest_bigbatch/","model/R20_H_bigbatch/","model/compare30_traintest_bigbatch/","model/compare30_H_bigbatch/"]
# plotlist = ["model/R20_H_bigbatch/","model/compare30_H_bigbatch/","model/PFCHPC30_H_bigbatch/"]
# plotlist = ["model/R20_H_bigbatch/","model/PFCHPC30_H_bigbatch/"]
# plotlist = ["model/R20_H_bigbatch/","model/R20_H_uniHPC_bigbatch/","model/R20_H_uniPFC_bigbatch/"]
# plotlist = ["model/R20_H_bigbatch/","model/R20_H_stopinit_bigbatch/"]
# plotlist = ["model/R20_H/","model/R20_uniPFC_H/"]
# plotlist = ["model/R20_131/","model/R20_141/","model/R20_cont_add/","model/R20_161/","model/R20_171/"]
# plotlist = ["model/compare_131/","model/compare_141/","model/compare_cont/","model/compare_161/","model/compare_171/"]
# plotlist = ["model/R20_cue_1"+str(delay_length)+"1/","model/compare30_cue_1"+str(delay_length)+"1/","model/PFCHPC_30_cue_1"+str(delay_length)+"1/","model/PFCHPC_30_cue_1to3/"]
# plotlist = ["model/R20_cue_1"+str(delay_length)+"1_smallbatch5/","model/compare30_cue_1"+str(delay_length)+"1_smallbatch5/","model/PFCHPC30_cue_1"+str(delay_length)+"1_smallbatch5/"]
# plotlist = ["model/R20_cue_131_1to3/","model/compare30_cue_1to3/","model/PFCHPC_30_cue_1to3/","model/compare30_cue_1to3_cue2/"]
# plotlist = ["model/R20_cue_1to3_smallbatch5/","model/compare30_cue_1to3_smallbatch5/","model/PFCHPC30_cue_1to3_smallbatch5/"]
# plotlist = ["model/R20_cue_1to3_smallbatch4/","model/compare30_cue_1to3_smallbatch4/","model/PFCHPC30_cue_1to3_smallbatch4/"]
# plotlist = ["model/R20_cue_1until3_smallbatch5/","model/compare30_cue_1until3_smallbatch5/","model/PFCHPC30_cue_1until3_smallbatch5/"]
# plotlist = ["model/R20_cue_1until3_smallbatch4/","model/compare30_cue_1until3_smallbatch4/","model/PFCHPC30_cue_1until3_smallbatch4/"]
# plotlist = ["model/R20_cue_1until3_smallbatch5_decay/","model/compare30_cue_1until3_smallbatch5_decay/"]
# plotlist = ["model/R20_traintest/"]

# plotlist = ["model/R20_traintest/","model/R20_H_bigbatch/","model/R20_cue_1until3_smallbatch5/"]


epoch_list = np.arange(0,265,5)
colorlist = ["C0","C8","C9"]
colornum=0
for model in plotlist:
    a,b,c,v = epoch_count_b(model)
    print(model)
    print(a,b,c)
#     plt.plot(c)
    plt.plot(epoch_list,c,color=colorlist[colornum])
    plt.fill_between(epoch_list,c-v,c+v, color=colorlist[colornum], alpha=0.25)
    colornum+=1
plt.yticks(fontsize=26)
plt.xticks(fontsize=26)


In [ ]:
def choose_maxmodel(path,sparse,indexnum):
    model_list = glob.glob(path+'*s'+str(sparse)+'_100_'+str(indexnum)+'_*epoch*.pth')
    model_list = sorted(model_list)
    model_list = sorted(model_list,key=len,reverse=False)
    n = model_list[-1].split("_epoch")[-1].split(".pth")[0]
    return model_list[-1], int(n)

sparse = 1
indexnum = 2
model_path,past_epochnum = choose_maxmodel("model/R20_cont_add/",sparse,indexnum)
print(model_path,past_epochnum)

In [ ]:
##########    Test for mixture weight!!!!!!!!!!!!!   #########################
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import expit
import pandas as pd

%matplotlib notebook

plt.rcParams["font.size"] = 20
plt.rcParams["figure.subplot.left"] = 0.15

def Culc_gate(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = torch.cat([hiddens[1][0][0],hiddens[2][0]])
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#             traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)


def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
#     order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(traj[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(traj[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result

def vec_var(datas):
    datas = np.array(datas)
    average = np.average(datas,axis=0)
    result = 0
    for data in datas:
        result += np.linalg.norm(data-average)
    result /= datas.shape[0]
    return result
    
def lyapunov_exp(data):
    result = np.mean(np.log(np.abs(np.diff(data))))
    return result

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
#         HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
#         PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens


def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq    


def main(model):
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 3

    
    colors = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
    
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
        
#     model_path = 'model/R20_131/ReModel_L2_interRNNrand_OUT1_131_s6_100_1_epoch125.pth'
    model_path = model
    rnn.load_state_dict(torch.load(model_path))
        
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_pre = torch.clone(p.data)
            if n == "HPC.weight_ih":
                HPC_w_pre = torch.clone(p.data)
            if n == "Re.weight_ih":
                Re_w_pre = torch.clone(p.data)
            if n == "Re.weight_hh":
                Re_inw = torch.clone(p.data)
            if n == "linear.weight":
                OUT_w_pre = torch.clone(p.data)
                   
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_test()
    data_limit = 100
    est_length = 0
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
#     for k in range(data.shape[0]*est_length):
#             output,hidden = rnn(output,hidden)
#             traj.append(output.tolist())
#             PFCstate.append(hidden[0][0].tolist())
#             HPCstate.append(hidden[1][0].tolist())
#             Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_test()
    data = mkOwnRandomBatch(train_y, batch_size)
#     init_point = torch.rand(10,2)*1
#     data_limit = 125
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
            Gate_states_noise.append(Culc_gate(output,params,hidden))
#     for k in range(data.shape[0]*est_length):
# #             hidden = rnn.noiseHidden_rand(hidden)
#             output,hidden = rnn(output,hidden) 
# #             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
# #             hidden = rnn.noiseHidden_rand(hidden)
#             traj_noise.append(output.tolist())
#             PFCstate_noise.append(hidden[0][0].tolist())
#             HPCstate_noise.append(hidden[1][0].tolist())
#             Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")


    plt.figure()
    plt.plot(traj[:,0,0],color=colors[-1])
    plt.plot(traj_noise[:,0,0],color=colors[-2])
    plt.yticks(fontsize=28)
    plt.xticks(fontsize=28)


#     PFC_corrlist = np.array([])
#     HPC_corrlist = np.array([])
#     cross_corrlist = np.array([])
#     for i in range(20):
#         plt.figure()
# #         plt.ylim(-1,1)
#         plt.plot(traj[:,0,0])
#         plt.plot(traj_noise[:,0,0])
# #         plt.plot(np.abs(traj[:,0,0]-traj_noise[:,0,0]))
#     #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
# #                 plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
# #                 plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),alpha=0.5)
# #                 plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
# #                 plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]),alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:,i],alpha=0.5)
# #             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate)[:,i]),alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #                 plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
# #                 plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
# #                 plt.plot(np.array(Gate_states_noise)[:,1,20+i],alpha=0.5)
# #                 plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
# #                 plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
# #                 print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]))[0,1])
# #                 print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]))[0,1])
# #                 print(np.corrcoef(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i])))
#                 cross_corrlist = np.append(cross_corrlist,np.corrcoef(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i])))
#                 PFC_corrlist = np.append(PFC_corrlist,np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]))[0,1])
#                 HPC_corrlist = np.append(HPC_corrlist,np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]))[0,1])
# #         plt.title("neuron#"+str(i+1))
#     print(np.mean(PFC_corrlist),np.mean(HPC_corrlist),np.mean(cross_corrlist))
    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
    dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
#     dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("PFC correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
    PFC_a = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,0]-feature[data_limit:,0]))[0,1]
    PFC_b = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,1]-feature[data_limit:,1]))[0,1]
    PFC_c = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,2]-feature[data_limit:,2]))[0,1]
    
    plt.figure()
    plt.plot(np.abs(feature[:100,0]-feature[100:,0]))
    plt.plot(np.abs(feature[:100,1]-feature[100:,1]))
    plt.plot(np.abs(feature[:100,2]-feature[100:,2]))
    plt.yticks(fontsize=28)
    plt.xticks(fontsize=28)
    plt.show()

    plt.figure()
    plt.plot(np.abs(feature[:100,0]),color=colors[-3])
    plt.plot(np.abs(feature[100:,0]),color=colors[-4])
    plt.yticks(fontsize=28)
    plt.xticks(fontsize=28)
    plt.show()

#     print(pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(dfs.shape[1])]).head())
    print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
    dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("HPC correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
    HPC_a = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,0]-feature[data_limit:,0]))[0,1]
    HPC_b = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,1]-feature[data_limit:,1]))[0,1]
    HPC_c = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:data_limit,2]-feature[data_limit:,2]))[0,1]
    
    plt.figure()
    plt.plot(np.abs(feature[:data_limit,0]-feature[data_limit:,0]))
    plt.plot(np.abs(feature[:data_limit,1]-feature[data_limit:,1]))
    plt.plot(np.abs(feature[:data_limit,2]-feature[data_limit:,2]))
    plt.yticks(fontsize=28)
    plt.xticks(fontsize=28)
    plt.show()
    
    plt.figure()
    plt.plot(np.abs(feature[:100,0]),color=colors[-3])
    plt.plot(np.abs(feature[100:,0]),color=colors[-4])
    plt.yticks(fontsize=28)
    plt.xticks(fontsize=28)
    plt.show()

#     print(pd.DataFrame(feature, columns=["PC{}".format(x + 1) for x in range(dfs.shape[1])]).head())
#     print(pd.DataFrame(pca.explained_variance_ratio_, index=["PC{}".format(x + 1) for x in range(dfs.shape[1])]))

    pca = PCA()
    dfs = np.array(HPCstate)[:,0]
#     dfs = np.array(Restate)
    pca.fit(dfs)
    HPCfeature = pca.transform(dfs)
    
    pca = PCA()
    dfs = np.array(PFCstate)[:,0]
#     dfs = np.array(Restate)
    pca.fit(dfs)
    PFCfeature = pca.transform(dfs)
    
    plt.figure()
    plt.plot(HPCfeature[:,0],HPCfeature[:,2])
    plt.yticks(fontsize=26)
    plt.xticks(fontsize=26)
    plt.show()

    plt.figure()
    plt.plot(PFCfeature[:,0],PFCfeature[:,2])
    plt.yticks(fontsize=26)
    plt.xticks(fontsize=26)
    plt.show()

    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
    dfs = np.concatenate((np.array(Restate)[:],np.array(Restate_noise)[:]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("Re correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
#     Re_a = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0]))[0,1]
#     Re_b = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1]))[0,1]
#     Re_c = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2]))[0,1]
    
#     plt.figure()
# #     plt.plot(np.abs(feature[:100,0]-feature[100:,0]))
# #     plt.plot(np.abs(feature[:100,1]-feature[100:,1]))
# #     plt.plot(np.abs(feature[:100,2]-feature[100:,2]))
# #     plt.plot(np.abs(moving_average(feature[:100,0])-moving_average(feature[100:,0])))
# #     plt.plot(np.abs(moving_average(feature[:100,1])-moving_average(feature[100:,1])))
# #     plt.plot(np.abs(moving_average(feature[:100,2])-moving_average(feature[100:,2])))
#     plt.plot(np.abs(feature[:100,0]),color=colors[-3])
#     plt.plot(np.abs(feature[100:,0]),color=colors[-4])
#     plt.show()
    
    return np.mean([PFC_a,PFC_b,PFC_c]),np.mean([HPC_a,HPC_b,HPC_c]),np.max([PFC_a,PFC_b,PFC_c]),np.max([HPC_a,HPC_b,HPC_c])
#     return np.max([PFC_a,PFC_b,PFC_c]),np.max([HPC_a,HPC_b,HPC_c])


if __name__ == '__main__':
    fig = plt.figure()
    correlation_fig = fig.subplots()
#     correlation_fig.set_xlim(-1,1)
#     correlation_fig.set_ylim(0,1)

#     model_list = glob.glob('model/R20_131/*OUT1**s8_100_2_*epoch*.pth')
#     model_list = sorted(model_list)
#     model_list = sorted(model_list,key=len,reverse=False)
#     k=0
#     for model in model_list:
#         print(model,PFC,HPC)
#         PFC,HPC = main(model)
# #         correlation_fig.plot(PFC,HPC,"o")
# #         correlation_fig.text(PFC,HPC,model.split("epoch")[-1].split(".")[0])
#         correlation_fig.plot(k*5,PFC/HPC,"o")
# #         correlation_fig.text(k*5,PFC/HPC,model.split("epoch")[-1].split(".")[0])
#         k+=1

    for i in range(1):
        model_list = glob.glob('model/R20_traintest/*OUT1**s'+str(i+9)+'_100_9_*epoch150.pth')
        model_list = sorted(model_list)
        model_list = sorted(model_list,key=len,reverse=False)
        ratio_list = []
        ratio_list_max = []
        k=0
        for model in model_list:
            print(model)
#             PFC,HPC = main(model)
            PFC,HPC,PFC_max,HPC_max = main(model)
            ratio_list.append(PFC/HPC)
#             ratio_list_max.append(PFC_max/HPC_max)
            ratio_list_max.append(np.abs(PFC_max-HPC_max))
#             correlation_fig.plot(PFC,HPC,"o")
#             correlation_fig.text(PFC,HPC,model.split("epoch")[-1].split(".")[0])
#             correlation_fig.plot(k*5,PFC/HPC,"o")
    #         correlation_fig.text(k*5,PFC/HPC,model.split("epoch")[-1].split(".")[0])
            k+=1
#         correlation_fig.plot(np.arange(0,200,5),np.array(ratio_list)-np.mean(ratio_list),"o")
#         ratio_list = np.array(ratio_list).clip(-2,2)
#         ratio_list = moving_average(ratio_list)[2:-2]
#         correlation_fig.plot(np.arange(0,len(ratio_list)*5,5),np.array(ratio_list))
        
        ratio_list_max = np.array(ratio_list_max).clip(-2,2)
        ratio_list_max = moving_average(ratio_list_max)[2:-2]
        correlation_fig.plot(np.arange(0,len(ratio_list_max)*5,5),np.array(ratio_list_max))
        
        

In [ ]:
##################################  Compare!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!   #############################################

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import glob

%matplotlib notebook


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)

class MyLSTM_comp(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM_comp, self).__init__()

        self.hidden_size = hidden_size
        self.LSTM1 = nn.LSTMCell(input_size, hidden_size)
        self.LSTM2 = nn.LSTMCell(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    def forward(self, input, hiddens):
        input = input.float()
        hidden1 = self.LSTM1(input, hiddens[0])
        hidden2 = self.LSTM2(hidden1[0], hiddens[1])
        output = self.linear(hidden2[0])
        return output, [hidden1,hidden2]

    def initHidden(self):
        hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [hidden,hidden]
    
    def initHidden_rand(self):
        hidden = [torch.rand(self.batch_size, self.hidden_size)*0.01, torch.rand(self.batch_size, self.hidden_size)*0.01]
        return [hidden,hidden]  

class MyLSTM_PFCHPC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=1):
        super(MyLSTM_PFCHPC, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        input = input.float()
        Re = hiddens[0][0]
        HPC_input = torch.cat([input,Re],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = hiddens[1][0]
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden]
    
    def initHidden_rand(self):
        const = 0.01
        var = 0.01
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size)*const, torch.rand(self.batch_size, self.hidden_size)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size)*var, torch.rand(self.batch_size, self.hidden_size)*var]
        return [PFC_hidden,HPC_hidden]
    
    
def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list


    

def main(model):
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 30
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    # model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_cue18_s5_100_1_2.pth'
    # model_path = 'model/ReModel_L2_interRNNrand_AddRe_OUT5_cue13_s9_100_1.pth'
    # model_path = 'model/R20_cue_131/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_2_epoch150.pth'
#     model_path = 'model/R20_cue_131/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_3_epoch195.pth'
#     model_path = 'model/R20_cue_131/ReModel_L2_interRNNrand_OUT1_cue7_131_s7_100_1_epoch195.pth'
#     model_path = 'model/R20_cue_131_1to3/ReModel_L2_interRNNrand_OUT1_cue7_131_1to3_s4_100_1_epoch175.pth'
    model_path = model
    # model_path = 'model/v4_2Model_MTRNN2_cue_9.pth'
    filename = "primal_long131test"
    sparse = 1
    
#     delay_length = np.random.randint(1, 3)
    delay_length = 3
    train_x,train_y = mkOwnDataSet_auto(training_size,delay_length)

#     rnn = MyLSTM_comp(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_PFCHPC(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyMTRNN2(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_y, batch_size)
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    # hidden = rnn.initHidden()
    data_limit = 100
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
#     for k in range(data.shape[0]*8):
#             output,hidden = rnn(output,hidden) 
#             traj.append(output.tolist())
#             PFCstate.append(hidden[0][0].tolist())
#             HPCstate.append(hidden[1][0].tolist())
#             Restate.append(hidden[2][0].tolist())
#             if np.any(cue_point==k+data_limit):
#                 output = torch.cat([output,torch.ones(10,1)],axis=1)
#             else:
#                 output = torch.cat([output,torch.zeros(10,1)],axis=1)
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
#     fig = plt.figure()
#     print(pltdata.shape)
#     plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
#     plt.plot(traj[:,0,0],traj[:,0,1])
#     plt.show()
#     MakeAnimation2(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    # MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    # MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")
    # MakeAnimation_img(np.array(Restate),"Re")
    #MakeAnimation_testdata(pltdata[:,0,0],pltdata[:,0,1])
    
    
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_rand()
    data = mkOwnRandomBatch(train_x, batch_size)
#     data_limit = 125
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
#     for k in range(data.shape[0]*8):
# #             hidden = rnn.noiseHidden_rand(hidden)
#             output,hidden = rnn(output,hidden) 
# #             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
# #             hidden = rnn.noiseHidden_rand(hidden)
#             traj_noise.append(output.tolist())
#             PFCstate_noise.append(hidden[0][0].tolist())
#             HPCstate_noise.append(hidden[1][0].tolist())
#             Restate_noise.append(hidden[2][0].tolist())
#             if np.any(cue_point_noise==k+data_limit):
#                 output = torch.cat([output,torch.ones(10,1)],axis=1)
#             else:
#                 output = torch.cat([output,torch.zeros(10,1)],axis=1)
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
#     MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_b = np.array(p.data)
            if n == "PFC.weight_hh":
                PFC_inw_b = np.array(p.data)
            if n == "HPC.weight_ih":
                HPC_w = np.array(p.data)
            if n == "Re.weight_ih":
                Re_w_b = np.array(p.data)
            if n == "Re.weight_hh":
                Re_inw_b = np.array(p.data)
    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
#     MakeAnimation(traj[:,0,0],traj[:,0,1], traj_noise[:,0,0], traj_noise[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")

#     fig = plt.figure()
#     plt.plot(traj[:,0,0],traj[:,0,1])
# #     plt.plot(traj_noise[:,0,0],traj_noise[:,0,1])
#     plt.show()

#     plt.figure()
#     plt.plot(traj[:,0,0])
#     plt.plot(traj_noise[:,0,0])


#     PFC_corrlist = np.array([])
#     HPC_corrlist = np.array([])
#     cross_corrlist = np.array([])
#     for i in range(20):
#         plt.figure()
#         plt.ylim(-1,1)
#         plt.plot(traj[:,0,0])
#         plt.plot(traj_noise[:,0,0])
# #         plt.plot(np.abs(traj[:,0,0]-traj_noise[:,0,0]))
#     #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
# #                 plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
# #                 plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),alpha=0.5)
# #                 plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
# #                 plt.plot(np.array(HPCstate_noise)[:,k,i],alpha=0.5)
#                 plt.plot(np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]),alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
#     #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:,i],alpha=0.5)
# #             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate)[:,i]),alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #                 plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #                 plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
# #                 plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
# #                 plt.plot(np.array(Gate_states_noise)[:,1,20+i],alpha=0.5)
# #                 plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
# #                 plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
# #                 print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]))[0,1])
# #                 print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]))[0,1])
# #                 print(np.corrcoef(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i])))
#                 cross_corrlist = np.append(cross_corrlist,np.corrcoef(np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i])))
#                 PFC_corrlist = np.append(PFC_corrlist,np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(PFCstate)[:,k,i]-np.array(PFCstate_noise)[:,k,i]))[0,1])
#                 HPC_corrlist = np.append(HPC_corrlist,np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(np.array(HPCstate)[:,k,i]-np.array(HPCstate_noise)[:,k,i]))[0,1])
# #         plt.title("neuron#"+str(i+1))
#     print(np.mean(PFC_corrlist),np.mean(HPC_corrlist),np.mean(cross_corrlist))
    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
    dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
#     dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    ratios = pca.explained_variance_ratio_
    print("PFC correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
    PFC_a = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0]))[0,1]*ratios[0]
    PFC_b = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1]))[0,1]*ratios[1]
    PFC_c = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2]))[0,1]*ratios[2]

#     plt.figure()
#     plt.plot(np.abs(feature[:100,0]-feature[100:,0]))
#     plt.plot(np.abs(feature[:100,1]-feature[100:,1]))
#     plt.plot(np.abs(feature[:100,2]-feature[100:,2]))
#     plt.show()
    
#     plt.figure()
#     plt.plot(np.abs(feature[:100,0]),color=colors[-3])
#     plt.plot(np.abs(feature[100:,0]),color=colors[-4])
#     plt.show()
    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
    dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    ratios = pca.explained_variance_ratio_
    print("HPC correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
    HPC_a = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0]))[0,1]*ratios[0]
    HPC_b = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1]))[0,1]*ratios[1]
    HPC_c = np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2]))[0,1]*ratios[2]
    
    plt.figure()
    plt.plot(np.abs(feature[:100,0]-feature[100:,0]))
    plt.plot(np.abs(feature[:100,1]-feature[100:,1]))
    plt.plot(np.abs(feature[:100,2]-feature[100:,2]))
    plt.show()

#     return np.mean([PFC_a,PFC_b,PFC_c]),np.mean([HPC_a,HPC_b,HPC_c])
    return np.max([PFC_a,PFC_b,PFC_c]),np.max([HPC_a,HPC_b,HPC_c])

if __name__ == '__main__':
    fig = plt.figure()
    correlation_fig = fig.subplots()
#     correlation_fig.set_xlim(-1,1)
#     correlation_fig.set_ylim(0,1)

#     model_list = glob.glob('model/compare30_131/*s10_100_1_*epoch*.pth')
#     model_list = sorted(model_list)
#     model_list = sorted(model_list,key=len,reverse=False)
#     k=0
#     for model in model_list:
#         print(model)
#         PFC,HPC = main(model)
# #         correlation_fig.plot(PFC,HPC,"o")
# #         correlation_fig.text(PFC,HPC,model.split("epoch")[-1].split(".")[0])
#         correlation_fig.plot(k*5,PFC/HPC,"o")
# #         correlation_fig.text(k,PFC/HPC,model.split("epoch")[-1].split(".")[0])
#         k+=1
    
    for i in range(10):
#         model_list = glob.glob('model/compare30_131/*s'+str(i+1)+'_100_1_*epoch??.pth')
        model_list = glob.glob('model/PFCHPC_30_131/*s'+str(i+1)+'_100_2_*epoch*.pth')
        model_list = sorted(model_list)
        model_list = sorted(model_list,key=len,reverse=False)
        ratio_list = []
        k=0
        for model in model_list:
            PFC,HPC = main(model)
            ratio_list.append(PFC/HPC)
#             ratio_list.append(np.abs(PFC-HPC))
#             correlation_fig.plot(PFC,HPC,"o")
#             correlation_fig.text(PFC,HPC,model.split("epoch")[-1].split(".")[0])
#             correlation_fig.plot(k*5,PFC/HPC,"o")
    #         correlation_fig.text(k*5,PFC/HPC,model.split("epoch")[-1].split(".")[0])
            k+=1
#         correlation_fig.plot(np.arange(0,200,5),np.array(ratio_list)-np.mean(ratio_list),"o")
        ratio_list = np.array(ratio_list).clip(-2,2)
        ratio_list = moving_average(ratio_list)[2:-2]
        correlation_fig.plot(np.arange(0,len(ratio_list)*5,5),np.array(ratio_list))
        
        

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jul 11 00:31:22 2020

@author: munenoritakaku
"""

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jul  3 10:30:01 2020

@author: munenoritakaku
"""
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D


def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        else:
            traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims,interval=10)    
    ani.save('Comp_anim_0_l.gif', writer='pillow')
    plt.show()
    
    return 0


def mkDataSet(data_size, data_length=50, freq=60., noise=0.02):
    """
    params
      data_size : データセットサイズ
      data_length : 各データの時系列長
      freq : 周波数
      noise : ノイズの振幅
    returns
      train_x : トレーニングデータ（t=1,2,...,size-1の値)
      train_t : トレーニングデータのラベル（t=sizeの値）
    """
    train_x = []
    train_t = []

    for offset in range(data_size):
        train_x.append([[math.cos(2 * math.pi * (offset + i) / freq) + np.random.normal(loc=0.0, scale=noise), math.sin(4 * math.pi * (offset + i) / freq) + np.random.normal(loc=0.0, scale=noise)] for i in range(data_length)])
        train_t.append([math.cos(2 * math.pi * (offset + data_length) / freq),math.sin(4 * math.pi * (offset + data_length) / freq)])
    print(len(train_x[0]))
    return train_x, train_t

def mkOwnDataSet(data_size, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt("primal_long131test_r.csv",delimiter=',')
    y = np.loadtxt("primal_long131test_l.csv",delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y

def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 20
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x = make_Ttraj(1)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y = make_Ttraj(2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_x = np.array([[0,0]])
    data_y = np.array([[0,0]])
    order  = np.array([1,0,0,1])
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2,0,0,2])
    order = np.append(order, [2])
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
#     order = [1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,2]
    for idx in order:
        if idx == 1:
            target_x = x
            target_y = y
        elif idx == 0:
            target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], 20)])
            target_x = target
            target_y = target
        else:
            target_x = y
            target_y = x
        if idx != 0:
            target_x += np.random.normal(loc=0.0, scale=noise, size=target_x.shape)
            target_y += np.random.normal(loc=0.0, scale=noise, size=target_y.shape)

        data_x = np.append(data_x,target_x,axis=0)
        data_y = np.append(data_y,target_y,axis=0)
        
    print(data_x.shape,data_y.shape)
    return data_x[1:],data_y[1:]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.01):
   
    x,y = make_traj(delay_length)
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x.shape[0])])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y.shape[0])])
    return train_x, train_y


def mkRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    batch_t = []
    idx = np.random.randint(0, len(train_x) - batch_size)
    for _ in range(batch_size):
        batch_x.append(train_x[idx])
        batch_t.append(train_t[idx])
        idx = np.random.randint(0, len(train_x) - batch_size)
        #idx += 1
    return torch.tensor(batch_x).transpose(0,1), torch.tensor(batch_t)

def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    idx = np.random.randint(0, len(train_x) - batch_size)
    for _ in range(batch_size):
        batch_x.append(train_x[idx])
        idx = np.random.randint(0, len(train_x) - batch_size)
        #idx += 1
    return torch.tensor(batch_x).transpose(0,1)

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.LSTM1 = nn.LSTMCell(input_size, hidden_size)
        self.LSTM2 = nn.LSTMCell(hidden_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size

    def forward(self, input, hiddens):
        input = input.float()
        hidden1 = self.LSTM1(input, hiddens[0])
        hidden2 = self.LSTM2(hidden1[0], hiddens[1])
        output = self.linear(hidden2[0])
        return output, [hidden1,hidden2]

    def initHidden(self):
        hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [hidden,hidden]

    def initHidden_rand(self):
        hidden = [torch.rand(self.batch_size, self.hidden_size)*0.2, torch.rand(self.batch_size, self.hidden_size)*0.2]
        return [hidden,hidden]    


def make_bins(data):
    bins = np.array([])
    for i in range(len(data)-1):
        bins = np.append(bins,(data[i]+data[i+1])/2)
    return bins

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(states[start:])
    return states_list


def main():
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 30
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 3
    # model_path = 'model/compstack_model_long131test_3.pth'
    # model_path = 'model/compstack_model_171_s5_100_1.pth'
    # model_path = 'model/compstack_model_151_s10_100_1.pth'
    # model_path = 'model/compare/Compare_151_s2_100_2_epoch170.pth'
    # model_path = 'model/compare_cont/compstack_model_151_s9_100_10.pth'
    # model_path = 'model/compare_cont/Compare_151_s5_100_8_epoch125.pth'
    # model_path = 'model/compare_Y/compstack_model_151Y_s2_100_1add_5.pth'
    # model_path = 'model/compare_Y/Compare_151Y_s1_100_2add_1_epoch40.pth'
    # model_path = 'model/compare_H/Compare_121H_s10_100_1add_epoch230.pth'
    # model_path = 'model/compare30_H/Compare30_121H_s9_100_3_epoch125.pth'
    # model_path = 'model/compare_171/Compare_171_s3_100_3_epoch180.pth'
    model_path = 'model/compare30_131/compare30_131_s7_100_1_epoch105.pth'
#     model_path = 'model/compare30_151/compare30_151_s6_100_2_epoch195.pth'
#     model_path = 'model/compare_1212121/compare30_121_s1_100_3_epoch145.pth'
    # model_path = 'model/compare_transfer/Compare_121H_transfers1010_s2_100_1_epoch140.pth'
    # model_path = 'model/compare_transfer_1212121/compare_transfer_121_s2_100_1_epoch25.pth'

    # train_x,train_y = mkOwnDataSet(training_size,data_length)
    train_x,train_y = mkOwnDataSet_auto(training_size, delay_length)

    rnn = MyLSTM(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    data = mkOwnRandomBatch(train_x, batch_size)
    
    traj = []
    PFCstate = []
    HPCstate = []
    hidden = rnn.initHidden_rand()
    data_limit = 1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            traj.append(output.tolist())
    for k in range(data.shape[0]*4):
            #print(data[k].shape)
            output,hidden = rnn(output,hidden) 
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            traj.append(output.tolist())
#             print(output)
    traj = torch.tensor(traj)
    pltdata = torch.squeeze(data).numpy()
    traj = torch.squeeze(traj).numpy()
    fig = plt.figure()
    print(pltdata.shape)
    plt.plot(pltdata[:,0,0],pltdata[:,0,1],"--")
    plt.plot(traj[:,0,0],traj[:,0,1])
    plt.show()
    
    # MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    
    
    # for n, p in rnn.named_parameters():
    #     if n == "LSTM1.weight_ih":
    #         PFC_w = np.array(p.data)
    #     if n == "LSTM2.weight_ih":
    #         HPC_w = np.array(p.data)
    #     if n == "LSTM1.weight_hh":
    #         PFC_inw = np.array(p.data)
    #     if n == "LSTM2.weight_hh":
    #         HPC_inw = np.array(p.data)
                
    # fig2 = plt.figure()
    # ax1 = fig2.add_subplot(121)
    # ax2 = fig2.add_subplot(122)
    
    # ax1.imshow(PFC_w,cmap="coolwarm")
    # ax2.imshow(HPC_w,cmap="coolwarm")
    # ax1.set_title("max = {:.2f},min = {:.2f}".format(np.max(PFC_w),np.min(PFC_w)))
    # ax2.set_title("max = {:.2f},min = {:.2f}".format(np.max(HPC_w),np.min(HPC_w)))
    
    
    fig3 = plt.figure()
    ax3 = fig3.add_subplot(121)
    ax4 = fig3.add_subplot(122)
    ax3.imshow(np.corrcoef(np.array(PFCstate)[:,0]),cmap=plt.get_cmap("seismic"))
    ax4.imshow(np.corrcoef(np.array(HPCstate)[:,0]),cmap=plt.get_cmap("seismic"))
    ax3.set_title("PFC")   
    ax4.set_title("HPC")  
    
    pca = PCA()
    dfs = np.array(HPCstate)[:,0,:]
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
    pred = KMeans(n_clusters=2).fit_predict(feature)
    
    plt.figure(figsize=(6, 6))
    plt.scatter(feature[:, 0], feature[:, 1], alpha=0.8)
    plt.grid()
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    for i in range(200):
        plt.annotate(i,(feature[i, 0], feature[i, 1]))
    plt.show()
    
    fig3d = plt.figure()
    ax3d = Axes3D(fig3d)
    ax3d.plot(feature[:, 0], feature[:, 1], feature[:, 2],  alpha=0.3)
    
    # fig4 = plt.figure(figsize=(10,5))
    # ax5 = fig4.add_subplot(211)
    # ax6 = fig4.add_subplot(212)
    # #ax5.plot(np.array(PFCstate)[:,0,pred==1])
    # ax5.imshow(np.array(PFCstate)[:,0,:].T)
    # # ax5.plot(np.average(np.array(PFCstate)[:,0,pred==0],axis=1))
    # # ax5.plot(np.average(np.array(PFCstate)[:,0,pred==1],axis=1))
    # # ax5.plot(np.average(np.array(HPCstate)[:,0],axis=1))
    # ax6.imshow(np.array(HPCstate)[:,0,:].T)
    # ax5.set_title("PFC")   
    # ax6.set_title("HPC")  
    # print(pred==0)
    
    # fig5 = plt.figure(figsize=(5,5))
    # plt.plot(np.average(np.array(PFCstate)[:,0,:],axis=1),np.average(np.array(HPCstate)[:,0,:],axis=1))
    # plt.show()
    # #MakeAnimation_attracter(np.average(np.array(PFCstate)[:,0,pred==0],axis=1),np.average(np.array(PFCstate)[:,0,pred==1],axis=1))
        
    # #np.save("right_good_traj_PFC.npy",dfs)
    # #np.save("left_traj.npy",dfs)
    
    # fig_weight = plt.figure(figsize=(10,5))
    # fig1 = fig_weight.add_subplot(131)
    # fig2 = fig_weight.add_subplot(132)
    # fig3 = fig_weight.add_subplot(133)
    # PFC_w_hist = np.histogram(np.abs(PFC_w[PFC_w.nonzero()]),bins=80,density=True)
    # PFC_inw_hist = np.histogram(np.abs(PFC_inw[PFC_inw.nonzero()]),bins=40,density=True)
    # fig1.plot(np.log(make_bins(PFC_w_hist[1])),np.log(PFC_w_hist[0]+0.01))
    # fig1.plot(np.log(make_bins(PFC_inw_hist[1])),np.log(PFC_inw_hist[0]+0.01))
    
    # HPC_w_hist = np.histogram(np.abs(HPC_w[HPC_w.nonzero()]),bins=80,density=True)
    # HPC_inw_hist = np.histogram(np.abs(HPC_inw[HPC_inw.nonzero()]),bins=40,density=True)
    # fig2.plot(np.log(make_bins(HPC_w_hist[1])),np.log(HPC_w_hist[0]+0.01))
    # fig2.plot(np.log(make_bins(HPC_inw_hist[1])),np.log(HPC_inw_hist[0]+0.01))
    
#     for i in range(20):
#         plt.figure()
# #         plt.ylim(-1,1)
#         plt.plot(traj[:,0,0])
# #         plt.plot(np.array(PFCstate_noise)[:400,0,17],alpha=0.5)
#         for k in range(1):
#             plt.plot(np.array(HPCstate)[:,k,i],alpha=0.5)
#             plt.plot(np.array(PFCstate)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:120,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate)[:120,k,i]-np.array(PFCstate)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(PFCstate_noise)[:120,k,i]-np.array(PFCstate_noise)[120:240,k,i],alpha=0.5)
# #             plt.plot(np.array(Restate_noise)[:,i],alpha=0.5)
# #             plt.plot(np.array(Restate)[:,i]-np.array(Restate_noise)[:,i],alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate)[:,i]),alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #             plt.plot(moving_average(np.array(Restate)[:,i])-moving_average(np.array(Restate_noise)[:,i]),alpha=0.5)
# #             plt.plot(np.array(Gate_states)[:,0,0+i],alpha=0.5)
# #             plt.plot(np.array(Gate_states)[:,1,0+i],alpha=0.5)
# #             plt.plot(np.array(Gate_states_noise)[:,1,0+i],alpha=0.5)
# #             plt.plot(np.abs(np.array(Gate_states)[:,1,0+i]-np.array(Gate_states_noise)[:,1,0+i]),alpha=0.5)
# #             plt.plot(np.log(np.array(Gate_states)[:,1,0+i]/np.array(Gate_states)[:,1,20+i]),alpha=0.5)
# #             plt.plot(np.log(np.array(Gate_states_noise)[:,1,0+i]/np.array(Gate_states_noise)[:,1,20+i]),alpha=0.5)
# #             plt.plot(np.log(np.array(Gate_states)[:,0,0+i]/np.array(Gate_states)[:,0,20+i]),alpha=0.5)
# #             plt.plot(np.log(np.array(Gate_states_noise)[:,0,0+i]/np.array(Gate_states_noise)[:,0,20+i]),alpha=0.5)
#         plt.title("neuron#"+str(i+1))


    delays = pick_traj(traj[:,0], np.array(traj)[:])[3:-1]
    bifur = np.array([])
    for data in delays:
        check_bifur = np.argmax(np.abs(np.array(data)[:,0,0] - 0.5))
        bifur = np.append(bifur,data[check_bifur,:,0])
    if bifur.shape[0] == 0:
        pass
    plt.figure()
    plt.plot(traj[:,0,0])
    plt.plot(traj[:,0,1])
    print(np.var(bifur),np.abs(np.median(bifur)-np.mean(bifur)))
    print(bifur)
    
    pca = PCA()
    # dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0],np.array(Restate)),axis=1)
    # dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(HPCstate)[:,0]),axis=1)
    dfs = np.array(PFCstate)[:,0]
#     dfs = np.array(Restate)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    
#     delays = pick_traj(traj_noise[:,0], np.array(PFCstate_noise)[:,0])
    delays = pick_traj(traj[:,0], np.array(PFCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(HPCstate_noise)[:,0])
#     delays = pick_traj(traj[:,0], np.array(HPCstate)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Restate_noise)[:])
#     delays = pick_traj(traj[:,0], np.array(Restate)[:])
#     delays = pick_traj(traj_noise[:,0], np.array(traj_noise)[:])[3:-1]
#     delays = pick_traj(traj[:,0], np.array(Gate_states)[:,1])
#     delays = pick_delay(traj[:,0], np.array(Gate_states)[:,0])
#     delays = pick_traj(traj_noise[:,0], np.array(Gate_states_noise)[:,1])
#     delays = pick_delay(traj_noise[:,0], np.array(Gate_states_noise)[:,0])
#     delays = pick_delay(traj[:,0], feature[:])
#     delays = pick_traj(traj[:,0], feature[:])
    bifur = np.array([])
    for i in range(20):
        plt.figure()
        for data in delays[1:-1]:
            plt.plot(np.array(data)[:,0+i],alpha=0.5)
            print(len(data))
        plt.title("neuron#"+str(i+1))
#     print(np.var(bifur),np.median(bifur)-np.mean(bifur))

if __name__ == '__main__':
    main()

In [ ]:
# path = "model/R20_cue_1until3_smallbatch4/"
path = "model/compare30_cue_1until3_smallbatch4/"

with open(path+"good_list.txt", mode="r") as f:
    good_list = f.read().splitlines()
with open(path+"good_list_long.txt", mode="r") as f:
    good_list_long = f.read().splitlines()
for model in good_list:
    if model in good_list_long:
        print(model)
        
model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s1_100_2_epoch65.pth
model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s5_100_1_epoch95.pth
model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s7_100_3_epoch80.pth
model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_3_epoch50.pth
model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s10_100_3_epoch40.pth

In [ ]:
# path = "model/R20_cue_1until3_smallbatch4/"
path = "model/compare30_cue_1until3_smallbatch4/"

with open(path+"good_list.txt", mode="r") as f:
    good_list = f.read().splitlines()
with open(path+"good_list_long.txt", mode="r") as f:
    good_list_long = f.read().splitlines()
for model in good_list:
    if model in good_list_long:
        print(model)

        
model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s1_100_3_epoch35.pth
model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s6_100_1_epoch85.pth

In [ ]:
# path = "model/R20_cue_1until3_smallbatch4/"
path = "model/compare30_cue_1until3_smallbatch5/"

with open(path+"good_list_short.txt", mode="r") as f:
    good_list = f.read().splitlines()
with open(path+"good_list_long.txt", mode="r") as f:
    good_list_long = f.read().splitlines()
for model in good_list:
    if model in good_list_long:
        print(model)
        
# compare5
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s1_100_2_epoch145.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s2_100_2_epoch80.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s3_100_1_epoch175.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s3_100_3_epoch125.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s4_100_1_epoch95.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s4_100_2_epoch135.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s6_100_3_epoch85.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s7_100_1_epoch135.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s7_100_2_epoch90.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s7_100_3_epoch170.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s8_100_2_epoch55.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s8_100_3_epoch185.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s9_100_1_epoch140.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s9_100_2_epoch140.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s9_100_3_epoch185.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s10_100_2_epoch65.pth
# model/compare30_cue_1until3_smallbatch5/Compare30_cue_131_s10_100_3_epoch95.pth

# compare4
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s1_100_3_epoch35.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s2_100_3_epoch120.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s3_100_3_epoch100.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s4_100_1_epoch120.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s6_100_1_epoch110.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s7_100_2_epoch120.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s8_100_1_epoch150.pth
# model/compare30_cue_1until3_smallbatch4/Compare30_cue_131_s10_100_3_epoch175.pth

In [ ]:
path = "model/R20_cue_1until3_smallbatch5/"
# path = "model/compare30_cue_1until3_smallbatch5/"

with open(path+"good_list_short.txt", mode="r") as f:
    good_list = f.read().splitlines()
with open(path+"good_list_long.txt", mode="r") as f:
    good_list_long = f.read().splitlines()
for model in good_list:
    if model in good_list_long:
        print(model)
        
# Re5
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s1_100_1_epoch110.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s1_100_2_epoch95.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s2_100_1_epoch50.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s2_100_2_epoch155.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s2_100_3_epoch135.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s3_100_1_epoch40.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s3_100_3_epoch130.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s4_100_2_epoch140.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s5_100_2_epoch55.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s6_100_2_epoch90.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s6_100_3_epoch120.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s7_100_3_epoch85.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s8_100_1_epoch65.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s8_100_2_epoch150.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s8_100_3_epoch85.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_1_epoch150.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_2_epoch85.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_3_epoch130.pth
# model/R20_cue_1until3_smallbatch5/ReModel_L2_interRNNrand_OUT1_cue7_131_s10_100_2_epoch10.pth

# # Re4
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s1_100_1_epoch100.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s1_100_2_epoch65.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s2_100_3_epoch120.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s3_100_1_epoch145.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s4_100_2_epoch120.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s5_100_1_epoch95.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s6_100_2_epoch155.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s7_100_2_epoch155.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s7_100_3_epoch95.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s8_100_2_epoch140.pth
# model/R20_cue_1until3_smallbatch4/ReModel_L2_interRNNrand_OUT1_cue7_131_s9_100_3_epoch50.pth

In [ ]:
# path = "model/R20_cue_1until3_smallbatch5_decay/"
path = "model/compare30_cue_1until3_smallbatch5_decay/"
# path = "model/PFCHPC30_cue_1until3_smallbatch5/"


with open(path+"good_list_short.txt", mode="r") as f:
    good_list = f.read().splitlines()
with open(path+"good_list_long.txt", mode="r") as f:
    good_list_long = f.read().splitlines()
for model in good_list:
#     if int(model.split("_epoch")[-1].split(".pth")[0]) < 190:
#         continue
    if model in good_list_long:
        print(model)
        
        

# 5
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s1_100_3_epoch140.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s2_100_1_epoch100.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s2_100_3_epoch150.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s3_100_2_epoch85.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s4_100_1_epoch110.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s4_100_2_epoch180.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s5_100_3_epoch175.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s6_100_1_epoch115.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s6_100_2_epoch75.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s6_100_3_epoch50.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s7_100_1_epoch95.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s7_100_2_epoch75.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s7_100_3_epoch155.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s9_100_1_epoch175.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s9_100_2_epoch65.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s9_100_3_epoch135.pth
# model/PFCHPC30_cue_1until3_smallbatch5/v4_3_N30_cue_131_s10_100_1_epoch115.pth
        
# 4
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s1_100_1_epoch70.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s1_100_2_epoch165.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s1_100_3_epoch170.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s2_100_1_epoch115.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s2_100_2_epoch55.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s3_100_2_epoch50.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s5_100_2_epoch95.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s6_100_3_epoch50.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s7_100_1_epoch75.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s8_100_2_epoch85.pth
# model/PFCHPC30_cue_1until3_smallbatch4/v4_3_N30_cue_131_s9_100_1_epoch165.pth


In [ ]:
##########    Checker of Activity difference and grad   #########################
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import sklearn 
import nolds
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import expit
import pandas as pd
import glob
import scipy

%matplotlib notebook

def Culc_gate(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = torch.cat([hiddens[1][0][0],hiddens[2][0]])
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]

def Culc_gate_uniPFC(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = hiddens[1][0][0]
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]

def Culc_gate_uniHPC(input, params, hiddens):
    Re = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
    #Re = torch.zeros(10, 20)
    HPC_input = torch.cat([input,hiddens[2]],dim=1)
    PFC_input = torch.cat([hiddens[1][0][0],hiddens[2][0]])
    PFC_gates = params[0]@PFC_input+params[1]@hiddens[0][1][0]+params[2]+params[3]
    HPC_gates = params[4]@HPC_input[0]+params[5]@hiddens[1][1][0]+params[6]+params[7]
    PFC_gates = torch.sigmoid(PFC_gates)
    HPC_gates = torch.sigmoid(HPC_gates)
    
    return [PFC_gates.tolist(), HPC_gates.tolist()]
    
    

def MakeAnimation(data_x,data_y,traj_x,traj_y, data_limit):
    fig, ax = plt.subplots()
    ims = []
    for t in range(traj_x.shape[0]):
        if t < data_limit:
            traj = ax.plot(data_x[:t], data_y[:t], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#         else:
#             traj = ax.plot(data_x[:data_limit], data_y[:data_limit], "b", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or", data_x[data_limit-1:t], data_y[data_limit-1:t], "--b")
        else:
            traj = ax.plot(traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
#             traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob", traj_x[:t], traj_y[:t], "r", traj_x[t-1], traj_y[t-1], "or")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_v4_2_1_r.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_data(data_x,data_y, data_limit):
    fig, ax = plt.subplots()
    ax.set_xlim(0.2,0.8)
    ax.set_ylim(-0.05,0.8)
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b")
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=30)    
    ani.save('anim_Re_0.gif', writer='pillow')
    plt.show()
    
    return 0

def MakeAnimation_attracter(data_x,data_y):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data_x.shape[0]):
        traj = ax.plot(data_x[:t], data_y[:t], "b", data_x[t-1], data_y[t-1], "ob",)
        title = ax.text(0.5, 1.01, 'time={}'.format(t),
                     ha='center', va='bottom',
                     transform=ax.transAxes, fontsize='large')
        ims.append(traj+[title])
    ani = animation.ArtistAnimation(fig, ims, interval=10)    
    ani.save('anim_attracter2.gif', writer='pillow')
    plt.show()
    
    return 0


    
def MakeAnimation_img(data,module):
    fig, ax = plt.subplots()
    ims = []
    for t in range(data.shape[0]):
        img = ax.imshow(data[t].reshape(1,data.shape[1]))
        title = ax.text(0.5, 1.01, module+' time={}'.format(t),
             ha='center', va='bottom',
             transform=ax.transAxes, fontsize='large')
        ims.append([img,title])
    ani = animation.ArtistAnimation(fig, ims)    
    ani.save('anim_fire_'+module+'_.gif', writer='pillow')
    plt.show()
    
    return 0

def mkOwnDataSet(data_size, filename, data_length=100, freq=60., noise=0.005):
    
    x = np.loadtxt(str(filename+"_r.csv"),delimiter=',')
    y = np.loadtxt(str(filename+"_l.csv"),delimiter=',')
    train_x = []
    train_y = []
    
    for offset in range(data_size):
        train_x.append([[x[i][0] + np.random.normal(loc=0.0, scale=noise),x[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,x.shape[0],round(x.shape[0]/data_length))])
        train_y.append([[y[i][0] + np.random.normal(loc=0.0, scale=noise),y[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(0,y.shape[0],round(y.shape[0]/data_length))])
    return train_x, train_y


def mkOwnRandomBatch(train_x, train_t, batch_size=10):
    """
    train_x, train_tを受け取ってbatch_x, batch_tを返す
    """
    batch_x = []
    for _ in range(batch_size):
        idx = np.random.randint(0, len(train_x) - batch_size)
        batch_x.append(train_x[idx])
    return torch.tensor(batch_x).transpose(0,1)

def make_Ttraj(direction):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(5)*0.5
    if direction == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.5, 5)
    elif direction == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.5, 5)        
    backstraight_x = np.ones(5)*0.5

    straight_y = np.linspace(0, 0.5, 5)
    branch1_y = np.linspace(0.5, 0.5, 5)
    branch2_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 5)
    
    traj_x = np.append(traj_x, [straight_x,branch1_x,branch2_x,backstraight_x])
    traj_y = np.append(traj_y, [straight_y,branch1_y,branch2_y,backstraight_y])
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj

def make_Htraj(direction1, direction2):
    traj_x = np.array([])
    traj_y = np.array([])
    straight_x = np.ones(10)*0.5
    if direction1 == 1:
        branch1_x = np.linspace(0.5, 0.75, 5)
        branch2_x = np.linspace(0.75, 0.75, 5)
        branch3_x = np.linspace(0.75, 0.75, 5)
        branch4_x = np.linspace(0.75, 0.5, 5)
    elif direction1 == 2:
        branch1_x = np.linspace(0.5, 0.25, 5)
        branch2_x = np.linspace(0.25, 0.25, 5)   
        branch3_x = np.linspace(0.25, 0.25, 5)  
        branch4_x = np.linspace(0.25, 0.5, 5)       
    backstraight_x = np.ones(10)*0.5

    straight_y = np.linspace(0, 0.5, 10)
    branch1_y = np.linspace(0.5, 0.5, 5)
    if direction2 == 1:
        branch2_y = np.linspace(0.5, 0.25, 5)
        branch3_y = np.linspace(0.25, 0.5, 5)
    elif direction2 == 2:
        branch2_y = np.linspace(0.5, 0.75, 5)
        branch3_y = np.linspace(0.75, 0.5, 5)
    branch4_y = np.linspace(0.5, 0.5, 5)
    backstraight_y = np.linspace(0.5 , 0, 10)
    
    traj_x = np.append(traj_x, straight_x)
    traj_x = np.append(traj_x, [branch1_x,branch2_x,branch3_x,branch4_x])
    traj_x = np.append(traj_x, backstraight_x)
    traj_y = np.append(traj_y, straight_y)
    traj_y = np.append(traj_y, [branch1_y,branch2_y,branch3_y,branch4_y])
    traj_y = np.append(traj_y, backstraight_y)
    traj = np.concatenate([[traj_x],[traj_y]],axis=0).T
    return traj


def make_traj(delay_length):
    freq=60
    noise=0.005
    data_length = 40
    
    # x = np.loadtxt("right1.csv",delimiter=',')
    x_1 = make_Htraj(1,1)
    x_2 = make_Htraj(1,2)
    # y = np.loadtxt("left1.csv",delimiter=',')
    y_1 = make_Htraj(2,1)
    y_2 = make_Htraj(2,2)
    z = np.loadtxt("stay1.csv",delimiter=',')
    data_list = []

    ###  ver1  ###
    orders = []
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [4])
    orders.append(order) 
    order  = np.array([2])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [3])
    orders.append(order) 
    order  = np.array([3])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [1])
    orders.append(order) 
    order  = np.array([4])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2])
    orders.append(order) 
    
#     ###  ver2  ###
#     orders = []
#     order  = np.array([1])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [2])
#     orders.append(order) 
#     order  = np.array([2])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [3])
#     orders.append(order) 
#     order  = np.array([3])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [4])
#     orders.append(order) 
#     order  = np.array([4])
#     order  = np.append(order,np.zeros(delay_length))
#     order = np.append(order, [1])
#     orders.append(order) 

    ###  ver1's test  ###
    orders = []
    order  = np.array([1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [4])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [3])
    orders.append(order) 
    order  = np.array([2])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [3])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [4])
    orders.append(order) 
    order  = np.array([3])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [1])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [4])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2])
    orders.append(order) 
    order  = np.array([4])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [2])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [3])
    order  = np.append(order,np.zeros(delay_length))
    order = np.append(order, [1])
    orders.append(order) 
    
    # order = [1,0,0,0,2]
    # order = [2,0,0,1,0,0,0,2,0,0,1]
    for order in orders:
        data = np.array([[0,0]])
        for idx in order:
            if idx == 1:
                target = x_1
            elif idx == 2:
                target = x_2
            elif idx == 3:
                target = y_1
            elif idx == 4:
                target = y_2
            elif idx == 0:
#                 target = np.array([[z[i][0],z[i][1]] for i in np.random.choice(z.shape[0], data_length)])
                target = np.array([[0.5,0] for i in np.random.choice(z.shape[0], data_length)])
            if idx != 0:
                target += np.random.normal(loc=0.0, scale=noise, size=target.shape)
    
            data = np.append(data,target,axis=0)
        data_list.append(data[1:])
    return data_list[0],data_list[1],data_list[2],data_list[3]

def mkOwnDataSet_auto(data_size, delay_length, freq=60., noise=0.0001):
   
    x1,x2,y1,y2 = make_traj(delay_length)
    train_x1 = []
    train_x2 = []
    train_y1 = []
    train_y2 = []
    
    for offset in range(data_size):
        train_x1.append([[x1[i][0] + np.random.normal(loc=0.0, scale=noise),x1[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x1.shape[0])])
        train_x2.append([[x2[i][0] + np.random.normal(loc=0.0, scale=noise),x2[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(x2.shape[0])])

        train_y1.append([[y1[i][0] + np.random.normal(loc=0.0, scale=noise),y1[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y1.shape[0])])
        train_y2.append([[y2[i][0] + np.random.normal(loc=0.0, scale=noise),y2[i][1]+ np.random.normal(loc=0.0, scale=noise)] for i in np.arange(y2.shape[0])])

    return train_x1, train_x2, train_y1, train_y2

def plot_distance_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    plt.figure()
    plt.plot(result)
    
    plt.vlines(linelist,np.min(result),np.max(result))
    
    return result

def distance_bet2traj(traj1,traj2):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    for i in range(traj1.shape[0]):
        result.append(np.linalg.norm(traj1[i]-traj2[i]))
    return result

def plot_activity_bet2traj(traj1,traj2,linelist):
    traj1 = np.array(traj1)
    traj2 = np.array(traj2)
    result = []
    threshold = 0.5
    for i in range(traj1.shape[0]):
        result.append(np.abs(traj1[i]-traj2[i]))
        
    plt.figure()
    for i,data in enumerate(np.array(result).T):
        if np.any(data[20:105]>threshold):
            print(i)
            plt.plot(data)
    
    return result

def search_delay(traj):
    linelist = np.array([])
    flag = False
    for i in range(traj.shape[0]):
        if traj[i,1] > 0.45 and flag == False:
            linelist = np.append(linelist,i)
            flag = True
        if traj[i,1] < 0.45 and flag == True:
            linelist = np.append(linelist,i)
            flag = False
    return linelist

def pick_delay(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
        start = int(k)
    states_list.append(traj[start:])
    return states_list

def pick_traj(traj,states):
    pointlist = np.array([])
    flag = False
    threshold = 0.2
    for i in range(traj.shape[0]):
        if i < 5:
            continue
        if traj[i,1] < threshold and flag == False:
            pointlist = np.append(pointlist,i)
            flag = True
        if traj[i,1] > threshold and flag == True:
            pointlist = np.append(pointlist,i)
            flag = False
    states_list = []
    start = 0
    for i,k in enumerate(pointlist):
        end = int(k)
        if i % 2 != 0:
            states_list.append(states[start:end])
            start = int(k)
    states_list.append(traj[start:])
    return states_list

def match_length(states_list):
    min_length = 1000
    result = []
    for state in states_list:
        min_length = np.min((min_length, len(state)))
    for state in states_list:
        result.append(state[-min_length:])
        
    return result

def vec_var(datas):
    datas = np.array(datas)
    average = np.average(datas,axis=0)
    result = 0
    for data in datas:
        result += np.linalg.norm(data-average)
    result /= datas.shape[0]
    return result
    
def lyapunov_exp(data):
    result = np.mean(np.log(np.abs(np.diff(data))))
    return result

def culc_grad(data):
    datalen = len(data)
    grad_list = np.array([])
    pre_value = data[0]
    for i in range(datalen):
        grad = np.abs(data[i] - pre_value)
        grad_list = np.append(grad_list,grad)
        pre_value = data[i]
        
    return grad_list

class MyLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        super(MyLSTM, self).__init__()

        self.hidden_size = hidden_size
        self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+hidden_size, hidden_size)
        self.Re = nn.LSTMCell(hidden_size*2, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
        self.batch_size = batch_size
        nn.init.sparse_(self.PFC.weight_ih.data,1)
        nn.init.sparse_(self.HPC.weight_ih.data,1)
        nn.init.sparse_(self.PFC.weight_hh.data,1)
        nn.init.sparse_(self.HPC.weight_hh.data,1)

    def forward(self, input, hiddens):
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2][0]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2][0]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        return [PFC_hidden,HPC_hidden,Re_hidden]

class MyLSTM_RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, batch_size, sparse=5):
        super(MyLSTM_RNN, self).__init__()

        self.hidden_size_PFC = hidden_size
        self.hidden_size_HPC = hidden_size
        self.hidden_size_Re = hidden_size+0
        self.PFC = nn.LSTMCell(self.hidden_size_HPC+self.hidden_size_Re, self.hidden_size_PFC)
        # self.PFC = nn.LSTMCell(hidden_size*2, hidden_size)
        self.HPC = nn.LSTMCell(input_size+self.hidden_size_Re, self.hidden_size_HPC)
        self.Re = nn.RNNCell(self.hidden_size_HPC+self.hidden_size_PFC, self.hidden_size_Re)
        self.linear = nn.Linear(self.hidden_size_HPC, output_size)
        self.batch_size = batch_size
        sparse = 0.1*int(sparse)
        nn.init.sparse_(self.PFC.weight_ih.data,sparse)
        nn.init.sparse_(self.HPC.weight_ih.data,sparse)
        nn.init.sparse_(self.PFC.weight_hh.data,sparse)
        nn.init.sparse_(self.HPC.weight_hh.data,sparse)

    def forward(self, input, hiddens):
        input = input.float()
        Re_input = torch.cat([hiddens[0][0],hiddens[1][0]],dim=1)
        Re_hidden = self.Re(Re_input, hiddens[2])
        HPC_input = torch.cat([input,hiddens[2]],dim=1)
        HPC_hidden = self.HPC(HPC_input, hiddens[1])
        PFC_input = torch.cat([hiddens[1][0],hiddens[2]],dim=1)
        PFC_hidden = self.PFC(PFC_input, hiddens[0])
        output = self.linear(HPC_hidden[0])
        return output, [PFC_hidden, HPC_hidden, Re_hidden]

    def initHidden(self):
        HPC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        PFC_hidden = [torch.zeros(self.batch_size, self.hidden_size), torch.zeros(self.batch_size, self.hidden_size)]
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size)
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_rand(self):
        const = 0.1
        var = 0.1
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*const
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def initHidden_test(self):
        const = 0.2
        var = 0.2
        HPC_hidden = [torch.rand(self.batch_size, self.hidden_size_HPC)*const, torch.rand(self.batch_size, self.hidden_size_HPC)*const]
#         HPC_hidden = [torch.ones(self.batch_size, self.hidden_size_HPC)*const, torch.ones(self.batch_size, self.hidden_size_HPC)*const]
        PFC_hidden = [torch.rand(self.batch_size, self.hidden_size_PFC)*var, torch.rand(self.batch_size, self.hidden_size_PFC)*var]
#         PFC_hidden = [torch.ones(self.batch_size, self.hidden_size_PFC)*const, torch.ones(self.batch_size, self.hidden_size_PFC)*const]
#         Re_hidden = torch.ones(self.batch_size, self.hidden_size_Re)*const
        Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*var
        return [PFC_hidden,HPC_hidden,Re_hidden]
    
    def noiseHidden_rand(self, hiddens):
        c = 0
        v = 0.1
        HPC_hidden = torch.randn(self.batch_size, self.hidden_size_HPC)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.randn(self.batch_size, self.hidden_size_PFC)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_select(self, hiddens):
        c = 0
        v = 0.01
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
        Re_hidden[:,index] += torch.randn(self.batch_size, index.size)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens
    
    def noiseHidden_dis(self, hiddens, statr, statl):
        c = 0
        v =-0.1
        index = np.array([1,2,8,9,17])
        HPC_hidden = torch.zeros(self.batch_size, self.hidden_size_HPC)
        HPC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[1][0].add_(HPC_hidden)
        PFC_hidden = torch.zeros(self.batch_size, self.hidden_size_PFC)
        PFC_hidden[:,index] += torch.randn(self.batch_size, index.size)*c
        hiddens[0][0].add_(PFC_hidden)
        Re_hidden = torch.randn(self.batch_size, self.hidden_size_Re)*0.0
        Re_hidden += (-statr+statl)*v
#         Re_hidden = torch.rand(self.batch_size, self.hidden_size_Re)*v
#         Re_hidden[:,index] = torch.zeros(self.batch_size, index.size)*c
#         Re_hidden = torch.zeros(self.batch_size, self.hidden_size_Re)*c
#         Re_hidden[:,index] += hiddens[2][:,index]*v
        hiddens[2].add_(Re_hidden)
        return hiddens


def moving_average(data):
    y = np.ones(5)/5
    mean_seq = np.convolve(data, y)
    return mean_seq    


def main(model):
    training_size = 100
    test_size = 1000
    epochs_num = 10
    hidden_size = 20
    batch_size = 10
    data_length = 100
    inputsize = 2
    outputsize = 2
    delay_length = 3
    
    model_path = model
    
    colors = ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
    
#     delay_length = np.random.randint(1, 3)
    delay_length = 2
    train_x1,train_x2,train_y1,train_y2 = mkOwnDataSet_auto(training_size,delay_length)

#     rnn = MyLSTM_comp(inputsize, hidden_size, outputsize, batch_size)
    rnn = MyLSTM_RNN(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyLSTM_RNN_uniHPC(inputsize, hidden_size, outputsize, batch_size)
#     rnn = MyMTRNN2(inputsize, hidden_size, outputsize, batch_size)

    rnn.load_state_dict(torch.load(model_path))
    
    
    pattern = 1
    if  pattern == 1:
        data = mkOwnRandomBatch(train_x1, train_x1, batch_size).float()
    elif pattern == 2:
        data = mkOwnRandomBatch(train_x2, train_x2, batch_size).float()
    elif pattern == 3:
        data = mkOwnRandomBatch(train_y1, train_y1, batch_size).float()
    else:
        data = mkOwnRandomBatch(train_y2, train_y2, batch_size).float()
    
    params = []
    for p in rnn.parameters():
        params.append(p.data)
    
        
#     model_path = 'model/R20_131/ReModel_L2_interRNNrand_OUT1_131_s6_100_1_epoch125.pth'
    model_path = model
    rnn.load_state_dict(torch.load(model_path))
        
    for n, p in rnn.named_parameters():
            if n == "PFC.weight_ih":
                PFC_w_pre = torch.clone(p.data)
            if n == "HPC.weight_ih":
                HPC_w_pre = torch.clone(p.data)
            if n == "Re.weight_ih":
                Re_w_pre = torch.clone(p.data)
            if n == "Re.weight_hh":
                Re_inw = torch.clone(p.data)
            if n == "linear.weight":
                OUT_w_pre = torch.clone(p.data)
                   
    traj = []
    PFCstate = []
    HPCstate = []
    Restate = []
    Gate_states = []
    hidden = rnn.initHidden_rand()
    data_limit = 120
    est_length = 0
    init_point = torch.rand(10,2)*1
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj.append(output.tolist())
            PFCstate.append(hidden[0][0].tolist())
            HPCstate.append(hidden[1][0].tolist())
            Restate.append(hidden[2][0].tolist())
            Gate_states.append(Culc_gate(output,params,hidden))
#             Gate_states.append(Culc_gate_uniPFC(output,params,hidden))
#     for k in range(data.shape[0]*est_length):
#             output,hidden = rnn(output,hidden)
#             traj.append(output.tolist())
#             PFCstate.append(hidden[0][0].tolist())
#             HPCstate.append(hidden[1][0].tolist())
#             Restate.append(hidden[2][0].tolist())
#             Gate_states.append(Culc_gate(output,params,hidden))
    traj = torch.tensor(traj)
    traj = torch.squeeze(traj).numpy()
    
    pattern = 3
    if  pattern == 1:
        data = mkOwnRandomBatch(train_x1, train_x1, batch_size).float()
    elif pattern == 2:
        data = mkOwnRandomBatch(train_x2, train_x2, batch_size).float()
    elif pattern == 3:
        data = mkOwnRandomBatch(train_y1, train_y1, batch_size).float()
    else:
        data = mkOwnRandomBatch(train_y2, train_y2, batch_size).float()
    
    dividenum = int(np.array(PFCstate)[0:,0].shape[0]/2)
    traj_noise = []
    PFCstate_noise = []
    HPCstate_noise = []
    Restate_noise = []
    Gate_states_noise = []
    hidden = rnn.initHidden_rand()
#     data = mkOwnRandomBatch(train_y, batch_size)
#     init_point = torch.rand(10,2)*1
#     data_limit = 125
    for k in range(data_limit):
            #print(data[k].shape)
            output,hidden = rnn(data[k],hidden)
#             output,hidden = rnn(init_point,hidden)
            traj_noise.append(output.tolist())
            PFCstate_noise.append(hidden[0][0].tolist())
            HPCstate_noise.append(hidden[1][0].tolist())
            Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
#     for k in range(data.shape[0]*est_length):
# #             hidden = rnn.noiseHidden_rand(hidden)
#             output,hidden = rnn(output,hidden) 
# #             hidden = rnn.noiseHidden_dis(hidden, np.array(Restate)[k+data_limit], np.array(Restate)[dividenum+k+data_limit])
# #             hidden = rnn.noiseHidden_rand(hidden)
#             traj_noise.append(output.tolist())
#             PFCstate_noise.append(hidden[0][0].tolist())
#             HPCstate_noise.append(hidden[1][0].tolist())
#             Restate_noise.append(hidden[2][0].tolist())
#             Gate_states_noise.append(Culc_gate(output,params,hidden))
    traj_noise = torch.tensor(traj_noise)
    traj_noise = torch.squeeze(traj_noise).numpy()

    
    print(np.array(PFCstate)[:,0].shape)
#     MakeAnimation(pltdata[:,0,0],pltdata[:,0,1], traj[:,0,0], traj[:,0,1], data_limit)
    #MakeAnimation_img(np.array(PFCstate)[:,0],"PFC")
    #MakeAnimation_img(np.array(HPCstate)[:,0],"HPC")
    
    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
    dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
#     dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("PFC correlation")
#     PFC_activity = np.abs(feature[:data_limit,0]-feature[data_limit:,0])
#     PFC_activity = np.abs(moving_average(feature[:data_limit,0])-moving_average(feature[data_limit:,0]))
#     PFC_activity = np.abs((feature[:data_limit,0]-moving_average(feature[:data_limit,0])[2:-2])-(feature[data_limit:,0]-moving_average(feature[data_limit:,0])[2:-2]))
    PFC_activity = culc_grad(moving_average(feature[:data_limit,0]))
    
#     plt.figure()
#     plt.plot(feature[:data_limit,0],color="b")
#     plt.plot(feature[data_limit:,0],color="r")
#     plt.plot(moving_average(feature[:data_limit,0]),":",color="b")
#     plt.plot(moving_average(feature[data_limit:,0]),":",color="r")
#     plt.show()
    
    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
    dfs = np.concatenate((np.array(HPCstate)[:,0],np.array(HPCstate_noise)[:,0]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("HPC correlation")
#     HPC_activity = np.abs(feature[:data_limit,0]-feature[data_limit:,0])
    HPC_activity = culc_grad(moving_average(feature[:data_limit,0]))


    pca = PCA()
#     dfs = np.array(PFCstate)[:,0]
#     dfs = np.concatenate((np.array(PFCstate)[:,0],np.array(PFCstate_noise)[:,0]),axis=0)
    dfs = np.concatenate((np.array(Restate)[:],np.array(Restate_noise)[:]),axis=0)
    print(dfs.shape)
    pca.fit(dfs)
    feature = pca.transform(dfs)
    print("Re correlation")
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,0]-feature[100:,0])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,1]-feature[100:,1])))
#     print(np.corrcoef(np.abs(traj[:,0,0]-traj_noise[:,0,0]),np.abs(feature[:100,2]-feature[100:,2])))
#     Re_activity = np.abs(feature[:data_limit,0]-feature[data_limit:,0])
    Re_activity = np.abs(moving_average(feature[:data_limit,0])-moving_average(feature[data_limit:,0]))
#     Re_activity = np.abs((feature[:data_limit,0]-moving_average(feature[:data_limit,0])[2:-2])-(feature[data_limit:,0]-moving_average(feature[data_limit:,0])[2:-2]))
#     Re_activity = culc_grad(moving_average(feature[:data_limit,0]))
    
#     plt.figure()
#     plt.plot(feature[:data_limit,0],color="b")
#     plt.plot(feature[data_limit:,0],color="r")
#     plt.plot(moving_average(feature[:data_limit,0]),":",color="b")
#     plt.plot(moving_average(feature[data_limit:,0]),":",color="r")
#     plt.show()


    
    return PFC_activity, HPC_activity, Re_activity
#     return np.mean([PFC_a,PFC_b,PFC_c]),np.mean([HPC_a,HPC_b,HPC_c]),np.max([PFC_d]),HPC_d
#     return np.mean([PFC_a,PFC_b,PFC_c]),np.mean([HPC_a,HPC_b,HPC_c]),np.max([PFC_d]), HPC_e, frac_amp
#     return np.mean([PFC_a,PFC_b,PFC_c]),np.mean([HPC_a,HPC_b,HPC_c]),np.max([PFC_d]), HPC_e, np.sum(coherence_diff_list)
#     return np.max([PFC_a,PFC_b,PFC_c]),np.max([HPC_a,HPC_b,HPC_c])


if __name__ == '__main__':
    fig = plt.figure()
    correlation_fig = fig.subplots()
#     correlation_fig.set_xlim(-1,1)
#     correlation_fig.set_ylim(0,1)

#     model_list = glob.glob('model/R20_131/*OUT1**s8_100_2_*epoch*.pth')
#     model_list = sorted(model_list)
#     model_list = sorted(model_list,key=len,reverse=False)
#     k=0
#     for model in model_list:
#         print(model,PFC,HPC)
#         PFC,HPC = main(model)
# #         correlation_fig.plot(PFC,HPC,"o")
# #         correlation_fig.text(PFC,HPC,model.split("epoch")[-1].split(".")[0])
#         correlation_fig.plot(k*5,PFC/HPC,"o")
# #         correlation_fig.text(k*5,PFC/HPC,model.split("epoch")[-1].split(".")[0])
#         k+=1

    allRe_list = []
    allHPC_list = []
    allPFC_list = []
    for num in range(3):
        for i in range(10):
            path = 'model/R20_H_bigbatch/'
#             path = 'model/R20_H_uniHPC_bigbatch/'
            model_list = glob.glob(path+'*s'+str(i+1)+'_100_'+str(num+1)+'_*epoch*.pth')
            model_list = sorted(model_list)
            model_list = sorted(model_list,key=len,reverse=False)
            Re_list = []
            ratio_list_max = []
            with open(path+"good_list.txt", mode="r") as f:
                good_list = f.read().splitlines()
#             good_list = []
#             if i+1 == 4 and num+1 == 3:
#                 continue
#             if i+1 == 5 and num+1 == 2:
#                 continue
            
            first_goodmodel = [0,0]
            good_flag = False
            k=0
            for model in model_list:
#                 if model in good_list:
#                     pass
#                 else:
#                     continue
                print(model)
                if int(model.split("epoch")[-1].split(".")[0])>299:
                    continue
                PFC_activity, HPC_activity, Re_activity = main(model)
                allRe_list.append(Re_activity)
                allHPC_list.append(HPC_activity)
                allPFC_list.append(PFC_activity)

                k+=1
#             correlation_fig.plot(Re_activity,color="C{}".format(i))
    allRe_list = np.array(allRe_list)
    allHPC_list = np.array(allHPC_list)
    allPFC_list = np.array(allPFC_list)
    print(allRe_list.shape)
    correlation_fig.plot(np.mean(allRe_list,axis=0).T,color="C0")
    correlation_fig.errorbar(np.arange(allRe_list.shape[1]),np.mean(allRe_list,axis=0).T,yerr=np.sqrt(np.var(allRe_list,axis=0).T), color="C0", alpha=0.3)
#     correlation_fig.plot(np.mean(allHPC_list,axis=0).T,color="C1")
#     correlation_fig.errorbar(np.arange(allHPC_list.shape[1]),np.mean(allHPC_list,axis=0).T,yerr=np.sqrt(np.var(allHPC_list,axis=0).T), color="C1", alpha=0.3)
#     correlation_fig.plot(np.mean(allPFC_list,axis=0).T,color="C2")
#     correlation_fig.errorbar(np.arange(allPFC_list.shape[1]),np.mean(allPFC_list,axis=0).T,yerr=np.sqrt(np.var(allPFC_list,axis=0).T), color="C2", alpha=0.3)

#             allratio_list.append(np.array(ratio_list_max))

        